# Extract the Phenotype Data from UK BioBank
Author: Graeme Newton - graeme.newton@bristol.ac.uk / gwtnewton@outlook.com \
Date: 10/05/2024

This code should be run via a jupytrlab notebook on a Spark cluster on DNAnexus. \
Errors in this code are likely due to an undersized compute instance. If you encounter issues, please try select a compute instance with a larger RAM and CPU count. The majority of the outputs have been suppressed using a trailing ";" at the end of each line, to see the output from a particular line, assuming there is one, delete the ";". The SQL queries are designed to pull the correct dataset name from your current DNAnexus project. As such, the code should run and provide you with the data used to generate our phenotypes.

Recommended instance: 2 x mem1_ssd1_v2_x4 \
Approximate runtime: 30 minutes \
Approximate cost: £0.10

In [1]:
# Load the required packages and intialise Spark session - ONLY RUN THIS ONCE AT THE START
import pyspark;
import dxpy;
import dxdata;
import csv;
sc = pyspark.SparkContext();
spark = pyspark.sql.SparkSession(sc);

In [2]:
# Find the database file in your DNAnexus project - ONLY RUN THIS ONCE AT THE START
dispensed_database_name = dxpy.find_one_data_object(classname="database", name="app*", folder="/", name_mode="glob", describe=True)["describe"]["name"];
dispensed_dataset_id = dxpy.find_one_data_object(typename="Dataset", name="app*.dataset", folder="/", name_mode="glob")["id"];
rx_data_name = f"{dispensed_database_name}.gp_scripts"
spark.sql("USE " + dispensed_database_name); 
# spark.sql("SHOW TABLES").show(truncate=False); # Unblank if you want to see a list of available datasets
dataset = dxdata.load_dataset(id=dispensed_dataset_id);
participant = dataset["participant"];

## Extract Unique Drug Prescriptions (Optional)
This is the code to extract a list of the unique drugs in the UKB Rx dataset. This was used to see all possible drugs that could be extracted. From this list, we then select the analgesic/anti-neuropathic pain medications used for the prescription-based phenotypes in Newton et al. 2024.

In [ ]:
# Pull the prescription data from the Rx database and preview
df = spark.sql(f"SELECT eid, issue_date, bnf_code, dmd_code, drug_name FROM {rx_data_name} WHERE drug_name IS NOT NULL")
df.show(truncate = False)
df.count()

# Select the unique drug names, preview, and count the number of unique drugs
distinctDF = df.select("drug_name").distinct()
distinctDF.show(truncate=False)
distinctDF.count()

# Convert DataFrame to a list of dictionaries
data_dicts = [
    {"drug_name": row.drug_name}
    for row in distinctDF.rdd.toLocalIterator()
]
csv_file_path = "UKB_Rx_unique_drugs.csv"

#  Write the data
with open(csv_file_path, "w", newline="") as csv_file:
    fieldnames = ["drug_name"]

    writer = csv.DictWriter(csv_file, fieldnames=fieldnames)

    # Write header
    writer.writeheader()

    # Write rows with progress indication
    for idx, data_dict in enumerate(data_dicts):
        writer.writerow(data_dict)

print("Unique drugs CSV file written successfully.")

## Extract Analgesic and Anti-Neuropathic Drug Prescriptions
This is the code to extract the analgesic and anti-neuropathic drugs from the dataset used in phenotype generation.

In [ ]:
# list of drugs was generated from the unique scripts data based on regex patterns of desired drugs.
drugs = "'Paracetamol 500mg tablets', 'ASPIRIN disp tab 75mg', 'Venlafaxine 150mg modified-release capsules', 'Dipyridamole 200mg modified-release / Aspirin 25mg capsules', 'Sodium Valproate  M/R tablets  300 mg', 'Diclofenac 1.16% gel', 'DICLOFENAC SODIUM & MISOPROSTOL tabs 75mg + 200mcg', 'Amitriptyline Hydrochloride  Tablets  25 mg', 'Pregabalin 200mg capsules', 'Naproxen 250mg gastro-resistant tablets', 'Aspirin Dispersible TABS 75MG', 'CO-CODAMOL 30MG/500MG TABLETS', 'Diclofenac sodium 75mg modified-release tablets', 'Amitriptyline Hydrochloride  Tablets  10 mg', 'Tramadol Hydrochloride  Capsules  50 mg', 'Ibuprofen 10% gel', 'Oxycodone 5mg capsules', 'Depo-Medrone with Lidocaine suspension for injection 2ml vials (Pfizer Ltd)', 'DICLOFENAC SODIUM SR TAB 75mg', 'IBUPROFEN gel 5%', 'Tramadol 37.5mg / Paracetamol 325mg tablets', 'Co-codamol 30mg/500mg caplets (Kent Pharmaceuticals Ltd)', 'Codeine Linctus  Syrup  15 mg/5 ml', 'Co-codamol 15mg/500mg capsules', 'Aspirin  Dispersible Tablets  75 mg', 'DICLOFENAC SODIUM ec tab 25mg', 'PARACETAMOL caplets 500mg [ASPAR]', 'Lamotrigine 100mg tablets', 'Apomorphine 2mg sublingual tablets sugar free', 'Amitriptyline Hydrochloride  Tablets  50 mg', 'Tramadol Hydrochloride CAPS 50MG', 'Aspirin 300mg tablets', 'Fentanyl 100micrograms/hour transdermal patches', 'Gabapentin 600mg tablets', 'Paracetamol  Caplets  500 mg', 'Ibuprofen  Gel  10 %', 'Codeine Phosphate TABS 30MG', 'Buprenorphine 5micrograms/hour transdermal patches', 'DEPO-MEDRONE WITH LIDOCAINE injection 40mg/ml + 10mg/ml [PHARMACIA]', 'CO-CODAMOL eff tab 8mg+500mg', 'DICLOFENAC SODIUM & MISOPROSTOL tabs 50mg + 200micrograms', 'DIHYDROCODEINE tabs 30 mg', 'Fentanyl 75micrograms/hour transdermal patches', 'CARBAMAZEPINE RETARD TAB 400', 'Aspirin  Dispersible tablets  300 mg', 'Lidocaine 5% / Hydrocortisone acetate 0.275% ointment', 'Lidocaine 2% and Chlorhexidine gel', 'CODEINE PHOSPHATE sf linc 15mg/5ml', 'Mefenamic acid 500mg tablets', 'IBUPROFEN tabs 200mg', 'Amitriptyline 10mg tablets (Actavis UK Ltd)', 'CO-CODAMOL disp tab 8mg+500mg', 'IBUPROFEN TAB 400', 'Lidocaine 2.5% / Prilocaine 2.5% cream', 'NAPROXEN EC TABLETS 500MG', 'Aspirin  Tablets  300 mg', 'NAPROXEN ec tab 500mg', 'CO-CODAMOL 8MG/500MG TABLETS', 'Diclofenac 75mg/3ml solution for injection ampoules', 'Nortriptyline  Tablets  10 mg', 'Dihydrocodeine  Tablets  30 mg', 'Co-Codamol 8/500  Capsules', 'Ibuprofen  Tablets  200 mg', 'CARBAMAZEPINE TAB 100', 'Duloxetine  Gastro-resistant Capsules  20 mg', 'NAPROXEN tabs 250mg', 'CO-CODAMOL eff tab 30mg + 500mg', 'Lidocaine 20mg/2ml (1%) solution for injection ampoules', 'Mefenamic acid 250mg capsules', 'Ibuprofen  Gel  5 %', 'CO-CODAMOL 30/500MG CAPS', 'Naproxen  E/C Tablets  500 mg', 'Ibuprofen 200mg capsules', 'Methylprednisolone 80mg/2ml / Lidocaine 20mg/2ml (1%) suspension for injection vials', 'Diclofenac Sodium  E/C Tablets  50 mg', 'Venlafaxine  M/R capsules  75 mg', 'CELECOXIB caps 100mg', 'Codeine Phosphate  Tablets  30 mg', 'Celecoxib CAPS 100MG', 'Pregabalin  Capsules  150 mg', 'IBUPROFEN tabs 400 mg', 'Lidocaine 1% / Hydrocortisone 0.2% spray', 'PARACETAMOL + CODEINE PHOSPHATE caps 500mg + 30mg', 'Oxycodone 20mg modified-release tablets', 'Morphine 30mg modified-release tablets', 'Ibuprofen 10% gel (Thornton & Ross Ltd)', 'Paracetamol  Soluble tablets  500 mg', 'DICLOFENAC SODIUM mr tab 75mg', 'diamorphine hydrochloride powder for solution for injection 5mg', 'Amitriptyline tablets 10mg', 'CO-CODAMOL (codeine & paracetamol) tabs 15mg + 500mg', 'Diclofenac Sodium Ec TABS 25MG', 'IBUPROFEN TABLETS 800MG', 'Morphine Sulphate  M/R tablets  30 mg', 'Oxycodone 20mg capsules', 'Diclofenac 100mg suppositories (A A H Pharmaceuticals Ltd)', 'Diclofenac sodium 75mg gastro-resistant modified-release capsules', 'DICLOFENAC SODIUM dual release caps 75mg', 'LIDOCAINE injection preservative-free 1% [HAMELN]', 'Oxycodone 30mg modified-release tablets', 'Lidocaine 0.33% / Cetalkonium chloride 0.01% oromucosal gel sugar free', 'Buprenorphine 400microgram sublingual tablets sugar free (Teva UK Ltd)', 'Tramadol 150mg modified-release capsules', 'Aspirin E/C Tablets 300 mg', 'Mirtazapine TABS 45MG', 'Morphine 200mg modified-release tablets', 'PARACETAMOL TABS 500MG', 'diclofenac sodium EC tablets 25mg', 'DICLOFENAC SODIUM AQUEOUS GEL 1%', 'Tramadol Hydrochloride  M/R capsules  200 mg', 'Diclofenac 0.1% eye drops 0.3ml unit dose preservative free', 'Paracetamol  Suppositories  500 mg', 'Buprenorphine 400microgram sublingual tablets sugar free', 'PHENYTOIN TABLETS 100 MG', 'Tramadol 50mg soluble tablets sugar free', 'Codeine phosphate Tablets 30 mg', 'CO-CODAMOL 30MG/500MG CAPLET TABLETS', 'IBUPROFEN TABLETS 200MG', 'Ibuprofen 200mg caplets (Wockhardt UK Ltd)', 'AMITRIPTYLINE TAB 10mg', 'Nortriptyline  Tablets  25 mg', 'NORTRIPTYLINE tabs 10mg', 'Diclofenac Retard  Tablets  100 mg', 'NAPROXEN EC TABLETS 375MG', 'Pregabalin CAPS 100MG', 'Lidocaine 400mg/20ml (2%) / Adrenaline (base) 100micrograms/20ml (1 in 200,000) solution for injection vials', 'Oxycodone 5mg / Naloxone 2.5mg modified-release tablets', 'Amitriptyline hydrochloride tablets 25mg', 'VENLAFAXINE MR CAPSULES 150MG', 'PREGABALIN caps 200mg', 'DIHYDROCODEINE 30MG TAB', 'AMITRIPTYLINE HCl mr cap 50mg', 'IBUPROFEN 400mg tablets', 'INDOMETHACIN caps 50mg', 'DICLOFENAC POTASSIUM tabs 25mg', 'Carbamazepine 100mg/5ml oral suspension sugar free (Actavis UK Ltd)', 'Codeine 15mg tablets (Actavis UK Ltd)', 'Aspirin 75mg dispersible tablets (A A H Pharmaceuticals Ltd)', 'Venlafaxine 225mg modified-release capsules', 'Diclofenac sodium 50mg gastro-resistant tablets (Teva UK Ltd)', 'codeine phosphate diabetic linctus 15mg/5ml', 'TRAMADOL 12hr mr tabs 100mg', 'Naproxen  E/c tablets  500 mg', 'CO-CODAMOL 530 MG tabs 500 mg paracetamol + 30 mg codeine', 'Diclofenac Potassium TABS 50MG', 'DICLOFENAC SODIUM DISPERSIBLE TABLETS 50MG', 'Lamotrigine 100mg dispersible tablets sugar free', 'Venlafaxine 37.5mg modified-release capsules', 'CARBAMAZEPINE TABLETS 100MG', 'DIHYDROCODEINE TARTRATE TABLETS 40MG', 'MEFENAMIC ACID CAP 250mg', 'Gabapentin 800mg tablets', 'Tramadol 300mg modified-release tablets', 'DULOXETINE gastro-res cap 60mg', 'PHENYTOIN caps 300mg', 'MIRTAZAPINE TABLETS 30MG', 'Co-codamol 12.8mg/500mg tablets', 'Celecoxib 100 mg capsules', 'PHENYTOIN caps 100mg', 'Lidocaine  Ointment  5 %', 'NAPROXEN TAB 500', 'Paracetamol 500mg caplets (Vantage)', 'Diclofenac 1% gel', 'MIRTAZAPINE TABLETS 45MG', 'BUPRENORPHINE 35 MICROGRAMS/HOUR PATCH', 'Celecoxib 200 mg capsules', 'Celecoxib  Capsules  100 mg', 'DICLOFENAC SODIUM ENTERIC COATED TABLETS 50MG', 'PARACETAMOL SOLUBLE TABLETS 500MG', 'Phenytoin 300mg capsule', 'Paracetamol 500mg soluble tablets (Kent Pharmaceuticals Ltd)', 'AMITRIPTYLINE TABS 10 MG', 'PARACETAMOL + METOCLOPRAMIDE sach 500mg+5mg', 'CODEINE PHOSPHATE TAB 60', 'Lidocaine 50mg/5ml (1%) solution for injection ampoules', 'Ibuprofen Tablets 400mg', 'CO-CODAMOL TABLETS 8/500', 'PHENYTOIN SODIUM caps 50mg', 'Gabapentin  Tablets  800 mg', 'DICLOFENAC SODIUM TAB 50', 'MORPHINE SULPHATE mr tab 10mg', 'DICLOFENAC modified release capsule 75mg', 'Methadone 1 Mg/Ml  Oral solution', 'phenytoin capsules 100mg', 'Diamorphine 30mg powder for solution for injection ampoules (Wockhardt UK Ltd)', 'Methadone Hydrochloride Sf Oral SOLN 1MG/ML', 'MIRTAZAPINE orodisp tab 15mg', 'nortriptyline capsules 25mg', 'SODIUM VALPROATE TAB 100mg', 'PARACETAMOL + CODEINE PHOSPHATE tabs 500mg + 8mg', 'PARACETAMOL SOL TABLETS 500MG', 'SODIUM VALPROATE EC tablets 200mg', 'NORTRIPTYLINE TABLETS 10MG', 'Diclofenac sodium 100mg modified-release tablets (A A H Pharmaceuticals Ltd)', 'MORPHINE SULPHATE tabs 10mg', 'Diclofenac Sodium  E/C  M/R Capsules  75 mg', 'IBUPROFEN JEL 5', 'Aspirin 75mg gastro-resistant tablets (Wockhardt UK Ltd)', 'Nortriptyline 10mg tablets (A A H Pharmaceuticals Ltd)', 'LIDOCAINE injection 1% [HAMELN]', 'Oxycodone Hydrochloride  M/R tablets  20 mg', 'Ibuprofen 100mg/5ml oral suspension sugar free', 'Diclofenac sodium enteric coated tablets 50mg', 'Co-codamol 30mg/500mg tablets (IVAX Pharmaceuticals UK Ltd)', 'Ibuprofen [EVANS] tablets 400mg', 'DICLOFENAC SOD_TAB 75MG M/R', 'Paracetamol 500mg / Dihydrocodeine 7.46mg effervescent tablets sugar free', 'Oxycodone Hydrochloride  Capsules  10 mg', 'Fentanyl 200microgram buccal tablets sugar free', 'Paracetamol 500mg capsules (Kent Pharmaceuticals Ltd)', 'Amitriptyline Hydrochloride TABLETS 10MG', 'co-codamol tablets 30mg + 500mg', 'DICLOFENAC EC tabs 50 MG', 'Tramadol Hydrochloride Mr CAPS 100MG', 'ASPIRIN DISPERSIBLE TAB 75mg', 'CO-CODAMOL DISPERSIBLE TABLET 8/500', 'Codeine 30mg tablets (Wockhardt UK Ltd)', 'VENLAFAXINE TABLETS 37.5MG', 'Terpin 32.5mg/5ml / Codeine 16mg/5ml linctus', 'Lidocaine 200mg/20ml (1%) solution for injection vials (Hameln Pharmaceuticals Ltd)', 'diclofenac sodium dispersible tablet 50mg', 'PARACETAMOL TAB 500mg', 'ASPIRIN TAB 75mg', 'BUPRENORPHINE HCl sublingual tab 400micrograms', 'IBUPROFEN TAB 600MG', 'IBUPROFEN mr cap 200mg', 'Co-codamol 30mg/500mg effervescent powder sachets sugar free', 'Ibuprofen 200mg Capsule', 'DICLOFENAC tabs s/r 75 mg', 'Diclofenac modified release tablet 75mg', 'CODEINE PHOS/IBUPROF TAB', 'Diclofenac Sodium  M/R tablets  100 mg', 'CO-CODAMOL (codeine & paracetamol) eff tab 30mg + 500mg', 'NAPROXEN EC TAB 500mg', 'Lidocaine With Chlorhexidine Syringe 6ml GEL', 'LIDOCAINE HYDROCHLORIDE injection 2% [AMCO]', 'Etoricoxib  Tablets  90 mg', 'NAPROXEN 500mg tablets', 'CODEINE (Generic Manuf) (form not specified)', 'DICLOFENAC SODIUM MULTIDOSE EYE DROPS 0.1%', 'INDOMETHACIN capsules 25mg', 'DICLOFENAC ec tabs 25mg', 'CELECOXIB 100MG CAPSULES', 'Depomedrone With Lidocaine Vial 2ml INJ 40MG/ML', 'DICLOFENAC SODIUM 75mg m/r tablets', 'Lidocaine 200mg/20ml (1%) solution for injection ampoules', 'Naproxen 500mg tablets (Teva UK Ltd)', 'LAMOTRIGINE tabs 100mg', 'PARACETAMOL tablets 500mg [ALMUS]', 'ASPIRIN 300mg e/c tablets', 'NAPROXEN E.C.', 'Lubricating chlorhexidine + lidocaine gel', 'Lubricating chlorhexidine with lidocaine gel Gel', 'DICLOFENAC SODIUM SUPPS 50MG', 'CO-CODAMOL tablets 8mg+500mg', 'Morphine 15mg modified-release tablets', 'CODEINE PHOSPHATE linctus  diabetic 15mg/-P42 0', 'Lidocaine 400mg/20ml (2%) solution for injection vials', 'PARACETAMOL + CODEINE tabs 500mg + 12.8mg', 'Morphine 1mg/5ml / Peppermint oil 1.5microlitres/5ml oral solution', 'Paracetamol with codeine phosphate capsules 500mg + 30mg', 'NU-SEALS ASPIRIN EC tablets 300mg [LILLY]', 'Morphine 60mg modified-release capsules', 'Co-Codamol 30/500 Tablets', 'MEFENAMIC ACID tabs forte 500 mg', 'Amitriptyline Hydrochloride Tablets 10mg', 'Perphenazine 2mg with Amitriptyline 25mg tablet', 'NAPROXEN TABS 500MG', 'Venlafaxine tablets 75mg', 'Co-Codamol 12.8mg/500mg TABS', 'Ibuprofen Sf Oral SUSP 100MG/5ML', 'Mirtazapine  Orodispersible tablets  30 mg', 'MORPHINE SULPHATE mr tab 5mg', 'DIHYDROCODEINE 10mg/ TAB', 'Carbamazepine 100mg tablets (Mylan Ltd)', 'MIRTAZAPINE orodisp tab 30mg', 'Lidocaine 50mg/5ml (1%) solution for injection ampoules (Martindale Pharmaceuticals Ltd)', 'ASPIRIN SOLUBLE TAB 300mg', 'DEPO-MEDRONE + LIDOCAINE inj', 'Ibuprofen 600mg tablets (Boston Healthcare Ltd)', 'DICLOFENAC EC tablets 50mg', 'Aspirin 75mg dispersible tablets (Almus Pharmaceuticals Ltd)', 'Morphine 100mg modified-release granules sachets sugar free', 'NAPROXEN TABLETS 250 MG', 'Lidocaine 0.5% gel', 'Dihydrocodeine Tartrate Tablets 30 mg', 'PHENYTOIN TAB 100mg', 'Paracetamol And Metoclopramide  Tablets  500 mg + 5 mg', 'Paracetamol tablets 500mg', 'Lidocaine And Chlorhexidine  Gel  2 % + 0.25 %', 'Apomorphine Hydrochloride Sls Sublingual TABS 3MG', 'Ibuprofen 200mg / Codeine 12.8mg tablets', 'CODEINE PHOSPHATE TAB 15mg', 'Ibuprofen Tabs 400mg', 'Nortriptyline TABLETS 10MG', 'Lidocaine 2% oromucosal spray', 'morphine sulfate 24 hour modified release capsules 30mg', 'Paracetamol And Metoclopramide  Oral Powder Sachets Sugar Free  500 mg + 5 mg', 'Mirtazapine 15mg/ml oral solution sugar free', 'CARBAMAZEPINE TAB 200mg', 'Celecoxib CAPSULES 200MG', 'LAMOTRIGINE TAB 50mg', 'Hydromorphone 8mg modified-release capsules', 'Methadone 10mg/1ml solution for injection ampoules', 'DICLOFENAC SODIUM TABS 50MG', 'Gabapentin 100mg capsules (Almus Pharmaceuticals Ltd)', 'ASPIRIN EC 75MG CAPSULE', 'Hydromorphone 16mg modified-release capsules', 'CO-CODAMOL 30/500 TABS 500 MG PARACETAMOL + 30MG CODEINE', 'ASPIRIN 75mg disp tabs', 'Ibuprofen 400mg caplets (Bristol Laboratories Ltd)', 'SODIUM VALPROATE TAB 200mg', 'Dihydrocodeine 30mg tablets (Mylan Ltd)', 'AMITRIPTYLINE tablets 25mg [CROSS-PHAR]', 'Pregabalin  Oral Solution  Sugar Free  20 mg/ml', 'Venlafaxine TABS 50MG', 'Tramadol Hydrochloride Mr CAPS 50MG', 'Mirtazapine 45mg tablets (Ranbaxy (UK) Ltd)', 'Paracetamol 500mg capsules (Teva UK Ltd)', 'Mexiletine Hydrochloride CAPS 200MG', 'Diclofenac sodium 75mg modified-release tablets (A A H Pharmaceuticals Ltd)', 'Celecoxib capsules 100mg', 'DICLOFENAC SR 75MG', 'Buprenorphine Sublingual Tablets Sugar Free 200 micrograms', 'DICLOFENAC SODIUM 75MG SR TABLET', 'Diclofenac 4% cutaneous spray', 'AMITRIPTYLINE HYDROCHLORIDE TABS 25MG', 'Ibuprofen 400 mg tablets', 'CO-CODAMOL SOL   1-2', 'CO-CODAMOL (codeine & paracetamol) caps 15mg + 500mg', 'Morphine sulphate 60mg/2ml solution for injection ampoules', 'Morphine sulfate 10mg/1ml suspension for injection vials', 'Hydrocortisone And Lidocaine  Spray  0.2 % + 1 %', 'Pregabalin  Capsules  200 mg', 'PARACETAMOL 500MG/ME TAB', 'AMITRIPTYLINE TAB 50', 'DICLOFENAC SODIUM MR CAPSULES 100MG', 'VENLAFAXINE modified release capsule 75mg', 'PHENYTOIN CAP 300mg', 'MEFENAMIC ACID tablets 500mg', 'Diclofenac Potassium  Tablets  25 mg', 'PREGABALIN CAPSULES 150MG', 'Ibuprofen Capsules 300 mg', 'Mirtazapine 30mg orodispersible tablets (Teva UK Ltd)', 'Sodium valproate 200mg/5ml oral solution', 'SODIUM VALPROATE EC TABLETS 200MG', 'CO-CODAMOL 30/500 EF TAB', 'TRAMADOL mr tab 150mg', 'SODIUM VALPROATE crushable tab 100mg', 'Etoricoxib TABS 30MG', 'Oxycodone Hydrochloride CAPS 10MG', 'Mirtazapine 45mg tablets (A A H Pharmaceuticals Ltd)', 'PARACET 500/CODEINE PHOS 8MG TAB', 'Venlafaxine Hydrochloride  M/R capsules  75 mg', 'NAPROXEN tablets TAB 500mg-P42 0', 'DICLOFENAC POTASSIUM TABLETS 50MG', 'Naproxen 250mg tablets (Teva UK Ltd)', 'DICLOFENAC SODIUM RETARD CAP 100', 'GABAPENTIN CAPSULES 400MG', 'TRAMADOL 12hr mr tabs 150mg', 'ORAMORPH oral solution 10mg/5ml', 'CO-CODAMOL tabs, dispersible 8/500', 'BUPRENORPHINE seven day patch 10micrograms/hour', 'PARACETAMOL 500MG', 'Depomedrone With Lidocaine Vial 1ml INJ 40MG/ML', 'Naproxen And Misoprostol  Tablets  500 mg + 200 micrograms', 'Morphine sulphate 12 30mg Modified-release capsule', 'MORPHINE SULPHATE mr cap 10mg', 'Diclofenac Diethylammonium  Gel  2.32 %', 'Lamotrigine 50mg tablets (Teva UK Ltd)', 'CO-CODAMOL dispersible tablet 8/500', 'Tramadol 50mg modified-release capsules (A A H Pharmaceuticals Ltd)', 'Amitriptyline 25mg tablets (Almus Pharmaceuticals Ltd)', 'Paracetamol TABLETS 500MG', 'CO-CODAMOL TABLETS 8/500-P42 0', 'Aspirin TABLETS 75MG', 'LAMOTRIGINE TABS 25MG', 'Methylprednisolone And Lidocaine  Injection  40 mg/ml + 10 mg/ml  1 ml vial', 'Dihydrocodeine 30mg tablets (Teva UK Ltd)', 'PHENYTOIN SOD CAP 100MG', 'Naproxen 500mg gastro-resistant tablets (Actavis UK Ltd)', 'Ibuprofen 400mg tablets (Boston Healthcare Ltd)', 'Tramadol Hydrochloride  M/R tablets  75 mg', 'Diclofenac Sodium  M/R capsules  100 mg', 'Dexibuprofen 300mg tablets', 'Methadone 20mg/2ml solution for injection ampoules', 'Nu-Seals Aspirin E/C Tablets 75 mg', 'DICLOFENAC suppositories 100 mg', 'DICLOFENAC SOD SR TABS 75MG', 'TRAMADOL capsules 50mg', 'Diclofenac sodium 100 mg modified release capsules', 'DICLOFENAC tablets TAB 50mg-P42 0', 'Paracetamol 500mg tablets (Accord Healthcare Ltd)', 'Buprenorphine  Transdermal patches  5 micrograms/hour (5 mg per patch)', 'Carbamazepine 400mg modified-release tablets (A A H Pharmaceuticals Ltd)', 'SODIUM VALPROATE ec tab 500mg', 'CARBAMAZEPINE tabs 200 mg', 'Fentanyl 1.2mg lozenges', 'INDOMETHACIN SR TAB 75', 'lidocaine hydrochloride with adrenaline 1:200 000 injection-', 'AMITRIPTYLINE 10MG TAB', 'Aspirin Ec TABLETS 75MG', 'Fentanyl 1.6mg lozenges', 'ASPIRIN EFFERVESCENT tabs 100 MG', 'co-dydramol (dihydrocodeine and paracetamol) suspension sugar-free 10mg + 500mg/5ml', 'Co-codamol 30mg/500mg tablets (Sandoz Ltd)', 'ASPIRIN disp tab 75 MG', 'LAMOTRIGINE TABLETS 25MG', 'CELECOXIB 200MG CELECOXIB 200MG CAP', 'Paracetamol 500mg tablets (Wockhardt UK Ltd)', 'buprenorphine (form not specified)', 'Co-Codamol 15/500 Tablets', 'Ibuprofen 400mg', 'CELECOXIB 200MG CAPS', 'CO-CODAMOL EFF 30MG/500MG TAB TAB-P42 0', 'NAPROXEN TAB 375mg', 'OXYCODONE HCl + NALOXONE HCl mr tab 5mg + 2.5mg', 'Gabapentin 250mg/5ml oral solution', 'BUPRENORPHINE 20 MICROGRAMS / HOUR PATCH', 'Depo-medrone with lidocaine injection -', 'Codeine phosphate 30mg with paracetamol 500mg tablets', 'Ibuprofen 5% foam', 'CODEINE LIN', 'CODEINE linctus [HILLCROSS]', 'AMITRIPTYLINE caps s/r 75 mg - DELETED 4/96', 'PARACETAMOL 500MG/METOCLOPROPAM. 5MG SAC', 'Paracetamol  Sugar free suspension  250 mg/5 ml', 'Diclofenac Sodium E/c tablets 50mg', 'Dihydrocodeine Tartrate TABLETS 30MG', 'mirtazapine tablets 30mg', 'Co-codamol Tablets 8mg+50', 'Ibuprofen TABLETS 600MG', 'metoclopramide with aspirin effervescent tablet 5mg + 325mg', 'CODEINE PHOSPHATE TABS 60MG', 'Diclofenac sodium enteric coated tablets 25mg', 'Tapentadol 75mg tablets', 'AMITRIPTYLINE HCl sf oral soln 10mg/5ml', 'Tramadol 150mg modified-release tablets (A A H Pharmaceuticals Ltd)', 'ASPIRIN TABLETS 300MG', 'Carbamazepine  C/R Tablets  200 mg', 'Gabapentin capsules 100mg', 'DIHYDROCODEINE tablets 30mg [BERK]', 'INDOMETHACIN tabs s/r 25 mg', 'Oxycodone Hydrochloride  M/R tablets  40 mg', 'Diclofenac potassium 12.5mg tablets', 'NAPROXEN tabs 250 mg -', 'Paracetamol 500mg / Ibuprofen 200mg tablets', 'Duloxetine Gastro Resistant CAPS 40MG', 'DICLOFENAC dual release cap 75mg', 'Diclofenac Sodium E/C, M/R Capsules 75 mg', 'KAOLIN & MORPHINE MIX', 'morphine sulfate 24 hour modified release capsules 90mg', 'PHENYTOIN tabs 50mg', 'MEFENAMIC ACID TABS FORTE 500 MG', 'DICLOFENAC SR 75MGS', 'ASPIRIN ec tabs 75mg', 'PARACETAMOL 500MG/30 TAB', 'Naproxen 250mg gastro-resistant tablets (Actavis UK Ltd)', 'DICLOFENAC SODIUM 75MG SR TAB', 'Amitriptyline Sr  Capsules  75 mg', 'PHENYTOIN SODIUM/ PHENOBARBITONE CAP', 'IBUPROFEN tablets 400mg [CP PHARM]', 'CODEINE PHOSPHATE 30mg', 'Morphine Sulfate  Injection  10 mg/ml  1 ml ampoule', 'DICLOFENAC DUAL RELE CAP 75', 'Lidocaine 70mg / Tetracaine 70mg cream', 'PHENYTOIN paed tab 50mg', 'DICLOFENAC SODIUM supp 25mg', 'ASPIRIN TABS 300MG', 'Paracetamol 500mg with dihydrocodeine 10mg tablet', 'Isometheptene mucate with paracetamol 65mg+325mg Capsule', 'Ibuprofen 200mg tablets (Galpharm International Ltd)', 'Co-codamol 8mg/500mg tablets (Aspar Pharmaceuticals Ltd)', 'INDOMETHACIN CAPS 25MG', 'Morphine Sulphate  M/R tablets  60 mg', 'PARACETAMOL + DIHYDROCODEINE tabs 500mg + 30mg', 'Carbamazepine  C/r tablets  200 mg', 'Carbamazepine  C/r tablets  400 mg', 'Tramadol capsules 50mg', 'Co-codamol 30mg/500mg effervescent tablets (Teva UK Ltd)', 'Tapentadol 250mg modified-release tablets', 'Etoricoxib TABLETS 120MG', 'Paracetamol Tablets 500mg', 'LIDOCAINE HCl gel 1%', 'Ibuprofen 100mg/5ml', 'ASPIRIN 75MG TABLET', 'NAPROXEN susp 25mg/ml', 'phenytoin sodium tablets 100mg', 'PARACETAMOL + CODEIN PHOSPHAT TABS 500MG+30MG', 'DICLOFENAC 50 MG TAB', 'Co-codamol 30mg/500mg caplets (Wockhardt UK Ltd)', 'ASPIRIN-P42 75MG TABLET', 'Codeine sugar free linctus 15mg/5ml', 'Paracetamol 120mg/5ml oral suspension paediatric', 'PARACETAMOL 500MG/30 CAP', 'Paracetamol  Orodispersible Tablets (Sugar Free)  250 mg', 'CARBAMAZEPINE TAB 200MG', 'Diclofenac-P42 50mg tab EC', 'Mirtazapine 15mg orodispersible tablets (Teva UK Ltd)', 'aspirin dispersible tablet 75 mg', 'FENTANYL patch 50mcg/hour', 'Methylprednisolone acetate with lidocaine injection 40mg/ml', 'Lacosamide  Tablets  100 mg', 'codeine phosphate tablets 15mg', 'Fentanyl 25 Micrograms/Hour PATCH', 'Lidocaine 0.66% / Aminoacridine 0.05% oromucosal gel suga...', 'AMITRIPTYLINE HCL TAB 10MG', 'Paracetamol 325mg with dextropropoxyphene 32.5mg tablet', 'Tylex Capsules 500 mg para. + 30mg codeine', 'NAPROXEN EC TAB 500MG', 'Etoricoxib  Tablets  120 mg', 'INDOMETHACIN caps s/r 75 mg', 'CO-CODAMOL EFF 30MG/500MG TAB', 'Ibuprofen  Fc tablets  800 mg', 'Sodium Valproate  Sugar-free liquid  200 mg/5 ml', 'INDOMETHACIN CAPS 50MG', 'MIRTAZAPINE TABLETS 15 MG', 'LIDOCAINE + AMINOACRIDINE gel 0.66% + 0.05%', 'DICLOFENAC SOD_TAB 100MG M/R', 'LIDOCAINE HYDROCHLORIDE injection 2% [MERCURY]', 'ASPIRIN DISP TAB 300MG', 'DICLOFENAC 100 MGS 100 MGS.', 'PARACETAMOL WITH CODEINE PHOSPHATE tablets 500m', 'DICLOFENAC 75mg/3mL injection', 'DICLOFENAC MODIFIED RELEASE TABLET 100 MG', 'NAPROXEN cr tab 500mg', 'Paracetamol  Oral solution  500 mg/5 ml', 'VENLAFAXINE 150mg m/r capsules', 'NAPROXEN 375mg e/c tablets', 'BUPRENORPHINE four day patch 35mcg/hour', 'diclofenac modified release tablet 75mg', 'Diclofenac sodium 25mg gastro-resistant tablets (Teva UK Ltd)', 'Depo-Medrone With Lidocaine  Injection, 2 Ml Vial', 'CARBAMAZEPINE liq 100mg/5ml', 'Sodium Valproate Crushable TABS 100MG', 'Codeine phosphate-P42 60mg tablet', 'Aspirin dispersible tablet 300mg', 'Diclofenac Sodium  Tablets  50 mg', 'Fentanyl 200microgram sublingual tablets sugar free', 'IBUPROFEN syrp 100mg/5ml', 'Buprenorphine  Transdermal patches  70 micrograms/hour (40 mg per patch)', 'Dihydrocodeine 30mg tablets (A A H Pharmaceuticals Ltd)', 'Venlafaxine TABLETS 37.5MG', 'Diclofenac Sodium Enteric Coated Tablets 50mg', 'Carbamazepine Mr TABS 400MG', 'codeine phosphate linctus 15mg/5ml', 'sodium valproate with valproic acid modified release granules 50mg', 'Lamotrigine 5mg dispersible tablets sugar free (Teva UK Ltd)', 'Fentanyl 25micrograms/hour transdermal patches (Actavis UK Ltd)', 'Aspirin enteric coated tablets 300mg', 'MORPHINE SULPHATE oral soln 30mg/5ml', 'Mefenamic Acid Tab 500mg', 'Mirtazapine 45mg tablets (Teva UK Ltd)', 'DICLOFENAC suppositories 50 mg', 'HYDROMORPHONE HCl caps 1.3mg', 'IBUPROFEN TABS 800MG', 'Paracetamol 500mg with codeine 12.8mg tablet', 'Cathejell with Lidocaine (Teleflex Medical)', 'Tramadol hydrochloride 50 mg capsules', 'TRAMADOL modified release tablet 100mg', 'CO-CODAMOL tablets 8/500', 'PARACETAMOL WITH CODEINE PHOSPHATE capsules 500mg + 30mg', 'PARACETAMOL TABS, SOLUBLE  500 MG', 'PARACETAMOL sf susp 250mg/5ml', 'Lidocaine 100mg/10ml (1%) solution for injection Sure-Amp am', 'PARACETAMOL WITH CODEINE PHOSPHATE effervescent tablet 500mg', 'AMITRIPTYLINE TAB 10MG', 'Carbamazepine-P42 100mg tablet', 'INDOMETHACIN MR CAP 75MG', 'Amitriptyline Hydrochloride  Capsules  25 mg', 'Morphine Sulfate M/R Capsules 10 mg', 'Amitriptyline hydrochloride tablets 10mg', 'Co-codamol 8mg/500mg tablets (Kent Pharmaceuticals Ltd)', 'Tramadol Hydrochloride M/R Capsules 50 mg', 'Methadone 10mg/1ml solution for injection ampoules (Martindale Pharmaceuticals Ltd)', 'Dihydrocodeine Tablets 30 mg', 'co-codamol capsules 30mg + 500mg', 'NORTRIPTYLINE CAP 25mg', 'Codeine 60mg tablets (Sandoz Ltd)', 'CARBAMAZEPINE tablets 200mg', 'Lidocaine  Cream  4 %', 'DIHYDROCODEINE TARTRATE C/R TAB 90mg', 'Tramadol Hydrochloride Mr TABS 75MG', 'Phenytoin sodium 100mg tablets (Wockhardt UK Ltd)', 'TRAMADOL MR TAB 150MG', 'PARACETAMOL caplets 500mg [ACTAVIS]', 'ASPIRIN 75MG DISP TABS', 'CARBAMAZEPINE TABLETS  100 MG', 'Diclofenac Potassium TABS 25MG', 'Diclofenac Sodium  M/R Capsules  100 mg', 'Ibuprofen Pain Relief Maximum Strength 10% gel (Numark Ma...', 'AMITRIPTYLINE HYDROCHLORIDE tabs 25mg', 'NAPROXEN 250mg tabs', 'DICLOFENAC 50MG TABLETS', 'NAPROXEN ENTERIC COATED TABLETS 500 MG', 'Lidocaine Hydrochloride Amp 5ml INJ 1%', 'LINCTUS CODEINE linctus', 'Lidocaine 50mg/5ml (1%) solution for injection Sure-Amp a...', 'DICLOFENAC suppository 100mg', 'Tramadol Hydrochloride  Soluble tablets  Sugar Free 50 mg', 'PARACETAMOL WITH METOCLOPRAMIDE tablets 500mg+5mg', 'IBUPROFEN 800mg m/r tablets', 'Tramadol Hydrochloride M/R Capsules 100 mg', 'Fentanyl patch 25mcg/hour', 'Phenytoin capsules 50mg', 'Ibuprofen Tablets 200mg', 'PARACETAMOL sf susp 500mg/5ml', 'DICLOFENAC RETARD 100MG CAPS', 'IBUPROFEN SYR 100', 'Paracetamol SUPPS 500MG', 'Tapentadol  Tablets  50 mg', 'SODIUM VALPROATE E/C TAB 500', 'ASPIRIN tablets 300mg [CO-OPERATI]', 'ibuprofen tablets 200mg', 'Tramadol 50mg capsules (IVAX Pharmaceuticals UK Ltd)', 'IBUPROFEN liquid capsules 400mg [DEXCEL]', 'ASPIRIN SOLUBLE 75MG TABS', 'Mirtazapine 15mg/ml oral solution sugar free (Rosemont Pharmaceuticals Ltd)', 'Morphine Sulphate  M/R tablets  5 mg', 'Ibuprofen 400mg tablets (Kent Pharmaceuticals Ltd)', 'Methadone 1mg/ml oral solution sugar free (A A H Pharmaceuticals Ltd)', 'Lamotrigine 25mg tablets (Teva UK Ltd)', 'Diamorphine 5mg powder for solution for injection ampoules (ViroPharma Ltd)', 'CO-CODAMOL capsules 8mg+500mg', 'PARACETAMOL TABS 500 MG', 'Buprenorphine 400microgram sublingual tablets sugar free (Arrow Generics Ltd)', 'DICLOFENAC 25mg tabs e/c', 'PARACETAMOL CAP 500mg', 'lidocaine with cetylpyridinium chloride and chlorocresol gel 2% w/v', 'Aspirin Dispersible Tablets 75mg', 'DIHYDROCODEINE elixir 10mg/5ml', 'APOMORPHINE sublingual tabs 3mg', 'PHENYTOIN SODIUM CAPSULES 100MG', 'Lidocaine Hydrochloride Antiseptic GEL 2%', 'DULOXETINE TABLETS 40MG', 'Diclofenac sodium 100mg modified-release capsules (A A H Pharmaceuticals Ltd)', 'FENTANYL 25 MICROGRAMS/HOUR PATCH', 'Amitriptyline hydrochloride 10 mg tablets', 'AMITRIPTYLINE TAB 100', 'tramadol modified release capsule 100mg', 'PARACETAMOL SUS 250mg', 'PARACETAMOL susp 250mg/5ml', 'DICLOFENAC SODIUM enteric coated tablets 25mg', 'CELECOXIB 200 MGS', 'IBUPROFEN TABLETS 200 MG', 'Lidocaine 0.66% / Aminoacridine 0.05% oromucosal gel', 'Gabapentin 100mg capsules (Sandoz Ltd)', 'AMITRIPTYLINE HCL TAB 25MG', 'DICLOFENAC RETARD 100 MG', 'Codeine 30mg tablets (Ranbaxy (UK) Ltd)', 'INDOMETHACIN S/R CAP 25', 'Lidocaine 100mg/5ml (2%) solution for injection ampoules (AMCo)', 'DICLOFENAC eye drp 0.1%', 'DICLOFENAC SODIUM SR 75MG TABS', 'PARACETAMOL oral susp paed 120mg/5ml', 'Paracetamol 120mg/5ml oral suspension', 'Diamorphine 10mg powder for solution for injection ampoules (Wockhardt UK Ltd)', 'DICLOFENAC SOD TAB S/R 75MG', 'DICLOFENAC SODIUM SUPPS 25MG', 'FENTANYL lozenge + applicator 200micrograms', 'OXYCODONE HCl + NALOXONE HCl mr tab 10mg + 5mg', 'Ibuprofen Pump Spray 5 %', 'Paracetamol BP 500mg', 'Mirtazapine Tablets 45mg', 'methadone (form not specified)', 'Morphine Sulphate Oral SOLN 10MG/5ML', 'Lidocaine 5% / Phenylephrine 0.5% nasal spray (Martindale Pharmaceuticals Ltd)', 'tramadol modified release capsule 50mg', 'ibuprofen gel 5%', 'phenytoin capsules 25mg', 'Paracetamol Sf Oral SUSP 120MG/5ML', 'NAPROXEN 500mg', 'IBUPROFEN + PSEUDOEPHEDRINE HCl tabs 200mg + 30mg', 'CODEINE PHOSPHATE TABLETS 60MG', 'Diclofenac Tablets S/R 100 mg', 'CO-CODAMOL SF EFF POWDER', 'DICLOFENAC SODIUM SUP 50mg', 'PARACET/CODEINE PHOS CAP 500MG/30MG', 'VENLAFAXINE MR CAP 75mg', 'Tramadol Hydrochloride Mr TABS 50MG', 'CODEINE diabetic linctus [T & R]', 'MORPHINE SULPHATE mr tab 60mg', 'Naproxen 500mg tablets (Wockhardt UK Ltd)', 'naproxen sodium tablets 275mg', 'Paracetamol 500mg caplets (Lloyds Pharmacy Ltd)', 'Diclofenac Sodium  E/C, M/R Capsules  75 mg', 'Diclofenac Sodium Dispersible TABLETS 50MG', 'co-codamol effervescent tablet 30mg + 500mg', 'DICLOFENAC 75MG TABS', 'Codeine 25mg/5ml oral solution (Thornton & Ross Ltd)', 'ORAMORPH SR 30mg m/r tablets', 'CARBAMAZEPINE 100MG', 'Tramadol 100mg/ml oral drops', 'Lidocaine 100mg/10ml (1%) solution for injection ampoules (AMCo)', 'DICLOFENAC injection 25mg/ml [3ml amp]', 'CODEINE PHOSPHATE ELI 15mg', 'Diamorphine Hydrochloride  Injection  30mg/amp', 'CODEINE PHOSPHATE', 'morphine sulfate modified release capsules 100mg', 'Carbamazepine C/R Tablets 200 mg', 'PARACETAMOL 450MG/CODEINE 8.1MG TAB', 'AMITRIPTYLINE HYDROCHLORIDE MR CAPSULES 25MG', 'DICLOFENAC caps m/r 100 mg', 'Morphine hydrochloride 30mg suppositories', 'Venlafaxine 75mg/5ml oral solution', 'TRAMADOL MR CAP 50mg', 'MIRTAZAPINE ORODISPERSIBLE 30MG TABLETS', 'Paracetamol Caplet TABLETS 500MG', 'Methylprednisolone acetate with lidocaine 40mg/ml + 10mg/ml Injection', 'CARBAMAZEPINE TABS 100MG', 'CO-CODAMOL 30MG/500MG TAB', 'diamorphine hydrochloride elixir 5mg/5ml', 'TRAMADOL CAPSULES 50MG', 'Celecoxib 200mg caps', 'Depo-Medrone with Lidocaine suspension for injection 1ml vials (Lexon (UK) Ltd)', 'Amitriptyline 50mg tablets (Wockhardt UK Ltd)', 'DICLOFENAC INJECTION 25 MG/ML', 'AMITRIPTYLINE HYDROCHLORIDE tablets 50mg', 'Tramadol Hydrochloride  M/R Capsules  200 mg', 'Methylprednisolone 40mg/1ml / Lidocaine 10mg/1ml (1%) suspen', 'diclofenac sodium (roi) capsules 50mg', 'paracetamol with codeine phosphate effervescent tablet 500m-', 'NAPROXEN EC TABS 500MG', 'Aspirin 300mg suppositories', 'Diclofenac Sodium  Cutaneous Solution  16 mg/ml', 'DICLOFENAC SODIUM AMP 3ML INJ 25MG/ML', 'Carbamazepine 200mg tablets (Actavis UK Ltd)', 'PARACETAMOL + DIHYDROCODEINE eff tab 500mg + 7.46mg', 'CO-CODAMOL 30/500MG 1', 'DICLOFENAC dual release capsules 75mg', 'Dihydrocodeine Tartrate TABLETS 40MG', 'Naproxen 500mg Tabs', 'morphine sulfate oral liquid', 'Naproxen 375mg gastro-resistant tablets (Actavis UK Ltd)', 'Gabapentin 600mg tablets (Actavis UK Ltd)', 'diclofenac sodium (form not specified)', 'NORTRIPTYLINE tablets 10mg', 'DICLOFENAC DUAL RELEASE CAP 75', 'Diamorphine 5mg powder for solution for injection vials', 'NAPROXEN SODIUM TAB 275mg', 'naproxen granules 500mg', 'NAPROXEN SUP 500', 'DICLOFENAC modified release 75mg', 'Paracetamol 500mg caplets (Numark Management Ltd)', 'NAPROXEN SODIUM TAB 275', 'CO-CODAMOL TABLETS 8/500MG', 'Lidocaine 100mg/10ml (1%) solution for injection Mini-Pla...', 'Ibuprofen 100mg/5ml oral suspension sugar free (Sandoz Ltd)', 'Lidocaine hydrochloride injection 2%', 'GABAPENTIN CAPSULES  300MGS', 'Venlafaxine Tablets 75 mg', 'Ibuprofen 200mg tablets (Thornton & Ross Ltd)', 'AMITRIPTYLINE 10MG TABLETS', 'Paracetamol with codeine phosphate capsules 500mg + 8mg', 'IBUPROFEN/CODEINE PHOSPHATE 200MG/12.8MG TAB', 'Phenytoin sodium capsules 25mg', 'Lamotrigine tablets 25mg', 'DICLOFENAC 75MG TABLETS SR', 'venlafaxine tablets 50mg', 'Ibuprofen  Pump Spray  5 %', 'Diclofenac Diethylammonium Gel 1.16%', 'Mefenamic Acid Tablets 50', 'DICLOFENAC SODIUM modified release capsule 75mg', 'morphine sulfate 12 hour modified release capsules 200mg', 'PARACETAMOL SOLUBLE TABLET 500 MG', 'ASPIRIN enteric coated tablets 75mg [GALEN]', 'Pregabalin CAPSULES 100MG', 'PARACETAMOL 500MG TABLETS', 'CO-CODAMOL DISPERSIBLE TABLET 500 MG + 8 MG', 'co-codamol dispersible tablet 8/500', 'TRAMADOL HYDROCHLORIDE ORODISPERSIBLE TABLETS 50MG', 'DICLOFENAC & MISOPROSTOL tablet 50mg + 200mcg', 'Codeine phosphate linctus 15mg/5ml', 'ASPIRIN eff tab 300mg', 'Codeine linctus 15mg/5ml', 'Dihydrocodeine 10mg/5ml oral solution (Martindale Pharmaceuticals Ltd)', 'co-codamol (codeine and paracetamol) capsules 8mg+500mg', 'Co-codamol 30mg/500mg effervescent tablets (Fannin UK Ltd)', 'Diclofenac Sodium Multidose Eye DROPS 0.1%', 'PHENYTOIN SODIUM TABLETS 100MG', 'DICLOFENAC SODIUM SUP 100', 'Buprenorphine 400microgram sublingual tablets sugar free (Actavis UK Ltd)', 'Co-codamol forte tablets 30mg + 500mg', 'IBUPROFEN GRANULES 600MG (SACHETS)', 'Naproxen 375mg Tablet', 'amitriptyline hydrochloride tablets 10 mg', 'Paracetamol 250mg/5ml oral suspension sugar free (A A H Pharmaceuticals Ltd)', 'Diclofenac Sodium Gel 3 %', 'Paracetamol Capsules 500 mg', 'CARBAMAZEPINE tablets 100mg [IVAX]', 'CELECOXIB 200MG CAPSULES', 'Lamotrigine Tablets 100 mg', 'IBUPROFEN TABS 200 MG', 'DICLOFENAC EC 25mg', 'DIHYDROCODEINE (Generic Manuf) (form not specified)', 'LIDOCAINE HCl + PRESERVATIVE inj 2%', 'Mirtazapine Tablets 30mg', 'Mirtazapine Tablets 15mg', 'Co-codamol 8mg/500mg capsules', 'Mirtazapine 45mg tablets', 'Gabapentin 300mg capsules', 'Venlafaxine 75mg modified-release capsules', 'Tramadol 50mg capsules', 'Amitriptyline 10mg tablets', 'Paracetamol And Dihydrocodeine  Tablets  500 mg + 20 mg', 'PARACETAMOL sol tab 500mg', 'Duloxetine 30mg gastro-resistant capsules', 'Tramadol 100mg modified-release tablets', 'Co-codamol 30mg/500mg capsules', 'TRAMADOL HYDROCHLORIDE CAPSULES 50MG', 'Aspirin TABS 75MG', 'Lamotrigine  Tablets  100 mg', 'DICLOFENAC SODIUM mr cap 75mg', 'Ibuprofen  Tablets  400 mg', 'Ibuprofen 400mg capsules', 'LIDOCAINE + HYDROCORTISONE ACETATE oint 5% + 0.275%', 'Aspirin 300mg dispersible tablets', 'IBUPROFEN tabs 400mg', 'MEFENAMIC ACID tabs 500mg', 'Diclofenac Sodium  E/c tablets  50 mg', 'ASPIRIN ec tab 75mg', 'AMITRIPTYLINE HCl tabs 10mg', 'CELECOXIB CAPSULES 200MG', 'Diclofenac sodium 75mg modified-release capsules', 'Codeine 15mg/5ml linctus', 'DICLOFENAC SODIUM EC TABLETS 50MG', 'Venlafaxine Mr CAPS 75MG', 'Paracetamol 500mg / Dihydrocodeine 20mg tablets', 'Oramorph 10mg/5ml oral solution (Boehringer Ingelheim Ltd)', 'Etoricoxib 120mg tablets', 'Venlafaxine 75mg tablets', 'Aspirin 300mg gastro-resistant tablets', 'Phenytoin sodium 100mg capsules', 'Morphine sulfate 20mg/1ml solution for injection ampoules', 'AMITRIPTYLINE TAB 25', 'Naproxen  Tablets  250 mg', 'Tramadol 50mg modified-release tablets', 'Sodium valproate 200mg gastro-resistant tablets', 'Co-Codamol 30mg/500mg TABS', 'Diclofenac sodium 25mg gastro-resistant tablets', 'Ibuprofen Tablets 400 mg', 'tramadol 12 hour modified release tablets 200mg', 'Co-Codamol 8mg/500mg TABS', 'Ibuprofen  Tablets  600 mg', 'ORAMORPH oral soln 10mg/5ml', 'Fentanyl 12micrograms/hour transdermal patches', 'CO-CODAMOL tabs 15mg + 500mg', 'Ibuprofen GEL 5%', 'BUPRENORPHINE patch 5micrograms/hour', 'Oxycodone 20mg / Naloxone 10mg modified-release tablets', 'Buprenorphine 20micrograms/hour transdermal patches', 'Amitriptyline 25mg/5ml oral solution sugar free', 'Oxycodone 5mg modified-release tablets', 'Paracetamol TABS 500MG', 'SODIUM VALPROATE MR TABLETS 300MG', 'Sodium valproate 100mg tablets', 'Dihydrocodeine 120mg modified-release tablets', 'Oxycodone 5mg/5ml oral solution sugar free', 'INDOMETHACIN CAP 25MG', 'Venlafaxine  Tablets  37.5 mg', 'Tapentadol 50mg modified-release tablets', 'VENLAFAXINE mr cap 75mg', 'Paracetamol And Dihydrocodeine  Tablets  500 mg + 30 mg', 'Sodium Valproate  M/R tablets  200 mg', 'Lidocaine 5% ointment', 'Gabapentin  Tablets  600 mg', 'DICLOFENAC tabs 50mg', 'Diclofenac Diethylammonium  Gel  1.16 %', 'Oxycodone 10mg/1ml solution for injection ampoules (Wockhardt UK Ltd)', 'Paracetamol 500mg tablets (Sigma Pharmaceuticals Plc)', 'Fentanyl 25micrograms/hour transdermal patches', 'DICLOFENAC mr tab 75mg', 'Diclofenac 0.1% eye drops', 'Tapentadol  M/R tablets  100 mg', 'GABAPENTIN CAPSULES 300MG', 'CODEINE PHOSPHATE TAB 30mg', 'Diclofenac Sodium Aqueous GEL 1%', 'CODEINE PHOSPHATE linc 15mg/5ml', 'Co-Codamol 8/500 Effervescent  Tablets', 'DICLOFENAC SODIUM TAB 50mg', 'tramadol 12 hour modified release tablets 150mg', 'CELECOXIB caps 200mg', 'Duloxetine  Gastro-resistant Capsules  60 mg', 'Paracetamol Caplet TABS 500MG', 'Codeine 60mg tablets', 'Gabapentin CAPS 100MG', 'Co-Codamol 30/500  Effervescent tablets', 'Ibuprofen 5% cream', 'GABAPENTIN caps 100mg', 'Co-codamol [STERWIN] effervescent tablet 8mg+500mg', 'CO-CODAMOL dispersible tablet 8mg+500mg [RHONE]', 'Co-Codamol 8/500 Tablets', 'Codeine 15mg/5ml linctus sugar free', 'ASPIRIN E/C TAB 75', 'MEFENAMIC ACID-P42 500MG TABLET', 'OXYCODONE HCl mr tab 10mg', 'DICLOFENAC 50MG/MISO TAB', 'Naproxen E/C Tablets 250 mg', 'IBUPROFEN TAB 400mg', 'TRAMADOL SR modified release tablet 100mg [ZENTIVA]', 'Mirtazapine  Tablets  30 mg', 'Diclofenac Diethylammonium Gel 1.16 %', 'DICLOFENAC mr tab 100mg', 'NAPROXEN TABLETS 250MG', 'CARBAMAZEPINE mr tab 200mg', 'DICLOFENAC enteric coated tablets 50mg', 'Tapentadol 50mg tablets', 'Morphine Sulfate  Oral solution  10 mg/5 ml', 'Mefenamic Acid CAPS 250MG', 'Paracetamol with codeine phosphate tablets 500mg + 30mg', 'DIHYDROCODEINE TARTRATE TABLETS 30MG', 'Lidocaine 100mg/10ml (1%) solution for injection ampoules', 'Lamotrigine TABS 200MG', 'CO-DYDRAMOL (dihydrocodeine & paracetamol) tabs 10mg+500mg', 'Tramadol Hydrochloride M/R Tablets 100 mg', 'Dihydrocodeine Tartrate TABS 30MG', 'Diclofenac sodium 50mg gastro-resistant tablets (Actavis UK Ltd)', 'Diclofenac modified release capsule 75mg', 'Sodium valproate 500mg gastro-resistant tablets', 'TRAMADOL caps 50 mg', 'Diclofenac  Medicated Plaster  1 % Patch', 'CELECOXIB CAPSULES 100MG', 'INDOMETHACIN S/R CAP 75mg', 'Codeine 25mg/5ml oral solution', 'Pregabalin  Capsules  100 mg', 'Venlafaxine TABS 37.5MG', 'Diclofenac sodium 50mg gastro-resistant / Misoprostol 200...', 'Ibuprofen 800mg tablets', 'Buprenorphine 35micrograms/hour transdermal patches', 'ASPIRIN disp tab 300mg', 'AMITRIPTYLINE TAB 50mg', 'DICLOFENAC SODIUM supp 100mg', 'DICLOFENAC DISPERSIBLE TAB 50mg', 'Lidocaine 2% cream', 'Diclofenac Sodium  Suppositories  100 mg', 'amitriptyline hydrochloride modified release capsules 75mg', 'Co-codamol 8mg/500mg tablets (Wockhardt UK Ltd)', 'Codeine Phosphate  Linctus  15 mg/5 ml', 'ETORICOXIB tabs 90mg', 'Lidocaine 1% gel (Teva UK Ltd)', 'DICLOFENAC SODIUM gel 3%', 'AMITRIPTYLINE HCl tabs 50mg', 'Co-codamol 30mg/500mg capsules (Teva UK Ltd)', 'Ibuprofen 5% spray', 'Lidocaine 40mg/2ml (2%) solution for injection ampoules', 'amitriptyline hydrochloride tablets 25mg', 'Codeine  Linctus sugar free  BP', 'oxycodone hydrochloride injection 10mg/ml', 'Ibuprofen Gel 10 %', 'Carbamazepine 400mg tablets', 'Aspirin 75mg gastro-resistant tablets (Actavis UK Ltd)', 'Tramadol 50mg capsules (Teva UK Ltd)', 'Oramorph 20mg/ml concentrated oral solution (Boehringer Ingelheim Ltd)', 'Ibuprofen Gel 5 %', 'Paracetamol 500mg caplets (Galpharm International Ltd)', 'Paracetamol 500mg caplets (Actavis UK Ltd)', 'Co-Codamol 12.8/500  Tablets', 'PARACETAMOL 500MG/METOCLOPROP.HCL 5MG TAB', 'VENLAFAXINE TABLETS 75MG', 'Morphine 30mg modified-release capsules', 'PHENYTOIN caps 50mg', 'GABAPENTIN caps 400mg', 'DICLOFENAC 50mg e/c tablets', 'Diclofenac potassium 25mg tablets', 'Carbamazepine TABS 100MG', 'Co-Codamol 15/500  Capsules', 'Lidocaine 200mg/10ml (2%) solution for injection Mini-Plasco ampoules (B.Braun Medical Ltd)', 'Co-codamol 8 mg + 500 mg capsules', 'Venlafaxine  M/R tablets  150 mg', 'Carbamazepine 200mg modified-release tablets (A A H Pharmaceuticals Ltd)', 'Venlafaxine  M/R Capsules  75 mg', 'Apomorphine 3mg sublingual tablets sugar free', 'Co-codamol 8mg/500mg tablets (A A H Pharmaceuticals Ltd)', 'Diclofenac Sodium Dispersible TABS 50MG', 'FENTANYL patch 12micrograms/hour', 'Co-codamol 8mg/500mg caplets (Actavis UK Ltd)', 'Mefenamic Acid TABS 500MG', 'Codeine 30mg tablets (Almus Pharmaceuticals Ltd)', 'Oxcarbazepine 150mg tablets', 'IBUPROFEN tablets 600mg', 'Fentanyl  Transdermal patches  12 micrograms/hour', 'Mefenamic Acid  Tablets  500 mg', 'Paracetamol 500mg caplets (IVAX Pharmaceuticals UK Ltd)', 'Naproxen 250mg tablets (A A H Pharmaceuticals Ltd)', 'ETORICOXIB TABLETS 60MG', 'Ibuprofen 300mg modified-release / Codeine 20mg tablets', 'Lidocaine hydrochloride 0.5% / Cetylpyridinium chloride 0.025% oromucosal gel sugar free', 'Diclofenac 16mg/ml topical solution', 'Carbamazepine 100mg tablets (Actavis UK Ltd)', 'Indomethacin-P42 50mg capsule', 'NAPROXEN & MISOPROSTOL tabs 500mg+200mcg', 'ASPIRIN S/R 600mg', 'Oxycodone 5mg/5ml oral solution sugar free (Wockhardt UK Ltd)', 'DICLOFENAC SODIUM+MISOPROSTOL 50mg/200micrograms tablets', 'Paracetamol 500mg tablets (Actavis UK Ltd)', 'Morphine sulfate 10mg/5ml oral solution unit dose vials sugar free', 'CARBAMAZEPINE tabs 200mg', 'morphine sulfate 24 hour modified release capsules 150mg', 'CO-CODAMOL EFFERVESCENT TAB', 'DICLOFENAC', 'BUPRENORPHINE 10 MICROGRAMS/HOUR PATCH', 'Tramadol Hydrochloride Sf Soluble TABS 50MG', 'Co-codamol 30mg/500mg effervescent tablets (A A H Pharmaceuticals Ltd)', 'Duloxetine  Gastro-resistant Capsules  40 mg', 'Ibuprofen cream 5%', 'Celecoxib 200mg cap', 'Ibuprofen 200mg / Pseudoephedrine hydrochloride 30mg tablets', 'PARACETAMOL 500MG/30MG CODEINE PHOSPHATE TAB', 'Depo-medrone + lidocaine Injection 40 mg/ml', 'Sodium valproate 300mg modified-release tablets', 'Mexiletine 50mg capsules', 'INDOMETHACIN CAP 25', 'DICLOFENAC GEL 1.16%', 'Fentanyl 400microgram sublingual tablets sugar free', 'Carbamazepine 100mg chewable tablets sugar free', 'Morphine sulfate 15mg/1ml solution for injection ampoules', 'PARACETAMOL + CODEINE PHOSPHATE tabs 500mg + 30mg', 'Lidocaine  Injection  1 %, 20 mg/2 ml ampoule', 'DULOXETINE gastro-res cap 20mg', 'Co-codamol 30mg/500mg capsules (Zentiva)', 'CO-CODAMOL TABLETS 8MG+500MG', 'morphine sulfate 12 hour modified release capsules 30mg', 'PREGABALIN caps 100mg', 'CO-CODAMOL TABS 8/500', 'Diclofenac sodium 50mg gastro-resistant / Misoprostol 200mic', 'Oxycodone 120mg modified-release tablets', 'Fentanyl 75micrograms/hour transdermal patches (A A H Pharmaceuticals Ltd)', 'CO-CODAMOL 30/500MG TABLETS', 'tramadol (roi) tablets 50mg', 'paracetamol with metoclopramide tablets 500mg+5mg', 'ETORICOXIB tabs 30mg', 'Lidocaine And Prilocaine  Cream  2.5 % + 2.5 %', 'Diamorphine 10mg powder for solution for injection ampoules (ViroPharma Ltd)', 'Paracetamol500mg with codeine phosphate 15mg tablet', 'Celecoxib 200 mg capsule', 'Co-codamol 8mg/500mg caplets (DE Pharmaceuticals)', 'CO-CODAMOL DISP TAB 8MG+500MG', 'CO-CODAMOL DISP TABLETS 8MG+500MG', 'Sodium valproate 200mg gastro-resistant tablets (A A H Pharmaceuticals Ltd)', 'Venlafaxine 75mg tablets (Actavis UK Ltd)', 'BUPRENORPHINE patch 70mcg/hour', 'Venlafaxine Hydrochloride  M/R Capsules  75 mg', 'KAOLIN & MORPHINE mix', 'INDOMETHACIN caps 25 mg', 'Lidocaine OINT 5%', 'Lidocaine hcl 1% Injection', 'DICLOFENAC DISPERSIBLE TABS 50MG', 'DICLOFENAC SODIUM S/R TAB 100mg', 'DICLOFENAC MR TAB 100MG', 'Apomorphine Hydrochloride  Solution for infusion  5 mg/ml  10 ml pre-filled syringe', 'ASPIRIN TABS 75MG', 'TRAMADOL 12hr mr tabs 200mg', 'Naproxen 500mg gastro-resistant tablets (A A H Pharmaceuticals Ltd)', 'IBUPROFEN mousse 5%', 'phenytoin capsules 300mg', 'Tramadol 100mg modified-release capsules (A A H Pharmaceuticals Ltd)', 'Diclofenac 25mg suppositories', 'CELECOXIB 100MG CAPSULE', 'Lidocaine 20mg/2ml (1%) solution for injection ampoules (Hameln Pharmaceuticals Ltd)', 'PARACETAMOL WITH CODEINE PHOSPHATE tabs 500mg + 30mg', 'CODEINE PHOSPHATE TABLETS 15MG', 'Codeine 30mg tablets (Sandoz Ltd)', 'Venlafaxine Hydrochloride  M/R Capsules  150 mg', 'Phenytoin Sodium CAPS 100MG', 'IBUPROFEN tablets 200mg', 'Aspirin TABS 300MG', 'Aspirin 300mg gastro-resistant tablets (Mylan Ltd)', 'Morphine Sulphate  Tablets  20 mg', 'Tramadol 200mg modified-release capsules (A A H Pharmaceuticals Ltd)', 'Amitriptyline 10mg tablets (Wockhardt UK Ltd)', 'Dihydrocodeine Tartrate  M/R tablets  90 mg', 'Gabapentin 100mg capsules (Mylan Ltd)', 'AMITRIPTYLINE-P42 25mg tablet', 'CO-CODAMOL 15MG/500MG CAPLET TABLETS', 'diclofenac sodium (roi) gel 1%', 'Morphine sulphate 20mg/1ml solution for injection ampoules', 'Hydromorphone 1.3mg capsules', 'IBUPROFEN + MENTHOL gel 5%', 'Morphine 60mg modified-release granules sachets sugar free', 'SODIUM VALPROATE mr tab 500mg', 'Fentanyl 200microgram lozenges', 'Diamorphine 30mg powder for solution for injection ampoules', 'MIRTAZAPINE ORODISPERSIBLE TABLETS 15MG', 'Amitriptyline Hydrochloride Tablets 10 mg', 'Aspirin 75mg dispersible tablets (Teva UK Ltd)', 'Mirtazapine tablets 30mg', 'Mefenamic acid tablets 500mg', 'DIHYDROCODEINE mr tab 60mg', 'diamorphine hydrochloride powder for solution for injection 30mg', 'diclofenac sodium enteric coated tablets 50mg', 'Gabapentin 400mg capsules (Mylan Ltd)', 'ASPIRIN ec tab 300mg', 'TRAMADOL MR 100MG', 'ASPIRIN EC 300MG', 'DICLOFENAC DISPERSIBLE tablets 50mg', 'LIDOCAINE HCl + ADRENALINE 1:200,000 inj 1%', 'Co-codamol 30mg/500mg caplets (AMCo)', 'Mefenamic Acid TABLETS 500MG', 'CELECOXIB CAPSULES 200MGM', 'AMITRIPTYLINE HYDROC CAP 25', 'Lidocaine 20mg/2ml (1%) solution for injection ampoules (AMCo)', 'Dihydrocodeine Tartrate Mr TABS 120MG', 'Aspirin  E/c tablets  300 mg', 'buprenorphine three-day patch 35mcg/hour', 'paracetamol with codeine phosphate and caffeine capsules 500mg+8mg+30mg', 'DIHYDROCODEINE + PARACETAMOL tabs 20mg + 500mg', 'Tramadol 12 Modified-release tablet', 'Apomorphine Hydrochloride Sls Sublingual TABS 2MG', 'Gabapentin 600mg tablets (Teva UK Ltd)', 'CODEINE PHOSPHATE SYRUP 25MG/5ML', 'TRAMADOL HCL CAP 50', 'AMITRIPTYLINE tabs 25 mg', 'DICLOFENAC 50MG tabs', 'Celecoxib capsules 200mg', 'Nortriptyline 10mg / Fluphenazine 500microgram tablets', 'Ibuprofen 10% gel (A A H Pharmaceuticals Ltd)', 'LAMOTRIGINE TAB 200mg', 'Lacosamide 150mg tablets', 'Sodium valproate 500mg gastro-resistant tablets (Teva UK Ltd)', 'OXYCODONE HYDROCHLORIDE MR TABLETS 20MG', 'Mirtazapine 15mg tablets (Arrow Generics Ltd)', 'CARBAMAZEPINE TAB 100mg', 'Aspirin 75mg gastro-resistant tablets (A A H Pharmaceuticals Ltd)', 'ETORICOXIB 90MG TAB', 'DICLOFENAC EC tab 50mg', 'PHENYTOIN tablets 100mg [IVAX]', 'SODIUM VALPROATE E/C TAB 500mg', 'Codeine phosphate with paracetamol tablets 15mg + 500mg', 'Lacosamide  Tablets  50 mg', 'Dihydrocodeine Tartrate  M/R Tablets  60 mg', 'Buprenorphine  Transdermal patches  20 micrograms/hour', 'PHENYTOIN SODIUM caps 25mg', 'Mexiletine Hydrochloride CAPS 50MG', 'Oxycodone 40mg / Naloxone 20mg modified-release tablets', 'IBUPROFEN gel', 'Ibuprofen Tablets 600 mg', 'tramadol 24 hour modified release tablets 200mg', 'Ibuprofen 400mg Tabs', 'Lamotrigine  Tablets  25 mg', 'ibuprofen and codeine tablets 200mg + 12.5mg', 'IBUPROFEN GEL - SPECIFY CONCTN. 5/99-P42 0', 'Ibuprofen  Pump spray  5 %', 'Morphine sulfate 10mg/1ml solution for injection ampoules (Martindale Pharmaceuticals Ltd)', 'Oxycodone 20mg/2ml solution for injection ampoules (Wockhardt UK Ltd)', 'DICLOFENAC EC TABLETS 25 MG', 'NAPROXEN EC TAB 500', 'KAPAKE(PARA 500MG/30MG CODEINE) TAB', 'PARACETAMOL WITH CODEINE PHOSPHATE tablets 500mg+30mg', 'Phenytoin 100mg capsule', 'PHENYTOIN TAB 50mg', 'GABAPENTIN TITRATION PACK', 'PREGABALIN CAPSULES 50MG', 'Lidocaine 40mg/2ml (2%) solution for injection ampoules (Martindale Pharmaceuticals Ltd)', 'PARACETAMOL + CODEINE PHOSPHATE eff tab 500mg + 30mg', 'TRAMADOL HYDROCHLORIDE MR CAPSULES 150MG', 'IBUPROFEN S/R TAB 800', 'Paracetamol 500mg with codeine phosphate 30mg capsule', 'Paracetamol 500mg / Metoclopramide 5mg effervescent powde...', 'Mefenamic acid capsules 250mg', 'Ibuprofen CREAM 5%', 'PARACETAMOL + CODEINE PHOSPHATE caps 500mg + 8mg', 'CARBAMAZEPINE RETARD TABLETS 200-P42 0', 'Venlafaxine Mr TABS 225MG', 'Dihydrocodeine 30mg tablets (Actavis UK Ltd)', 'Fentanyl  Transdermal patches  25 micrograms/hour', 'Celecoxib Capsules 200 mg', 'Diclofenac Potassium TABLETS 50MG', 'Ibuprofen 200mg tablets (Boston Healthcare Ltd)', 'Aspirin Dispersible TABLETS 75MG', 'ETORICOXIB TABLETS 120MG', 'BUPRENORPHINE HCL SUBLINGUAL TAB 200 MCG', 'Tramadol Hydrochloride  M/R tablets  200 mg', 'CODEINE PHOSPHATE tablets 15mg', 'Diclofenac Potassium Tablets 50 mg', 'Dexibuprofen 400mg tablets', 'CODEINE PHOSPHATE TABLETS 30MG', 'Tramadol And Paracetamol Tablets 37.5 mg + 325 mg', 'Ibuprofen Tablets 200 mg', 'ETORICOXIB TABLETS 30MG', 'Amitriptyline 25mg tablets (Actavis UK Ltd)', 'paracetamol with codeine and buclizine ( combination pack) tablets', 'Paracetamol CAPSULES 500MG', 'Gabapentin 600mg tablets and Gabapentin 300mg capsules', 'DICLOFENAC ec tabs 50mg', 'CELECOXIB CAPSULES 100 MGM', 'Lidocaine 40mg/2ml (2%) solution for injection ampoules (AMCo)', 'VENLAFAXINE MR TABLETS 75MG', 'Nu-Seals Aspirin  E/c tablets  75 mg', 'PARACETAMOL CAP 500', 'INDOMETHACIN CONTROLLED RELEASE TABLET 50 MG', 'NAPROXEN TABS 250MG', 'Co-codamol 8mg/500mg effervescent tablets (Bayer Plc)', 'NU-SEALS ASPIRIN EC tabs 75mg', 'NAPROXEN tablets 250mg', 'ARTHROTEC-DICLOFENAC TAB', 'IBUPROFEN TAB 200', 'NAPROXEN 500mg m/r tablets', 'Diclofenac Potassium Tablets 50mg', 'IBUPROFEN-P42 400MG TABLET', 'DICLOFENAC tabs 100 MG', 'VENLAFAXINE XL 75 MG TAB', 'AMITRIPTYLINE HCl mix 10mg/5ml', 'Co-codamol 8mg/500mg effervescent tablets (Teva UK Ltd)', 'Codeine Phosphate TABLETS 30MG', 'NAPROXEN 500MG TAB', 'Aspirin 75 mg soluble tablets', 'Lidocaine 0.05% / Chlorhexidine 0.2% oromucosal spray sugar free', 'VENLAFAXINE tablets 37.5mg', 'Naproxen tablets 500mg', 'CARBAMAZEPINE RETARD TAB 200mg', 'Co-codamol 30mg/500mg effervescent tablets (Actavis UK Ltd)', 'PARACETAMOL capsules 500mg', 'DEPO-MEDRONE + LIDOCAINE INJ 40MG/ML+10MG/ML', 'Celecoxib Capsules 100 mg', 'DICLOFENAC TABLETS 100 MG', 'SODIUM VALPROATE EC TABLETS 200 MG', 'LACOSAMIDE TABLETS 50MG', 'Ibuprofen 400mg 1tds', 'Methylprednisolone And Lidocaine  Injection  40 mg/ml + 10 mg/ml  2 ml vial', 'Codeine 15mg/5ml linctus sugar free (Thornton & Ross Ltd)', 'Venlafaxine Hydrochloride  Tablets  50 mg', 'Diclofenac Sodium E/C Tablets 25 mg', 'Carbamazepine 100mg tablets (A A H Pharmaceuticals Ltd)', 'CODEINE PHOSPHATE 30mg tablets', 'Diclofenac Sodium  Dispersible tablets  50 mg', 'ASPIRIN SOLUBLE 75mg', 'Co-codamol 30mg/500mg tablets (Actavis UK Ltd)', 'Lamotrigine 5mg dispersible tablets sugar free', 'ASPIRIN tabs dispersible 75 mg', 'Co-codamol 8mg/500mg effervescent tablets (Actavis UK Ltd)', 'MORPHINE SULPHATE 24hr mr caps 120mg', 'SODIUM VALPROATE CRUSHABLE TAB 100mg', 'PREGABALIN CAPSULES 200MG', 'Fentanyl 100micrograms/hour transdermal patches (A A H Pharmaceuticals Ltd)', 'lidocaine with benzalkonium chloride pump spray', 'SODIUM VALPROATE WITH VALPROIC ACID controlled', 'IBUPROFEN tablets 600mg [SOVEREIGN]', 'Buprenorphine Sf Sublingual TABS 200MICROGRAMS', 'Indomethacin  Capsules  75 mg', 'Aspirin Dispersible Tablet 75mg', 'ASPIRIN DISPERSIBLE TAB 300mg', 'Morphine sulfate 10mg/1ml solution for injection ampoules (UCB Pharma Ltd)', 'Oxycodone Hydrochloride  M/R tablets  5 mg', 'Morphine sulfate 30mg/5ml oral solution unit dose vials sugar free', 'Sodium Valproate Mr TABS 200MG', 'Codeine Phosphate Tablets 15 mg', 'DICLOFENAC SR 100 MGM', 'Aspirin 300mg tablets (Sigma Pharmaceuticals Plc)', 'DICLOFENAC SODIUM 50 MG TABLET', 'Lamotrigine 50mg tablets (Arrow Generics Ltd)', 'IBUPROFEN MR CAPSULES 300MG', 'Aspirin  M/R tablets  100 mg', 'LIDOCAINE HCl + ADRENALINE 1:200,000 inj 2% w/v', 'Morphine 120mg modified-release capsules', 'DICLOFENAC SR TABLETS 75 MG', 'INDOMETHACIN mr tab 75mg', 'Dihydrocodeine 30mg tablets (Almus Pharmaceuticals Ltd)', 'NAPROXEN EC 500MG TAB', 'CODEINE PHOSPHATE diabetic linctus 15mg/5ml -', 'DICLOFENAC modified release tablet 100mg [ACTAVIS]', 'PARACETAMOL caplets 500mg [NUCARE]', 'Co-codamol 8mg/500mg tablets (Zentiva)', 'Mexiletine 200mg tablets', 'Mirtazapine 30mg tablets (Arrow Generics Ltd)', 'VENLAFAXINE HYDROCHLORIDE TAB 75mg', 'Aspirin Dispersible Table', 'IBUPROFEN 5% CREAM', 'SODIUM VALPROATE EC tabs 200mg', 'Codeine Linctus Syrup 15 mg/5 ml', 'VENLAFAXINE tabs 50mg', 'Fentanyl  Transdermal patches  100 micrograms/hour', 'FENTANYL 50 MICROGRAMS/HOUR PATCH', 'FENTANYL 12 MICROGRAMS/HOUR PATCH', 'Paracetamol with codeine phosphate tablets 500mg + 15mg', 'Co-codamol 30mg/500mg caplets (Mawdsley-Brooks & Company ...', 'Morphine tartrate 15mg/1ml / Cyclizine tartrate 50mg/1ml solution for injection ampoules', 'NAPROXEN', 'Aspirin And Metoclopramide  Oral Powder Sachets Sugar Free  900 mg + 10 mg', 'PARACETAMOL', 'Lidocaine With Adrenaline  Solution For Injection  2 % / 1:2', 'IBUPROFEN tabs 200 mg', 'Ibuprofen TABS 800MG', 'Diclofenac And Misoprostol  Tablets  75 mg + 200 micrograms', 'Naproxen Suspension 25mg/', 'MEFENAMIC ACID TAB 250MG', 'METHADONE conc oral soln sf 20mg/ml', 'DICLOFENAC 50mg tabs e/c', 'CODEINE 15mg/5ml linct', 'Gabapentin 100mg capsules (Wockhardt UK Ltd)', 'Mefenamic acid 500mg tablets (Actavis UK Ltd)', 'Etoricoxib tablets 120mg', 'CO-CODAMOL', 'Pregabalin 100mg capsules (Teva UK Ltd)', 'naproxen modified release tablet 375mg', 'ibuprofen tablets 600mg', 'PHENYTOIN SODIUM 100mg caps', 'NAPROXEN 500mg e/c tablets', 'MORPHINE SULPHATE TABLETS 10MG', 'Phenytoin Sodium TABS 100MG', 'VENLAFAXINE MR CAP 150MG', 'Lacosamide  Tablets  200 mg', 'Amitriptyline 10mg tablets (Kent Pharmaceuticals Ltd)', 'LAMOTRIGINE TABLETS 200MG', 'sodium valproate with valproic acid modified release granules 250mg', 'Midrid Capsules 65 mg + Paracetamol 325mg', 'protriptyline tablets 5mg', 'Pregabalin CAPS 150MG', 'DIHYDROCODEINE tabs s-r 60 mg', 'AMITRIPTYLINE 50MG CAPS', 'Paracetamol Soluble Tablet 500mg', 'CO-CODAMOL 30/500 tabs 500mg paracetamol-P42 0', 'ASPIRIN 75MG DISP TABLETS', 'Pregabalin CAPS 75mg', 'Codeine Linctus Syrup 15mg/5ml', 'Lidocaine 5% ointment (Teva UK Ltd)', 'DICLOFENAC 100MG', 'MIRTAZAPINE TABLETS 30 MG', 'Diamorphine 5mg powder for solution for injection ampoules (Wockhardt UK Ltd)', 'Aspirin 300mg gastro-resistant tablets (Sandoz Ltd)', 'SODIUM VALPROATE CRUSHABLE TABLETS 100MG', 'tramadol (form not specified)', 'Paracetamol 500mg / Pseudoephedrine hydrochloride 30mg tablets', 'DICLOFENAC tabs 25 mg', 'Morphine 100mg modified-release capsules', 'MIRTAZAPINE 30MG TABLETS', 'Fentanyl 600microgram buccal tablets sugar free', 'VENLAFAXINE MR CAP 150mg', 'NAPROXEN mr tab 500mg', 'naproxen controlled release tablet 500mg', 'indomethacin slow-release capsules 75 mg', 'TRAMADOL HCL SR TAB 100mg', 'Fentanyl 300microgram sublingual tablets sugar free', 'DICLOFENAC modified release capsules 100mg [SANDOZ]', 'Lamotrigine 2mg dispersible tablets sugar free', 'Buprenorphine 2mg sublingual tablets sugar free (A A H Pharmaceuticals Ltd)', 'ASPIRIN NUSEALS 300MG', 'Venlafaxine TABLETS 75MG', 'Duloxetine 30mg gastro-resistant capsules (Actavis UK Ltd)', 'Sodium Valproate  E/c tablets  500 mg', 'morphine sulfate 12 hour modified release capsules 100mg', 'DICLOFENAC EC tablets 25mg', 'IBUPROFEN modified release tablet 800mg', 'DICLOFENAC SR 75MG TABLETS', 'amitriptyline hydrochloride tablets 50mg', 'SODIUM VALPROATE EC TABLETS 500MG', 'MORPHINE SULPHATE 12hr mr caps 60mg', 'FENTANYL patch 100mcg/hour', 'dihydrocodeine tablets 30mg', 'Fentanyl 50 Micrograms/Hour PATCH', 'Ibuprofen Mousse 125g  5%', 'Aspirin 300mg gastro-resistant tablets (Teva UK Ltd)', 'Amitriptyline hydrochloride 25 mg tablets', 'MEFENAMIC ACID-P42 250MG CAPSULE', 'Codeine Phosphate Oral SOLN 25MG/5ML', 'CO-CODAMOL tabs 8/500', 'Morphine sulfate 10mg suppositories', 'Pregabalin 25mg capsules (Teva UK Ltd)', 'Etoricoxib Tablets 60 mg', 'Sodium valproate with valproic acid 200mg modified release tablets', 'DICLOFENAC RETARD 100MG TAB', 'dextropropoxyphene hcl with paracetamol tablets 32.5mg + 325mg', 'DICLOFENAC SODIUM SUP 50', 'ASPIRIN tabs, e/c 75 mg', 'Venlafaxine Hydrochloride Tablets 75mg', 'Co-Codamol 8/500 Effervescent Tablets', 'Diamorphine 30mg powder for solution for injection ampoules (ViroPharma Ltd)', 'DICLOFENAC SODIUM transdermal patch 1%', 'Mirtazapine Orodispersible tablets 45mg', 'Mirtazapine 30mg tablets (A A H Pharmaceuticals Ltd)', 'Oxycodone Hydrochloride Capsules 5 mg', 'Oxycodone Hydrochloride Capsules 10 mg', 'diclofenac and misoprostol tablets 75mg + 200mcg', 'PARACETAMOL sach 1g', 'Oramorph 30mg/5ml oral solution unit dose vials (Boehringer Ingelheim Ltd)', 'Diclofenac sodium 75mg modified-release capsules 56 capsule', 'Mirtazapine 30mg orodispersible tablets (A A H Pharmaceuticals Ltd)', 'DEPO-MEDRONE WITH LIDOCAINE injection 40mg/ml + 10mg/ml', 'Apomorphine 30mg/3ml solution for injection pre-filled disposable devices', 'Fentanyl 100micrograms/hour transdermal patches (Actavis UK Ltd)', 'Ibuprofen-P42 600mg tablet', 'OXYCODONE HCl mr tab 80mg', 'DEXIBUPROFEN TABLETS 300MG', 'NAPROXEN tabs 500 MG', 'DICLOFENAC SR 75MG TABLET', 'Paracetamol with dihydrocodeine tablets 500mg + 30mg', 'COCODAMOL EFFERVESCENT COCODAMOL EFFER', 'Ibuprofen 200mg tablets (Vantage)', 'Co-codamol 8mg/500mg effervescent tablets (Almus Pharmaceuticals Ltd)', 'mefenamic acid tablets 500mg', 'naproxen enteric coated tablets 250mg', 'Buprenorphine 35 Micrograms/Hour PATCH', 'Lidocaine 5% / Phenylephrine 0.5% nasal spray', 'PHENYTOIN CAP 300', 'Oxycodone And Naloxone  M/R tablets  10 mg + 5 mg', 'PROTRIPTYLINE HYDROCHLORIDE TAB 10mg', 'Paracetamol Effervescent TABS 500MG', 'IBUPROFEN CRM 5%', 'Paracetamol  Sugar free suspension  120 mg/5 ml', 'DICLOFENAC SODIUM topical soln 1.6%', 'DICLOFENAC MR TAB 100mg', 'AMITRIPTYLINE HYDROCHLORIDE tabs 25 MG', 'DICLOFENAC injection 25mg/ml [3ml amp(s)]', 'INDOMETHACIN CAPSULES 25 MG', 'PARACETAMOL EFFERVESCENT TAB 500', 'Carbamazepine tablets 200mg', 'CODEINE linctus [CO-OPERATI]', 'DICLOFENAC AND MISOPROSTOL tablets 75mg + 200mcg', 'MEFENAMIC ACID 500mg tabs', 'Oxcarbazepine tablets 300mg', 'Phenytoin sodium capsules 100mg', 'PARACETAMOL tablets 500mg [M&A PHARM]', 'Diclofenac sodium 75mg modified-release capsules (Actavis UK Ltd)', 'ASPIRIN EC TAB 300MG', 'Sodium valproate 100mg modified-release granules sachets ...', 'DICLOFENAC INJ 75mg', 'Ibuprofen 5 % gel', 'lidocaine hydrochloride injection 2%', 'DICLOFENAC suppository 50mg', 'Etoricoxib Tablets 30 mg', 'co-codamol capsules 8mg+500mg', 'Venlafaxine 37.5mg tablets (A A H Pharmaceuticals Ltd)', 'DIHYDROCODEINE TABS S-R 60 MG', 'Morphine Sulphate M/R Tablets 10mg', 'Ibuprofen  Suspension  100 mg/5 ml', 'BUPRENORPHINE four day patch 52.5mcg/hour', 'DICLOFENAC SR TABLETS 75MG', 'TRAMADOL SR 200MG TABLET', 'PARACETAMOL SF ELIX 120MG/5ML', 'PARACETAMOL (Generic Manuf) (form not specified)', 'Ibuprofen 400mg tablets sugar coated (Actavis UK Ltd)', 'Lidocaine And Chlorhexidine  Gel (11 Ml Syringe)  2 % + 0.25 %', 'Gabapentin 400mg capsules (Teva UK Ltd)', 'Dihydrocodeine tablets 30mg', 'ORAMORPH SR tablets 10mg [BOEH I HSP]', 'CO-CODAMOL tablets 30mg + 500mg', 'diclofenac sodium and misoprostol tablets 50mg + 200microgr-', 'APOMORPHINE sublingual tablets 3mg', 'Co-codamol tablets 15mg + 500mg', 'Diamorphine 100mg powder for solution for injection vials (Teva UK Ltd)', 'ibuprofen tablets 800mg', 'Tramadol 100mg/ml oral drops (AMCo)', 'Ibuprofen modified release tablet 800mg', 'Aspirin tablets 75mg', 'IBUPROFEN 800mg tablets', 'Lidocaine And Chlorhexidine  Gel (6 Ml Syringe)  2 % + 0.25 %', 'Tramadol hydrochloride  50 mg modified release capsules', 'DICLOFENAC TABLETS 50MG', 'PREGABALIN TABLETS 75MG', 'INDOMETHACIN TABLETS 50MG', 'DICLOFENAC inj 25mg/ml', 'DICLOFENAC SODIUM gel spray 4%', 'DIHYDROCODEINE SUSTAINED RELEASE tablets 60mg', 'LIDOCAINE HCl gel 0.5%', 'Venlafaxine 37.5mg tablets (Actavis UK Ltd)', 'mefenamic acid capsules 250 mg', 'diclofenac injection 25mg/ml', 'IBUPROFEN grans 400mg', 'Carbamazepine 250mg suppositories', 'Lidocaine 4% solution', 'Pregabalin CAPSULES 75MG', 'Tramadol Hydrochloride Effervescent Powder Sugar Free 50 mg/', 'IBUPROFEN SF ORAL SUSP 100MG/5ML', 'Naproxen and misoprostol 500mgwith200microgram combined Tablet', 'IBUPROFEN granules 600mg', 'NAPROXEN & MISOPROSTOL (combined) tabs 500mg+200mcg', 'Co-codamol 8mg/500mg tablets (Almus Pharmaceuticals Ltd)', 'Fentanyl 75 Micrograms/Hour PATCH', 'Mirtazapine Sf Oral SOLN 15MG/ML', 'Morphine Sulfate  M/R capsules  30 mg', 'Depo-Medrone With Lidocaine  Injection, 1 Ml Vial', 'Oxycodone Hydrochloride Mr TABS 40MG', 'Tramadol SR TABS 100MGS', 'PARACETAMOL soluble tab 500mg', 'PHENYTOIN CHEWABLE TAB 50mg', 'AMITRIPTYLINE HYDROCHLORIDE MIX 10mg', 'NORTRIPTYLINE TABLETS 25MG', 'SOL ASPIRIN', 'IBUPROFEN 5% GEL-P42 0', 'TRAMADOL HYDROCHLORIDE 150mg m/r capsules', 'IBUPROFEN E.C. 400MG', 'Gabapentin 100mg capsules (A A H Pharmaceuticals Ltd)', 'Diclofenac sodium and misoprostol tablets 50mg + 200microgra', 'Lamotrigine Tablets 25 mg', 'amitriptyline hydrochloride injection 10mg/ml', 'Diamorphine 5mg powder for solution for injection vials (Teva UK Ltd)', 'Paracetamol 1g oral powder sachets', 'Tabs Amitriptyline 25mg', 'NU-SEALS ASPIRIN tablets 600mg [LILLY]', 'Co-codamol 8mg/500mg effervescent tablets (Fannin UK Ltd)', 'ASPIRIN E/C TABLETS 75-P42 0', 'Mirtazapine  Orodispersible Tablets  15 mg', 'DICLOFENAC SODIUM supp 12.5mg', 'DICLOFENAC SODIUM AND MISOPROSTOL tablets 75mg + 200mcg', 'TRAMADOL tabs 100 mg m/r -', 'paracetamol with codeine phosphate effervescent tablet 500mg + 30mg', 'NAPROXEN MR TABLETS 500MG', 'venlafaxine XL TABS 75mg', 'DICLOFENAC SODIUM ENTERIC COATED TABLETS 25MG', 'CARBAMAZEPINE LIQ 100mg', 'Mefenamic acid 500mg tablets (A A H Pharmaceuticals Ltd)', 'NU-SEALS ASPIRIN TAB 300mg', 'Codeine phosphate with paracetamol tablets 30mg + 500mg', 'Lidocaine 100mg/10ml (1%) solution for injection pre-fill...', 'Aspirin tablets 300mg', 'Tramadol Hydrochloride  M/R Tablets  150 mg', 'Gabapentin 300mg capsules (Almus Pharmaceuticals Ltd)', 'NAPROXEN enteric coated tablets 250mg', 'Codeine 6.75mg/5ml oral solution', 'Diclofenac gel 1%', 'PARACETAMOL SOLUBLE tab 500 MG', 'VENLAFAXINE modified release capsules 75mg [HILLCROSS]', 'CARBAMAZEPINE 100mg tabs', 'DICLOFENAC SODIUM tabs 50mg', 'Lidocaine injection 2% w/v', 'DIHYDROCODEINE 60mg tabs', 'Mefenamic acid Tablets 500 mg', 'Duloxetine 60mg gastro-resistant capsules (Teva UK Ltd)', 'TRAMADOL CAPSULESS/R 50MG-P42 0', 'AMITRIPTYLINE 10MG', 'Ibuprofen 600mg tablets (Ranbaxy (UK) Ltd)', 'GABAPENTIN 300mg', 'AMITRIPTYLINE HCl sf mix 10mg/5ml', 'LIDOCAINE HCl gel 2% w/v', 'CO-CODAMOL CAP 30MG/500MG', 'CO-CODAMOL tablets 8mg+500mg [NUCARE]', 'Mefenamic Acid Tablets 500mg', 'Aspirin 75mg gastro-resistant tablets (Kent Pharmaceuticals Ltd)', 'Amitriptyline 25mg/5ml oral solution sugar free (Rosemont Pharmaceuticals Ltd)', 'Tapentadol  M/R tablets  150 mg', 'Diclofenac Sodium Mr TABLETS 100MG', 'MEFENAMIC ACID 250mg capsules', 'CO-CODAMOL DISPERSIBLE TABLETS 8MG+500MG', 'VENLAFAXINE MR CAP 75MG', 'CO-CODAMOL tabs', 'diclofenac sodium (as diethylammonium) gel 1%', 'PARACETAMOL dissolving tab 250mg', 'IBUPROFEN crm 5 %', 'CODEINE LINCTUS', 'Carbamazepine Chewable Tablet 100mg', 'Co-codamol capsules 8mg+500mg', 'Diclofenac  Cutaneous Spray  4 %', 'DICLOFENAC SOD 25MG TAB', 'Diamorphine 30mg powder for solution for injection vials (Teva UK Ltd)', 'Paracetamol 500mg capsules (Wockhardt UK Ltd)', 'NAPROXEN TABLETS 500mg', 'IBUPROFEN CAP 200', 'tramadol orodispersible tablet 50mg', 'Pregabalin  Oral solution  75 mg/5 ml', 'DICLOFENAC SODIUM 75MG DUAL RELEASE CAPS', 'Fentanyl 200microgram buccal films sugar free', 'Morphine sulfate 1mg/1ml solution for injection ampoules', 'LIDOCAINE HYDROCHLORIDE GEL 2%', 'buprenorphine hydrochloride sublingual tablets 200micrograms', 'Aspirin 75 mg enteric coated tablets', 'Tylex Capsules 500 mg paracetamol + 30mg code', 'Codeine Phosphate Tablets 15mg', 'TRAMADOL sach 100mg', 'Kaolin and Morphine mixture (Thornton & Ross Ltd)', 'AMITRIPTYLINE HYDROCHLORIDE 25mg', 'Lamotrigine 50mg dispersible tablets sugar free', 'dihydrocodeine with paracetamol effervescent tablet 20mg + 500mg', 'DICLOFENAC 100mg supps', 'Amitriptyline Tablets 25 mg', 'Indomethacin capsules 25mg', 'Methadone 50mg/1ml solution for injection ampoules', 'diclofenac dispersible tablets 50mg', 'TRAMADOL capsules 50mg [GEN (UK)]', 'Indomethacin-P42 25mg capsule', 'Amitriptyline 25mg / Perphenazine 2mg tablets', 'CODEINE PHOSPHATE TABLETS 30 MG', 'AMITRIPTYLINE HYDROCHLORIDE 10mg', 'Pregabalin 20mg/ml oral solution sugar free (Teva UK Ltd)', 'Aspirin 325mg / Caffeine 15mg tablets', 'NAPROXEN EC TAB 250mg', 'Lamotrigine 100mg tablets (Actavis UK Ltd)', 'NAPROXEN 250mg', 'Benylin with Codeine oral solution (Pfizer Consumer Healthcare Ltd)', 'DEPO-MEDRONE + LIDOCAINE INJ 40MG/ML+10MG/ML[1m', 'Carbamazepine 100mg tablets', 'Dihydrocodeine 30mg tablets', 'Naproxen 500mg tablets', 'Paracetamol 500mg soluble tablets', 'Aspirin 75mg gastro-resistant tablets', 'AMITRIPTYLINE HCl tabs 25mg', 'Depo-Medrone with Lidocaine suspension for injection 1ml vials (Pfizer Ltd)', 'Pregabalin 300mg capsules', 'Venlafaxine 37.5mg tablets', 'ASPIRIN EC TABLETS 75MG', 'Ibuprofen 600mg tablets', 'Amitriptyline 50mg tablets', 'Naproxen 500mg gastro-resistant tablets', 'Co-Codamol 8/500  Tablets', 'Lamotrigine 200mg tablets', 'Aspirin  Dispersible tablets  75 mg', 'VENLAFAXINE mr cap 150mg', 'ASPIRIN DISPERSIBLE TABLETS 75MG', 'Duloxetine 60mg gastro-resistant capsules', 'Codeine 15mg tablets', 'Celecoxib 200mg capsules', 'Lamotrigine 50mg tablets', 'DICLOFENAC POTASSIUM tabs 50mg', 'Paracetamol  Capsules  500 mg', 'Morphine sulfate 10mg/5ml oral solution', 'Diclofenac sodium 75mg gastro-resistant / Misoprostol 200microgram tablets', 'Paracetamol  Tablets  500 mg', 'Amitriptyline Hydrochloride TABS 25MG', 'Gabapentin 100mg capsules', 'Diclofenac sodium 50mg gastro-resistant / Misoprostol 200microgram tablets', 'PARACETAMOL TABLETS 500MG', 'Diclofenac Sodium E/C Tablets 50 mg', 'CO-CODAMOL 8MG/500MG EFFERVESCENT TABLETS', 'Duloxetine  Gastro-resistant Capsules  30 mg', 'NAPROXEN ec tab 250mg', 'lidocaine injection 1%', 'NAPROXEN tabs 500mg', 'CO-CODAMOL tabs 30mg + 500mg', 'Pregabalin 50mg capsules', 'Gabapentin 300mg capsules (Teva UK Ltd)', 'Carbamazepine  Tablets  100 mg', 'Diamorphine 10mg powder for solution for injection ampoules', 'OXYCODONE HCl mr tab 40mg', 'VENLAFAXINE mr tab 75mg', 'DICLOFENAC E/C TAB 50mg', 'PARACETAMOL caps 500mg', 'IBUPROFEN GEL 10%', 'DICLOFENAC DIETHYLAMMONIUM gel 1.16%', 'Fentanyl 50micrograms/hour transdermal patches', 'Dihydrocodeine 90mg modified-release tablets', 'Tramadol 50mg modified-release capsules', 'Venlafaxine  M/R capsules  150 mg', 'Etoricoxib 60mg tablets', 'Ibuprofen 300mg modified-release capsules', 'Diclofenac 50mg suppositories', 'diclofenac sodium tablets 25mg', 'diclofenac sodium tablets 50mg', 'ETORICOXIB tabs 120mg', 'DEPO-MEDRONE WITH LIDOCAINE  (2ML) INJ', 'Mirtazapine 15mg orodispersible tablets', 'Lamotrigine TABS 100MG', 'Pregabalin 25mg capsules', 'MEFENAMIC ACID TAB 500mg', 'Etoricoxib TABLETS 90MG', 'DICLOFENAC ec tab 50mg', 'Paracetamol 1g tablets', 'Paracetamol CAPS 500MG', 'CODEINE linctus 15mg/5ml [RUSCO]', 'Venlafaxine  M/R capsules  225 mg', 'PARACETAMOL tabs 1000mg', 'DEPO-MEDRONE + LIDOCAINE inj 40mg/1ml + 10mg/1ml', 'Venlafaxine  Tablets  75 mg', 'Paracetamol 500mg soluble tablets (Zentiva)', 'Mirtazapine TABS 30MG', 'Diclofenac 100mg suppositories', 'diclofenac sodium enteric coated tablets 25mg', 'CO-CODAMOL (codeine & paracetamol) tabs 30mg + 500mg', 'Buprenorphine  Transdermal patches  5 micrograms/hour', 'LIDOCAINE HYDROCHLORIDE injection 1% [AMCO]', 'Buprenorphine 200microgram sublingual tablets sugar free', 'LIDOCAINE HCl inj 2%', 'methylprednisolone acetate with lidocaine injection 40mg/ml + 10mg/ml', 'PARACETAMOL CAPSULES 500MG', 'Carbamazepine 200mg modified-release tablets', 'Pregabalin  Capsules  50 mg', 'Paracetamol  Oral Suspension (Sugar-Free)  250 mg/5 ml', 'DICLOFENAC GEL 100gm', 'Morphine Sulphate  Oral solution  10 mg/5 ml', 'CO-CODAMOL 8MG/500MG CAPS', 'OXYCODONE HCl mr tab 5mg', 'IBUPROFEN SPRAY 5%', 'CO-CODAMOL (codeine & paracetamol) caps 30mg + 500mg', 'DICLOFENAC SODIUM supp 50mg', 'Oramorph 10mg/5ml oral solution unit dose vials (Boehringer Ingelheim Ltd)', 'Morphine Sulphate  M/R tablets  15 mg', 'Co-codamol 8mg/500mg caplets (Vantage)', 'CARBAMAZEPINE mr tab 400mg', 'Lamotrigine TABS 50MG', 'Tramadol 200mg modified-release tablets', 'Paracetamol 500mg tablets (Vantage)', 'Naproxen  E/C Tablets  250 mg', 'INDOMETHACIN SR TAB 75mg', 'CO-CODAMOL 30MG/500MG CAPS', 'DEPO-MEDRONE + LIDOCAINE inj 80mg/2ml + 20mg/2ml', 'DIHYDROCODEINE TARTR TAB 60', 'Oxycodone 15mg modified-release tablets', 'DICLOFENAC SODIUM S/ TAB 100', 'IBUPROFEN TAB 600mg', 'Lidocaine 0.66% / Aminoacridine 0.05% oromucosal gel sugar free', 'Methadone 1mg/ml oral solution sugar free', 'LAMOTRIGINE tabs 25mg', 'DICLOFENAC & MISOPROSTOL tabs 75mg + 200mcg', 'DICLOFENAC EC 50MG TABLET', 'Diclofenac Sodium  M/R tablets  75 mg', 'Phenytoin sodium 50mg tablets', 'PHENYTOIN SODIUM caps 100mg', 'GABAPENTIN tabs 600mg', 'PHENYTOIN SODIUM tabs 100mg', 'Pregabalin  Capsules  25 mg', 'Cathejell Lidocaine C (Teleflex)', 'Dihydrocodeine Tartrate  Tablets  30 mg', 'DICLOFENAC SODIUM SR TAB 75', 'AMITRIPTYLINE 25mg tablets', 'Duloxetine  Gastro-Resistant Capsules  20 mg', 'DICLOFENAC MR CAP 75MG', 'Phenytoin sodium 25mg capsules (Flynn Pharma Ltd)', 'Morphine sulfate 30mg/1ml solution for injection ampoules', 'LIDOCAINE injection 1% [MARTINDALE]', 'Paracetamol 500mg tablets (Teva UK Ltd)', 'Buprenorphine 70micrograms/hour transdermal patches', 'PREGABALIN caps 225mg', 'Dihydrocodeine 10mg/5ml oral solution', 'BUPRENORPHINE HCl sublingual tab 200micrograms', 'CO-CODAMOL EFFERVESC TAB', 'PREGABALIN caps 300mg', 'diamorphine hydrochloride powder for solution for injection 10mg', 'Ibuprofen GEL 10%', 'Lacosamide 50mg tablets', 'VENLAFAXINE modified release capsules 150mg [HILLCROSS]', 'PREGABALIN caps 75mg', 'Phenytoin sodium 25mg capsules', 'DICLOFENAC & MISOPROSTOL tabs 50mg + 200micrograms', 'Cathejell Lidocaine C  Lubricating jelly', 'ISOMETHEPTENE 65mg/PARACETAMOL 325mg CAP', 'Gabapentin 300mg capsules (Mylan Ltd)', 'ETORICOXIB tabs 60mg', 'NAPROXEN EC TABLETS 250MG', 'Oxcarbazepine 300mg tablets', 'Venlafaxine 37.5mg modified-release tablets', 'DICLOFENAC supp 50mg', 'DICLOFENAC MR 75MG TABS', 'Aspirin 300 mg enteric coated tablets', 'DICLOFENAC supp 100mg', 'Ibuprofen 5% gel (Thornton & Ross Ltd)', 'Lidocaine 400mg/20ml (2%) solution for injection vials (Hameln Pharmaceuticals Ltd)', 'TRAMADOL 100MG SR TABS', 'Paracetamol 500mg with codeine phosphate 8mg tablet', 'PERPHENAZINE + AMITRIPTYLINE tabs 2mg + 10mg', 'Buprenorphine  Transdermal patches  35 micrograms/hour', 'IBUPROFEN TABS S/R 800 MG', 'DICLOFENAC SODIUM MR CAPSULES 75MG', 'Carbamazepine 200mg chewable tablets sugar free', 'Paracetamol 500mg caplets (Almus Pharmaceuticals Ltd)', 'TRAMADOL mr cap 50mg', 'DICLOFENAC SR TAB 75', 'Oxycodone 10mg capsules', 'Dihydrocodeine with paracetamol 10mg+500mg tablets', 'DICLOFENAC MR TAB 75MG', 'Cathejell Lidocaine C (Teleflex Medical)', 'DICLOFENAC AND MISOPROSTOL', 'TRAMADOL mr tab 100mg', 'MIRTAZAPINE orodisp tab 45mg', 'LIDOCAINE oint 5%', 'DICLOFENAC AND MISOPROSTOL tabs 75mg + 200mcg', 'Etoricoxib  Tablets  60 mg', 'DICLOFENAC SR TAB 75mg', 'IBUPROFEN CRE 5%', 'Mefenamic Acid Capsules 250 mg', 'Amitriptyline Hydrochloride  Capsules  75 mg', 'DICLOFENAC modified release tablet 100mg', 'Co-codamol 15mg/500mg tablets (A A H Pharmaceuticals Ltd)', 'FENTANYL patch 50micrograms/hour', 'Lidocaine 4% cream', 'Co-codamol 30mg/500mg capsules (AMCo)', 'Paracetamol with codeine phosphate tablets 500mg + 8mg', 'DICLOFENAC SODIUM SUP 100mg', 'Methadone Hydrochloride Oral SOLN 1MG/ML', 'Naproxen 500mg gastro-resistant tablets (Almus Pharmaceuticals Ltd)', 'Gabapentin 100mg capsules (Teva UK Ltd)', 'Morphine Sulfate  Tablets  10 mg', 'Fentanyl 400microgram lozenges', 'Diclofenac sodium 50mg gastro-resistant tablets (A A H Pharmaceuticals Ltd)', 'MORPHINE SULPHATE SF ORAL UNIT DOSE VIAL 5ML  10MG/5ML', 'Gabapentin 400mg/5ml oral suspension', 'DICLOFENAC SODIUM TAB 25', 'Nuseals Aspirin Gastro Resistant TABS 75MG', 'DULOXETINE gastro-res cap 30mg', 'IBUPROFEN 400MG TABS', 'DICLOFENAC 75MG M/R CAPS', 'Diclofenac Sodium Dispersible Tablets Sugar Free 50 mg', 'Naproxen 125mg/5ml oral suspension', 'IBUPROFEN TABLETS 600 MG', 'Co-codamol 30mg/500mg caplets (A A H Pharmaceuticals Ltd)', 'CODEINE PHOSPHATE tabs 30 mg', 'Codeine 15mg/5ml linctus sugar free (Pinewood Healthcare)', 'CO-CODAMOL TABLETS 15MG + 500MG', 'lidocaine hydrochloride gel 0.5%', 'NORTRIPTYLINE + FLUPHENAZINE tabs 10mg + 500micrograms', 'Diclofenac Sodium M/R Tablets 100 mg', 'DICLOFENAC SODIUM MR TABLETS 75MG', 'paracetamol tablets 500mg', 'CODEINE PHOSPHATE TAB 15', 'Naproxen Ec TABS 500MG', 'TRAMADOL mr cap 150mg', 'TRAMADOL mr cap 200mg', 'ibuprofen (form not specified)', 'Diclofenac Sodium Mr TABS 100MG', 'mefenamic acid dispersible tablet 250mg', 'Apomorphine Hydrochloride  Pre-filled multiple dose pen  10 mg/ml  3 ml pen', 'Dihydrocodeine Tartrate  M/R tablets  60 mg', 'Codeine 30mg tablets (Actavis UK Ltd)', 'FENTANYL patch 25micrograms/hour', 'Codeine 30mg tablets (Kent Pharmaceuticals Ltd)', 'OXYCODONE HCl oral liq 5mg/5ml', 'Ibuprofen 600mg tablets (Fannin UK Ltd)', 'Tramadol Hydrochloride  M/R capsules  50 mg', 'AMITRIPTYLINE TAB 10', 'DICLOFENAC SODIUM ec tabs 50mg', 'Ibuprofen 5% gel (A A H Pharmaceuticals Ltd)', 'Carbamazepine 100mg/5ml oral suspension sugar free', 'INDOMETHACIN caps 25mg', 'Phenytoin 50mg chewable tablets', 'Lidocaine 1% gel', 'ASPIRIN DISP TAB 75MG', 'Tramadol 100mg/2ml solution for injection ampoules', 'MEFENAMIC ACID TAB 500', 'LAMOTRIGINE tabs 200mg', 'DICLOFENAC SODIUM SR TABLETS 75 MG', 'aspirin dispersible tablet 75mg', 'AMITRIPTYLINE HCl mr cap 25mg', 'Codeine Phosphate LINCT 15MG/5ML', 'Co-codamol 30mg/500mg caplets (J M McGill Ltd)', 'Diclofenac Sodium SUPPS 50MG', 'DICLOFENAC RETARD 75MG TABLET', 'Diclofenac sodium 75mg modified-release capsules (A A H Pharmaceuticals Ltd)', 'DICLOFENAC SR 75MG TAB', 'Amitriptyline Hydrochloride TABLETS 25MG', 'Paracetamol 500mg capsules (Zentiva)', 'Lidocaine 400mg/20ml (2%) solution for injection ampoules', 'DICLOFENAC CAPSULES 75MG DR', 'VENLAFAXINE MR CAP 75', 'MORPHINE SULPHATE mr tab 15mg', 'DEPO-MEDRONE WITH LIDOCAINE INJ', 'IBUPROFEN crm 5%', 'Amitriptyline Hydrochloride Tablets 50 mg', 'DICLOFENAC ec tab 50mg [SANDOZ]', 'OXYCODONE HCl caps 5mg', 'Naproxen Tablets 250 mg', 'INDOMETHACIN CAP 50', 'Ibuprofen 400mg tablets film coated (Actavis UK Ltd)', 'Diclofenac ec tablets 25mg', 'morphine sulfate modified release capsules 10mg', 'Mirtazapine Tablets 30 mg', 'Fentanyl 400micrograms/dose nasal spray', 'Carbamazepine 200mg tablets (Mylan Ltd)', 'VENLAFAXINE XL 75MG CAPS', 'PARACETAMOL + CODEIN PHOSPHAT 500MG+30MG', 'Depo-Medrone With Lidocaine  Injection  2 ml vial', 'Aspirin Dispersible TABS 300MG', 'Buprenorphine 5 Micrograms / Hour PATCH', 'Buprenorphine 70 Micrograms/Hour PATCH', 'Aspirin Tablets 75 mg', 'codeine phosphate tablets 30mg', 'Mirtazapine  Orodispersible tablets  15 mg', 'TRAMADOL sol tab 50mg', 'PARACETAMOL TAB 1gm', 'morphine sulfate 12 hour modified release capsules 60mg', 'Co-codamol 30mg/500mg effervescent tablets (Mylan Ltd)', 'DICLOFENAC modified release tablet 75mg [GALEN]', 'APOMORPHINE sublingual tab 2mg', 'Amitriptyline Hydrochloride  Sugar-free oral solution  10 mg/5 ml', 'Carbamazepine Tablets 200 mg', 'diclofenac sodium 75mg sr tabs', 'ASPIRIN tabs, dispersible 75 mg', 'Pregabalin CAPS 300MG', 'Diclofenac Sodium  Dispersible tablets  Sugar Free 50 mg', 'ASPIRIN dispersible tablet 75mg', 'sodium valproate with valproic acid modified release tablet 500mg', 'Lacosamide 200mg tablets', 'DICLOFENAC TABS 50 MG', 'Gabapentin CAPS 400MG', 'Diclofenac Sodium  E/C Tablets  25 mg', 'ASPIRIN ENTERIC COATED TABLETS 75MG', 'Diclofenac 12.5mg suppositories', 'LACOSAMIDE tabs 100mg', 'OXCARBAZEPINE tabs 150mg', 'Apomorphine 50mg/10ml solution for infusion pre-filled syringes', 'PARACETAMOL + CODEINE PHOSPHATE SF eff powder 500mg+30mg', 'Apomorphine 20mg/2ml solution for injection ampoules', 'Methadone 5mg tablets', 'apomorphine injection 10mg/ml', 'Mefenamic Acid  Capsules  250 mg', 'Ibuprofen And Levomenthol  Gel  5 % + 3 %', 'CARBAMAZEPINE 200mg tablets', 'Lidocaine 20mg/2ml (1%) solution for injection ampoules (Martindale Pharmaceuticals Ltd)', 'PARACETAMOL + DIHYDROCODEINE tabs 500mg + 20mg', 'Ibuprofen 400mg tablets (Ranbaxy (UK) Ltd)', 'IBUPROFEN 200mg tablets', 'IBUPROFEN CREAM 5%', 'Mirtazapine 15mg tablets (A A H Pharmaceuticals Ltd)', 'Co-proxamol (dextropropoxyphene and paracetamol) 32.5mg with 325mg/5ml oral suspension sugar free', 'PARACETAMOL SF ORAL SUSP 250MG/5ML', 'Tramadol Hydrochloride  M/R Capsules  100 mg', 'Amitriptyline Sf Oral SOLN 25MG/5ML', 'DICLOFENAC  TABLETS 50MG TABLETS', 'CODEINE PHOSPHATE + PARACETAMOL tabs 8mg + 500mg', 'PREGABALIN CAPSULES 75MG', 'NAPROXEN CR TAB 500', 'Duloxetine 20mg gastro-resistant capsules (Actavis UK Ltd)', 'INDOMETHACIN SUP 100', 'Ibuprofen 400mg Granules', 'Ibuprofen 400mg tablets (A A H Pharmaceuticals Ltd)', 'LIDOCAINE OINT 5%', 'Pregabalin Capsules 50 mg', 'Aspirin 75mg gastro-resistant tablets (Almus Pharmaceuticals Ltd)', 'codeine phosphate with paracetamol tablets 30mg + 500mg', 'INDOMETHACIN CAP 50mg', 'Diclofenac Sodium  E/C  M/R capsules  75 mg', 'Tramadol 100mg modified-release tablets (A A H Pharmaceuticals Ltd)', 'PHENYTOIN caps 25mg', 'Diclofenac Tablets 50mg', 'Phenytoin 25mg capsule', 'Paracetamol 250mg/5ml oral suspension sugar free (Pinewood Healthcare)', 'Buprenorphine Transdermal Patches 5 micrograms/hour', 'Aspirin enteric coated tablets 75mg', 'Sodium Valproate  M/R tablets  500 mg', 'Methadone 2mg/5ml linctus', 'Tramadol 150mg modified-release capsules (A A H Pharmaceuticals Ltd)', 'AMITRIPTYLINE HCl + PERPHENAZINE tabs 10mg + 2mg', 'Etoricoxib TABS 120MG', 'Ibuprofen 200mg modified-release capsules', 'Naproxen 250mg gastro-resistant tablets (A A H Pharmaceuticals Ltd)', 'Lidocaine 400mg/20ml (2%) / Adrenaline 100micrograms/20ml...', 'co-codamol tablets 8mg+500mg', 'tramadol 24 hour modified release tablets 150mg', 'ASPIRIN DISPERSIBLE TAB 300', 'DICLOFENAC TABS RETARD 100MG', 'PHENYTOIN CAP 100mg', 'Ibuprofen 400mg tablets (Vantage)', 'Sodium Valproate Ec TABS 200MG', 'Diclofenac sodium 25mg tablets', 'AMITRIPTYLINE HCl + PERPHENAZINE tabs 25mg + 2mg', 'TRAMADOL mr tab 200mg', 'Tramadol Hydrochloride  M/R tablets  150 mg', 'Tramadol Hydrochloride Mr TABS 150MG', 'BUPRENORPHINE 52.5 MICROGRAMS/HOUR PATCH', 'DICLOFENAC 100mg tabs s/r', 'Fentanyl 100 Micrograms/Hour PATCH', 'SODIUM VALPROATE + VALPROIC ACID mr tab 300mg', 'Venlafaxine 37.5mg/5ml oral suspension', 'DEPOMEDRONE WITH LIDOCAINE VIAL 1ML INJ 40MG/ML', 'Ibuprofen 5% gel (Galpharm International Ltd)', 'Tramadol 50mg capsules (Niche Generics Ltd)', 'Morphine Sulphate  M/R tablets  10 mg', 'IBUPROFEN grans 600mg', 'AMITRIPTYLINE HYDROCHLORIDE tablets 10mg', 'PARACET 500/CODEINE PHOS 8MG DIS', 'MEFENAMIC ACID 500mg tablets', 'MORPHINE SULPHATE 12hr mr caps 100mg', 'Indomethacin  Capsules  25 mg', 'ASPIRIN E/C TAB 300mg', 'DIHYDROCODEINE mr tab 120mg', 'Co-codamol 8mg/500mg effervescent tablets (Sandoz Ltd)', 'Indomethacin  Capsules  50 mg', 'AMITRIPTYLINE HYDROCHLORIDE 25MG', 'Naproxen E/C Tablets 500 mg', 'TRAMADOL HYDROCHLORIDE MR TABLETS 200MG', 'DIHYDROCODEINE tabs 30 MG', 'IBUPROFEN 200mg tabs', 'BUPRENORPHINE patch 10micrograms/hour', 'Ibuprofen  Dissolving Tablets  200 mg', 'Phenytoin sodium 100mg tablets (Teva UK Ltd)', 'TRAMADOL SR modified release tablet 200mg [ZENTIVA]', 'Phenytoin Tablets 100 mg', 'Codeine 3mg/5ml linctus paediatric sugar free', 'CARBAMAZEPINE TABLETS', 'Codeine 15mg/5ml linctus sugar free (Kent Pharmaceuticals...', 'Lidocaine 44mg/2.2ml (2%) / Adrenaline 27.5micrograms/2.2...', 'AMITRIPTYLINE SF ORAL SOLN 25MG/5ML', 'INDOMETHACIN cr tab 75mg', 'ASPIRIN EC TABLETS 300MG', 'VENLAFAXINE tabs 37.5 mg', 'IBUPROFEN sf susp 100mg/5ml', 'DULOXETINE GASTRO RESISTANT CAPSULES 40MG', 'Aspirin-P42 300mg tablet', 'PARACETAM/DYHYDROCODEINE(500MG/20MG) TAB', 'Phenytoin Sodium CAPS 25MG', 'DIHYDROCODEINE 30mg tablets', 'lidocaine with adrenaline (base) 1:80,000 injection 2% w/v', 'DICLOFENAC 50MG EC TAB', 'co-codaprin (codeine and aspirin) tablets 8mg+400mg', 'perphenazine with amitriptyline tablets 2mg + 10mg', 'Venlafaxine Mr TABS 75MG', 'PARACETAMOL EFFERVES TAB 500', 'Tramadol 150mg modified-release tablets', 'Lamotrigine 200mg tablets (Teva UK Ltd)', 'DICLOFENAC SR 75MG m/r TABS', 'mirtazapine TABS 15mg', 'DICLOFENAC SOD TAB E/C 50MG', 'CODEINE PHOSPHATE TAB 60mg', 'Depo-medrone with lidocaine injection 40mg/ml + 10mg/ml', 'DICLOFENAC 25mg e/c tablets', 'Naproxen-P42 500mg tablet', 'Ibuprofen Tab 600mg', 'Diclofenac sodium 50mg gastro-resistant tablets (IVAX Pharmaceuticals UK Ltd)', 'Gabapentin 50mg/ml oral solution sugar free', 'Aspirin 75 mg tablets', 'Diclofenac 2% gel', 'MIRTAZAPINE ORODISPERSIBLE TABLETS 45MG', 'IBUPROFEN CRE 5', 'DEPO-MEDRONE C. LIDOCAINE injection', 'DEPOMEDRONE WITH LIDOCAINE VIAL 2ML INJ 40MG/ML', 'LAMOTRIGINE TABLETS 100MG', 'ASPIRIN PAED 75MG TABLETS', 'TRAMADOL MR CAP 100mg', 'Aspirin-P42 75mg Tab Soluble', 'COCODAMOL DISP. 8/500 TABLET', 'DEPO-MEDRONE C. LIDOCAINE injection 40/10mg/1ml', 'paracetamol capsules 500mg', 'LIDOCAINE pump spray 10%', 'Sodium valproate 150mg modified-release capsules', 'Lacosamide  Tablets  150 mg', 'CODEINE PHOSPHATE-P42 30MG TABLETS', 'Dihydrocodeine Tartrate TABS 40MG', 'PARACETAMOL EFFERVESCENT TABLETS 500MG', 'Co-codamol 30mg/500mg caplets (Waymade Healthcare Plc)', 'Gabapentin CAPSULES 300MG', 'NAPROXEN SODIUM tablets 275mg', 'DICLOFENAC MR CAP 100MG', 'Indomethacin Capsules 25 mg', 'Valproic Acid (As Semisodium Valproate)  E/c tablets  500 mg', 'Buprenorphine  Transdermal patches  20 micrograms/hour (20 mg per patch)', 'Valproic Acid (As Semisodium Valproate)  E/c tablets  250 mg', 'diclofenac dual release CAPS 75', 'Diclofenac Tablets S/R 75 mg', 'DICLOFENAC enteric coated 50mg', 'CODEINE PHOSPHATE linctus 15 mg/5ml', 'ASPIRIN tabs e/c 75 mg', 'LIDOCAINE injection 2% [HAMELN]', 'Ibuprofen TABLETS 200MG', 'Ibuprofen-P42 400mg tablet', 'Paracetamol 500mg capsules (A A H Pharmaceuticals Ltd)', 'SODIUM VALPROATE WITH VALPROIC ACID modified re', 'Nu-seals aspirin ec 75mg Gastro-resistant tablet (Eli Lilly and Company Ltd)', 'Naproxen 250mg tablets (Wockhardt UK Ltd)', 'Mefenamic acid 500mg tablets (Essential Generics Ltd)', 'venlafaxine modified release capsule 75mg', 'TRAMADOL HCL SR TAB 200mg', 'DICLOFENAC tabs 25mg', 'Diclofenac ec tablets 50mg', 'Diamorphine Hydrochloride  Injection  10mg/amp', 'Ibuprofen 400mg tablets (Aspar Pharmaceuticals Ltd)', 'MEFENAMIC ACID tabs 500 MG', 'DICLOFENAC SODIUM 25 MGM TAB', 'Lamotrigine 50mg tablets (Actavis UK Ltd)', 'Phenytoin sodium 100mg tablets (Actavis UK Ltd)', 'Aspirin Dispersible Low Dose  Tablets  75 mg', 'Lamotrigine 25mg tablets (Arrow Generics Ltd)', 'PARACETAMOL tabs 500 MG', 'NAPROXEN EC', 'Venlafaxine Hydrochloride Tablets 37.5 mg', 'Aspirin-P42 300mg tab EC', 'PHENYTOIN tabs 100mg', 'PHENYTOIN SODIUM tabs 50mg', 'Tramadol Hydrochloride Capsules 50mg', 'LIDOCAINE gel 1% [MARTINDALE]', 'Morphine Sulfate  Tablets  20 mg', 'aspirin enteric coated tablets 75mg', 'oxcarbazepine tablets 300mg', 'Ibuprofen  Capsules  200 mg', 'Mexiletine 100mg capsules', 'co-codamol (codeine and paracetamol) tablets 8mg+500mg', 'Paracetamol 120mg/5ml oral solution paediatric', 'IBUPROFEN TAB 800mg', 'FENTANYL patch 25micrograms/hour [SANDOZ]', 'PARACETAMOL tabs 500 mg', 'DICLOFENAC SR capsules 75mg', 'IBUPROFEN RETARD tabs 800 MG', 'FENTANYL 100 MICROGRAMS/HOUR PATCH', 'PREGABALIN CAPSULES 300MG', 'PHENYTOIN SODIUM 100mg capsules', 'Mirtazapine 45mg tablets (Arrow Generics Ltd)', 'Mefenamic acid-P42 500mg tablet', 'Mirtazapine 30mg tablets (Teva UK Ltd)', 'Diclofenac Tablets 50 mg', 'AMITRIPTYLINE HCl sf oral soln 25mg/5ml', 'ASPIRIN 75mg e/c tablets', 'Aspirin Dispersible Low Dose Tablets 75 mg', 'ASPIRIN', 'LIDOCAINE injection 1% [PHOENIX]', 'Tapentadol  M/R tablets  200 mg', 'Tapentadol 200mg modified-release tablets', 'SODIUM VALPROATE tabs e/c 200 mg', 'CODEINE PHOSPHATE syrp 25mg/5ml', 'Tramadol Hydrochloride  M/R capsules  150 mg', 'MORPHINE SULPHATE ORAL SOLN 10MG/5ML', 'Co-Codamol Tab Effervesc.', 'Paracetamol 500mg / Phenylephrine 6.1mg tablets', 'BUPRENORPHINE seven day patch 20micrograms/hour', 'Ibuprofen 400mg tablets (Wockhardt UK Ltd)', 'DICLOFENAC DISPERSIBLE TABLETS 50 MG', 'Lidocaine 100mg/5ml (2%) solution for injection ampoules (Martindale Pharmaceuticals Ltd)', 'morphine sulfate 12 hour modified release capsules 50mg', 'Celecoxib CAPSULES 100MG', 'Oxycodone 50mg/1ml solution for injection ampoules (Wockhardt UK Ltd)', 'Apomorphine 50mg/5ml solution for injection ampoules', 'IBUPROFEN TAB 800', 'CODEINE DIABETIC LIN', 'Naproxen Sodium  Tablets  275 mg', 'Morphine Sulphate Mr TABS 10MG', 'Morphine Sulphate Mr TABS 5MG', 'DICLOFENAC CAPS M/R 75MG', 'Mefenamic Acid Cap 250mg', 'Oxycodone Hydrochloride CAPS 5MG', 'DICLOFENAC SODIUM SR TABLETS 75MG', 'VENLAFAXINE HYDROCHL TAB 75', 'VENLAFAXINE HYDROCHL TAB 37.5', 'Paracetamol-P42 500mg tablet', 'CODEINE PHOSPHATE diabetic linctus 15mg/5ml', 'Paracetamol 240mg suppositories', 'VENLAFAXINE HYDROCHLORIDE TAB 75', 'CODEINE PHOSPHATE + PARACETAMOL caps 8mg+500mg', 'Diclofenac Sodium M/R Tablets 75mg', 'Dihydrocodeine Tartrate M/R Tablets 60mg', 'Diclofenac Sodium Mr CAPS 100MG', 'Aspirin 300mg tablets (Actavis UK Ltd)', 'Celecoxib Capsules 200mg', 'DICLOFENAC AND MISOPROSTOL TABLETS 75MG + 200MCG', 'Aspirin 500mg / Papaveretum 7.71mg dispersible tablets sugar free', 'Aspirin 75mg tablets (A A H Pharmaceuticals Ltd)', 'Tramadol 75mg modified-release tablets', 'Co-codamol 8mg/500mg tablets (Actavis UK Ltd)', 'Dihydrocodeine Tartrate Mr TABS 60MG', 'DICLOFENAC MR TAB 75-P42 0', 'METHADONE HYDROCHLORIDE ORAL SOLN 1MG/ML', 'Venlafaxine  M/R Tablets  37.5 mg', 'Lidocaine 200mg/20ml (1%) / Adrenaline (base) 100microgra...', 'PHENYTOIN CAP 25mg', 'DICLOFENAC 75MG/MISO TAB', 'Diclofenac 100mg slow release', 'AMITRIPTYLINE tabs 10 mg', 'Codeine Phosphate  Syrup  25 mg/5 ml', 'Naproxen 250mg effervescent tablets sugar free', 'Diamorphine 5mg powder for solution for injection ampoules (A A H Pharmaceuticals Ltd)', 'TRAMADOL HYDROCHLORIDE SF SOLUBLE TABLETS 50MG', 'Naproxen 500mg tablets (Almus Pharmaceuticals Ltd)', 'Fentanyl 400microgram buccal tablets sugar free', 'Ibuprofen  Sugar-free suspension  100 mg/5 ml', 'TRAMADOL TABLETS 100MG SR', 'Kaolin And Morphine Mixture bp', 'Oxycodone Hydrochloride  M/R tablets  15 mg', 'BUPRENORPHINE 70 MICROGRAMS/HOUR PATCH', 'CELECOXIB 100mg CAP', 'DICLOFENAC 25/50', 'Venlafaxine Mr CAPSULES 75MG', 'Carbamazepine Mr TABS 200MG', 'Codeine Phosphate  Tablets  60 mg', 'NUSEALS ASPIRIN EC TABLETS 75MG', 'DICLOFENAC gel (100 g tube)', 'CARBAMAZEPINE chewable tab 200mg', 'CODEINE PHOSPHATE TABS 15MG', 'methadone tablets 5mg', 'co-codamol dispersible tablet 8mg+500mg', 'DICLOFENAC DUAL RELEASE CAPS 75MG', 'Codeine phosphate 30mg with Paracetamol 500mg capsules', 'IBUPROFEN 400MG TAB', 'FENTANYL patch 75mcg/hour', 'ORAMORPH SR tablets 60mg', 'Sodium valproate with valproic acid 300mg modified release tablets', 'VENLAFAXINE MR 75MG', 'SODIUM VALPROATE + VALPROIC ACID cr tab 300mg', 'DICLOFENAC enteric coated tablets 25mg', 'Phenytoin Sodium  Capsules  50 mg', 'Pregabalin 75mg/5ml oral suspension', 'AMITRIPTYLINE TAB 100mg', 'Dihydrocodeine Tartrate Mr TABS 90MG', 'LIDOCAINE HCl + ADRENALINE 1:80,000 dental cartridge 2%', 'mirtazapine tablets 45mg', 'Diclofenac 0.074% mouthwash sugar free', 'Gabapentin Capsules 400 mg', 'lidocaine with adrenaline (base) 1:200,000 injection 1.5%', 'Diclofenac Sodium M/R Capsules 100 mg', 'GABAPENTIN oral solution 250mg/5ml [BCM]', 'CARBAMAZEPINE controlled release tablet 200mg [LAGAP]', 'INDOMETHACIN CONTROLLED RELEASE TABLET 75 MG', 'Lidocaine With Adrenaline  Solution For Injection  2 % / ...', 'LIDOCAINE HYDROCHLORIDE ANTISEPTIC GEL 1%', 'CO-CODAMOL capsules 30mg + 500mg', 'ASPIRIN 300 MG E/C TAB', 'Mefenamic acid 250 mg capsules', 'Ibuprofen TABLETS 800MG', 'CODEINE & PARACETAMOL TAB', 'IBUPROFEN SACH 600MG', 'IBUPROFEN gel GEL 5%-P42 0', 'CARBAMAZEPINE tablets 100mg', 'IBUPROFEN-P42 600MG TABLET', 'PHENYTOIN SODIUM CAPSULES 300MG', 'Morphine Sulphate  M/R capsules  10 mg', 'DICLOFENAC caps m/r 75mg', 'DICLOFENAC RETARD TABS 100MG', 'CARBAMAZEPINE TAB 200', 'CARBAMAZEPINE 100MG TAB', 'Diamorphine 30mg powder for solution for injection ampoules (A A H Pharmaceuticals Ltd)', 'DICLOFENAC EC tab 25mg', 'Co-Codamol 8/500  Dispersible Tablets', 'INDOMETHACIN S/R CAP 75', 'INDOMETHACIN SUPPOSITORY 100 MG', 'Methadone 10mg/ml oral solution sugar free', 'MEFENAMIC ACID TABLETS 500 MG', 'AMITRIPTYLINE TABS 25 MG', 'SODIUM VALPROATE MR TABLETS 500MG', 'Lidocaine 200mg/10ml (2%) solution for injection ampoules', 'IBUPROFEN tablets 400mg [T & R]', 'Co-codamol Tablets N/A', 'NAPROXEN tabs e/c 500 mg', 'VENLAFAXINE 37.5mg', 'Lidocaine  Injection  1 %, 50 mg/5 ml ampoule', 'Lidocaine 0.33% / Cetylpyridinium chloride 0.1% oromucosal gel sugar free', 'Lidocaine 100mg/5ml (2%) solution for injection pre-fille...', 'AMITRIPTYLINE HYDROCHLORIDE tabs 10mg', 'Carbamazepine  Tablets  400 mg', 'DICLOFENAC SODIUM S/R TAB 100', 'Diclofenac Sodium  Eye drops  0.1 %', 'DEPO-MEDRONE WITH LIDOCAINE (1ML) INJ 40mg/ml', 'Diclofenac Sodium E/C, M/R capsules 75mg', 'Mefenamic acid 500 mg tablets', 'NAPROXEN 500MG/MISOP TAB', 'Venlafaxine Mr TABS 150MG', 'DEPO-MEDRONE WITH LIDOCAINE injection', 'Mirtazapine 15mg tablets (Ranbaxy (UK) Ltd)', 'PARACETAMOL-P42 soluble 500mg tabs', 'Paracetamol with dihydrocodeine tablets 500mg + 10mg', 'Codeine 30mg tablets (A A H Pharmaceuticals Ltd)', 'Indomethacin  M/R Tablets  25 mg', 'Sodium valproate 500mg gastro-resistant tablets (Zentiva)', 'Amitriptyline Hydrochloride  Sugar-free oral solution  50 mg/5 ml', 'Diclofenac Sodium  Eye Drops  0.1 %', 'Diclofenac Sodium SUPPS 100MG', 'DICLOFENAC DISPERSIB TAB 50', 'ASPIRIN EC 300 MGM TABS', 'Oxcarbazepine  Tablets  150 mg', 'PHENYTOIN CAP 100', 'Morphine sulphate 10mg/1ml solution for injection ampoules', 'IBUPROFEN 10% gel', 'DICLOFENAC 75MG TAB', 'Co-codamol 8mg/500mg tablets (Tillomed Laboratories Ltd)', 'Dihydrocodeine Tablets 30mg', 'DICLOFENAC SOD GEL 1G', 'Diclofenac Sodium M/R Capsules 100mg', 'AMITRIPTYLINE 100 MG TAB', 'NORTRIPTYLINE TAB 10', 'CO CODAMOL EFFERV', 'Tramadol Hydrochloride Mr TABLETS 100MG', 'LIDOCAINE HYDROCHLORIDE injection 2%', 'LIDOCAINE WITH CHLORHEXIDINE SYRINGE 6ML GEL', 'Gabapentin 800mg tablets (Teva UK Ltd)', 'Celecoxib 100mg capsules (Actavis UK Ltd)', 'CELECOXIB 100mg capsules', 'Dihydrocodeine with paracetamol 20mg+500mg tablets', 'Paracetamol & Metoclopramide 500 mg + 5 mg / sachet sachets', 'Paracetamol & Codeine phosphate 500 mg + 30 mg soluble table', 'Diclofenac sodium 50 mg enteric coated tablets', 'Morphine 150mg modified-release capsules', 'TRAMADOL tabs 100mg m/r-P42 0', '*Mefenamic acid 250mg capsule-P42 0', 'LAMOTRIGINE SF DISPERSIBLE TABLETS 25MG', 'Aspirin-P42 75mg Tablet', 'Ibuprofen Tablets S/R 800 mg', 'Ibuprofen 200mg tablets film coated (Actavis UK Ltd)', 'Lidocaine  Gel  2 %', 'VENLAFAXINE tablets 75mg', 'diclofenac sodium and misoprostol tablets 75mg + 200mcg', 'Ibuprofen 600mg tablets (IVAX Pharmaceuticals UK Ltd)', 'Co-Codamol 30/500 Effervescent Tablets', 'MIRTAZAPINE ORODISPERSIBLE 15MG TABLETS', 'LACOSAMIDE tabs 150mg', 'Ibuprofen 400mg tablet', 'IBUPROFEN tabs 400 mg -', 'TRAMADOL caps 50 mg -', 'diclofenac sodium suppository 100mg', 'LAMOTRIGINE 100MG', 'LIDOCAINE crm 4%', 'DICLOFENAC DUAL RELEASE CAPSULES 75 MG', 'Tramadol modified release tablet 100mg', 'DICLOFENAC 75MG/MISOPROSTOL200 COMBINATION TABLETS', 'aspirin modified release tablet 324mg', 'Gabapentin 800mg tablets (A A H Pharmaceuticals Ltd)', 'Fentanyl 75micrograms/hour transdermal patches (Actavis UK Ltd)', 'CODEINE PHOSPHATE SYR', 'ASPIRIN TAB 150mg', 'Oxycodone Hydrochloride  Solution for injection  50 mg/ml  1 ml ampoule', 'CODEINE PHOSPHATE ELI 15', 'FENTANYL 75 MICROGRAMS/HOUR PATCH', 'DEPO-MEDRONE WITH LIDOCAINE INJECTION', 'Amitriptyline Hydrochloride Mr CAPS 25MG', 'Mefenamic acid 250mg capsules (A A H Pharmaceuticals Ltd)', 'Lidocaine Hydrochloride GEL 2%', 'IBUPROFEN oral suspension 100mg/5ml [NEOLAB]', 'carbamazepine retard 200mg', 'Carbamazepine Retard 200 mg', 'DICLOFENAC SR 75MG TABS', 'GABAPENTIN capsules 100mg', 'diclofenac ec tablets 50mg', 'Lidocaine  Injection  1 %  50 mg/5 ml ampoule', 'IBUPROFEN caps 300mg', 'Diclofenac-P42 75mg Tab Sr', 'CELECOXIB', 'Amitriptyline Hydrochloride TABLETS 50MG', 'DICLOFENAC 75mg e/c+m/r caps', 'tabs Co Codamol 30/500', 'Depo-Medrone with Lidocaine suspension for injection 1ml via', 'etoricoxib tablets 60mg', 'DIHYDROCODEINE TARTRATE MR TABLETS 90MG', 'Morphine sulphate 30mg/1ml solution for injection ampoules', 'Oxycodone Hydrochloride  Capsules  20 mg', 'Naproxen Mr TABS 500MG', 'Co-codamol Effervescent T', 'Mefenamic Acid Dispersible Tablets 250 mg', 'AMITRIPTYLINE 50mg tabs', 'DEPO-MEDRONE WITH LIDOCAINE injection 40mg/ml + 10mg/ml [1ml', 'Lamotrigine 200mg tablets (Actavis UK Ltd)', 'Ibuprofen 200mg tablets (Wockhardt UK Ltd)', 'DEPO-MEDRONE WITH LIDOCAINE  (2ML) INJ 40mg/ml', 'DICLOFENAC POTASSIUM TABLETS 25MG', 'Mirtazapine  Orodispersible Tablets  45 mg', 'Naproxen Tablets e/c 500 mg', 'Buprenorphine  Transdermal patches  15 micrograms/hour', 'NORTRIPTYLINE TABS 10MG', 'Lidocaine 70mg / Tetracaine 70mg medicated plasters', 'Gabapentin  Oral solution  250 mg/5 ml', 'DICLOFENAC ec tab 25mg [APS]', 'Co-Codamol 8/500 Capsules', 'Pregabalin capsules 150mg', 'Tramadol And Paracetamol  Effervescent Tablets Sugar Free  37.5 mg + 325 mg', 'Tramadol Hydrochloride  M/R Capsules  50 mg', 'LIDOCAINE HYDROCHLORIDE ANTISEPTIC GEL 2%', 'DICLOFENAC SODIUM (3ML) INJ 25mg/ml', 'Ibuprofen 600mg tablets (Sandoz Ltd)', 'PARACETAMOL 500mg capsules', 'CO-CODAMOL caps 8/500', 'Phenytoin sodium 50mg capsules (A A H Pharmaceuticals Ltd)', 'DICLOFENAC AND MISOPROSTOL tablets 50mg + 200mi', 'Diclofenac 75mg/2ml solution for injection vials', 'Aspirin  Dispersible Tablets  300 mg', 'Codeine 60mg tablets (Teva UK Ltd)', 'Mirtazapine 15mg orodispersible tablets (Aurobindo Pharma Ltd)', 'IBUPROFEN SYR 100mg/5ml', 'Oxycodone 10mg/ml oral solution sugar free (Wockhardt UK Ltd)', 'Paracetamol Soluble TABLETS 500MG', 'CO-CODAMOL tablets TAB 30mg+500mg-P42 0', 'CODEINE PHOSPHATE + PARACETAMOL eff tab 30mg + 500mg', 'NAPRATEC naproxen tabs and misoprostol tabs', 'IBUPROFEN S/R CAP 300mg', 'Paracetamol Soluble Tablets 500 mg', 'Naproxen Ec TABLETS 250MG', 'INDOMETHACIN CAPS 25 MG', 'TRAMADOL extra mr tab 150mg', 'SODIUM VALPROATE sf liq 200mg/5ml', 'LAMOTRIGINE disp tab 5mg', 'Diclofenac sodium modified release capsule 75mg', 'Lidocaine Medicated Plaster 5%', 'Tramadol Capsules 50mg', 'Nu-seals aspirin ec 300mg Gastro-resistant tablet (Eli Lilly and Company Ltd)', 'Diclofenac sodium 25mg gastro-resistant tablets (Kent Pharmaceuticals Ltd)', 'Diclofenac Sodium M/R capsules 75mg', 'DICLOFENAC modified release tablet 75mg [IVAX]', 'Indomethacin Capsules 50 mg', 'Phenytoin sodium tablets 100mg', 'AMITRIPTYLINE 25MG', 'Diclofenac Sodium Tablets 25 mg', 'TRAMADOL inj 100mg/2ml', 'diclofenac sodium modified release tablet 75mg', 'ASPIRIN & PAPAVERUTUM TAB 10mg', 'ASPIRIN (Generic Manuf) (form not specified)', 'CARE IBUPROFEN tablets 400mg [T & R]', 'DICLOFENAC SOD TAB 75MG M/R', 'Diclofenac Na 100mg m/r tab', 'Tramadol Hydrochloride Effervescent Powder 50 mg/sachet', 'venlafaxine (roi) modified release capsules 37.5mg', 'Ibuprofen sodium dihydrate 400mg tablets', 'DICLOFENAC SODIUM gr mr capsules 75mg', 'Venlafaxine 75mg tablets (A A H Pharmaceuticals Ltd)', 'PARACETAMOL SUSP 250MG/5ML', 'Oramorph Oral SOLN 10MG/5ML', 'DICLOFENAC 25 MG TABS', 'amitriptyline hydrochloride capsules 50 mg', 'Paracetamol  Mixture  250 mg/5 ml', 'pseudoephedrine with brompheniramine and codeine phosphate elixir', 'DIHYDROCODEINE 30mg tabs', 'Co-codamol Dispersible Ta', 'AMITRIPTYLINE 25MGS SR CAPS', 'DICLOFENAC disp TABS 50MG', 'Morphine tartrate 10mg/1ml / Cyclizine tartrate 50mg/1ml solution for injection ampoules', 'Lidocaine  Cream  2 %', 'IBUPROFEN tabs 400 mg IBUPROFEN 400mg tablets', 'HYDROMORPHONE HCl caps 8mg', 'Morphine sulphate 12 20mg Modified-release capsule', 'CELECOXIB 100MG CAPS-P42 0', 'Dihydrocodeine 60mg modified-release tablets', 'Co-codamol 8mg/500mg effervescent tablets', 'Ibuprofen 400mg tablets', 'Aspirin 75mg dispersible tablets', 'Amitriptyline 25mg tablets', 'Ibuprofen 5% gel', 'Diclofenac Sodium  M/R Tablets  75 mg', 'Naproxen 250mg tablets', 'Paracetamol 500mg / Dihydrocodeine 30mg tablets', 'Diclofenac sodium 50mg gastro-resistant tablets', 'Oxycodone 10mg modified-release tablets', 'Paracetamol 500mg caplets (Wockhardt UK Ltd)', 'Sodium valproate 500mg modified-release tablets', 'Paracetamol 500mg capsules', 'diclofenac sodium EC tablets 50mg', 'Lamotrigine 25mg tablets', 'lidocaine injection 2%', 'Aspirin  Tablets  75 mg', 'Diclofenac sodium 3% gel', 'Mirtazapine 45mg orodispersible tablets', 'Morphine 10mg modified-release tablets', 'Tramadol 200mg modified-release capsules', 'Pregabalin 150mg capsules', 'Paracetamol 500mg / Metoclopramide 5mg effervescent powder sachets sugar free', 'PARACETAMOL tabs 500mg', 'Naproxen 375mg gastro-resistant tablets', 'Diclofenac sodium 100mg modified-release capsules', 'Paracetamol 500mg caplets (A A H Pharmaceuticals Ltd)', 'Buprenorphine 10micrograms/hour transdermal patches', 'Venlafaxine 150mg modified-release tablets', 'Diclofenac sodium 100mg modified-release tablets', 'Pregabalin 75mg capsules', 'Methylprednisolone 40mg/1ml / Lidocaine 10mg/1ml (1%) suspension for injection vials', 'CO-CODAMOL tabs 8mg+500mg', 'Morphine 10mg modified-release capsules', 'Mirtazapine 30mg orodispersible tablets', 'Minims lidocaine and fluorescein eye drops 0.5ml unit dose (Bausch & Lomb UK Ltd)', 'CODEINE PHOSPHATE tabs 15mg', 'ASPIRIN tabs 75mg', 'Pregabalin 100mg capsules', 'Paracetamol 500mg / Metoclopramide 5mg tablets', 'Oxycodone 40mg modified-release tablets', 'CO-CODAMOL TAB', 'MIRTAZAPINE tabs 30mg', 'Lamotrigine  Tablets  200 mg', 'Lamotrigine  Tablets  50 mg', 'Duloxetine 40mg gastro-resistant capsules', 'Duloxetine 20mg gastro-resistant capsules', 'Phenytoin sodium 50mg capsules', 'Dihydrocodeine 40mg tablets', 'Gabapentin  Capsules  100 mg', 'Buprenorphine 20 Micrograms / Hour PATCH', 'Morphine sulfate 10mg/1ml solution for injection ampoules', 'Aspirin Dispersible Tablets 75 mg', 'Depo-Medrone with Lidocaine suspension for injection 2ml ...', 'DICLOFENAC SODIUM mr tab 100mg', 'Sodium valproate 200mg modified-release tablets', 'Lidocaine 200mg/20ml (1%) / Adrenaline (base) 100micrograms/20ml (1 in 200,000) solution for injection vials', 'Aspirin  E/C Tablets  75 mg', 'Venlafaxine 225mg modified-release tablets', 'Co-codamol tablets 8mg+500mg', 'Aspirin 75mg dispersible tablets (Actavis UK Ltd)', 'Celecoxib  Capsules  200 mg', 'DICLOFENAC SODIUM disp tab 50mg', 'co-codamol (codeine and paracetamol) (form not specified)', 'Naproxen  Tablets  500 mg', 'CO-CODAMOL (codeine & paracetamol) tabs 8mg+500mg', 'Co-Codamol 8mg/500mg Effervescent TABS', 'Amitriptyline 25mg modified-release capsules', 'Paracetamol 500mg soluble tablets (Teva UK Ltd)', 'Lidocaine With Chlorhexidine Syringe 11ml GEL', 'Amitriptyline 50mg modified-release capsules', 'Diclofenac And Misoprostol  Tablets  50 mg + 200 micrograms', 'DICLOFENAC tabs 50 mg', 'IBUPROFEN gel 10%', 'Amitriptyline 25mg tablets (Teva UK Ltd)', 'Co-Codamol 30mg/500mg CAPS', 'Morphine 60mg modified-release tablets', 'Ibuprofen lysine 400mg tablets', 'Paracetamol 250mg/5ml oral suspension sugar free', 'Naproxen 500mg tablets and Misoprostol 200microgram tablets', 'Nortriptyline TABS 25MG', 'Nortriptyline 25mg tablets (A A H Pharmaceuticals Ltd)', 'LIDOCAINE injection preservative-free 2% [HAMELN]', 'Paracetamol 250mg/5ml oral suspension', 'IBUPROFEN tabs 400 MG', 'Pregabalin  Capsules  75 mg', 'Diamorphine 5mg powder for solution for injection ampoules', 'MEFENAMIC ACID TABLETS 500MG', 'Co-Codamol 8mg/500mg CAPS', 'DICLOFENAC SODIUM EC TABLETS 25MG', 'Naproxen sodium 275mg tablets', 'Ibuprofen 400mg caplets (Lloyds Pharmacy Ltd)', 'Gabapentin 400mg capsules', 'Gabapentin CAPS 300MG', 'Mirtazapine  Tablets  15 mg', 'Co-codamol 15mg/500mg effervescent tablets sugar free', 'Diclofenac Sodium M/R Tablets 75 mg', 'MEFENAMIC ACID CAPSULES 250MG', 'Lidocaine 100mg/5ml (2%) solution for injection ampoules', 'PHENYTOIN TAB 100', 'Lidocaine  Medicated Plaster  5 %', 'DICLOFENAC DUAL RELEASE CAP 75mg', 'TRAMADOL HYDROCHLORIDE MR TABLETS 150MG', 'MEFENAMIC ACID caps 250mg', 'Diclofenac Potassium  Tablets  50 mg', 'MIRTAZAPINE tabs 45mg', 'Phenytoin sodium 300mg capsules', 'TRAMADOL HYDROCHLORIDE MR CAPSULES 100MG', 'Buprenorphine 10 Micrograms/Hour PATCH', 'Diclofenac Sodium  E/c tablets  25 mg', 'Ibuprofen 600mg effervescent granules sachets', 'Lidocaine 200mg/20ml (1%) solution for injection vials', 'CODEINE PHOSPHATE linctus 15mg/5ml', 'IBUPROFEN TABS 600MG', 'Carbamazepine 400mg modified-release tablets', 'Co-codamol effervescent tablet 30mg + 500mg', 'TRAMADOL HCL TAB 50mg', 'Kaolin and Morphine mixture', 'Paracetamol 120mg/5ml oral solution paediatric sugar free', 'Lidocaine 44mg/2.2ml (2%) / Adrenaline (base) 27.5micrograms/2.2ml (1 in 80,000) solution for injection cartridges', 'MORPHINE SULPHATE mr tab 30mg', 'Fentanyl 25micrograms/hour transdermal patches (A A H Pharmaceuticals Ltd)', 'Ibuprofen 400mg tablets (Almus Pharmaceuticals Ltd)', 'lidocaine hydrochloride ointment 5%', 'ASPIRIN 75mg dispersible tablets', 'DEPO-MEDRONE WITH LIDOCAINE (1ML) INJ', 'Tapentadol  M/R tablets  50 mg', 'Paracetamol 500mg tablets (Zentiva)', 'CODEINE PHOSPHATE LINCT 15MG/5ML', 'Co-Codamol 30/500  Caplets', 'Codeine 15mg tablets (Teva UK Ltd)', 'NAPROXEN TAB 250mg', 'co-proxamol (dextropropoxyphene and paracetamol) suspension sugar-free 32.5mg + 325mg/5ml', 'Diclofenac Sodium  Gel  3 %', 'Lamotrigine TABS 25MG', 'GABAPENTIN caps 300mg', 'Phenytoin sodium 50mg capsules (Flynn Pharma Ltd)', 'DICLOFENAC dual release caps 75mg', 'Naproxen  E/c tablets  375 mg', 'TRAMADOL HYDROCHLORIDE MR CAPSULES 200MG', 'Tramadol Hydrochloride  M/R tablets  100 mg', 'Aspirin  E/c tablets  75 mg', 'Amitriptyline Hydrochloride TABS 50MG', 'Naproxen Tablets 500 mg', 'Amitriptyline hydrochloride 50 mg tablets', 'Pregabalin 225mg capsules', 'Aspirin E/C Tablets 75 mg', 'tramadol 24 hour modified release tablets 100mg', 'ASPIRIN 75mg soluble tablets', 'TRAMADOL mr cap 100mg', 'IBUPROFEN TAB 600', 'PARACETAMOL-P42 500MG TABLET', 'TRAMADOL HYDROCHLORIDE MR CAPSULES 50MG', 'Lidocaine 50mg/10ml (0.5%) solution for injection ampoules', 'Lacosamide 100mg tablets', 'AMITRIPTYLINE TAB 25mg', 'NAPROXEN TABLETS 500 MG', 'MEFENAMIC ACID TABS 500MG', 'NORTRIPTYLINE TAB 10mg', 'Mirtazapine TABS 15MG', 'Venlafaxine  M/R tablets  225 mg', 'Oxycodone 60mg modified-release tablets', 'Oxycodone 10mg/ml oral solution sugar free', 'paracetamol (form not specified)', 'DICLOFENAC SODIUM SUP 25mg', 'CO-CODAMOL 30/500 TABLETS 500MG PARACETAMOL-P42 0', 'Ibuprofen Mr TABS 800MG', 'SODIUM VALPROATE ec tab 200mg', 'DICLOFENAC SODIUM RETARD CAP 100mg', 'Diclofenac sodium 75mg gastro-resistant modified-release ...', 'VENLAFAXINE MR CAPSULES 75MG', 'Gabapentin capsules 300mg', 'Pregabalin  Capsules  300 mg', 'VENLAFAXINE mr tab 150mg', 'IBUPROFEN tabs 600mg', 'NAPROXEN tabs 500 mg', 'PARACETAMOL tablets 500mg', 'Lamotrigine 25mg tablets (Actavis UK Ltd)', 'PARACETAMOL 500MG/30MG CODEINE PHOSPHATE CAP', 'Diclofenac enteric coated tablets 25mg', 'Nortriptyline TABS 10MG', 'CODEINE PHOSPHATE + PARACETAMOL tabs 30mg + 500mg', 'Mirtazapine TABS 15mg', 'DICLOFENAC SR 75 MGM TABLET', 'NAPROXEN ec tab 375mg', 'DIHYDROCODEINE + PARACETAMOL tabs 10mg + 500mg', 'PHENYTOIN SODIUM caps 300mg', 'DULOXETINE GASTRO RESISTANT CAPSULES 60MG', 'ASPIRIN enteric coated tablets 75mg', 'CODEINE PHOSPHATE tabs 60mg', 'Ibuprofen 100mg/5ml oral suspension', 'Venlafaxine  M/R tablets  75 mg', 'Venlafaxine  M/R tablets  37.5 mg', 'Codeine phosphate with paracetamol tablets 8mg + 500mg', 'Morphine 100mg modified-release tablets', 'IBUPROFEN 5% gel', 'Lidocaine 50mg/5ml (1%) solution for injection Sure-Amp ampoules (AMCo)', 'Duloxetine Gastro Resistant CAPS 60MG', 'Codeine 15mg/5ml linctus (Thornton & Ross Ltd)', 'TRAMADOL CAPS 50MG', 'Ibuprofen 5% / Levomenthol 3% gel', 'LIDOCAINE HYDROCHLORIDE injection 1% [MERCURY]', 'OXCARBAZEPINE tabs 300mg', 'IBUPROFEN TABS 800MG TAB 800', 'IBUPROFEN tabs 600 mg', 'DICLOFENAC 75MG TABS S/R', 'CELECOXIB capsules 100mg', 'Amitriptyline 10mg tablets (A A H Pharmaceuticals Ltd)', 'PARACETAMOL + CODEINE & BUCLIZINE ( Combination Pack) tabs', 'Paracetamol 120mg/5ml oral suspension paediatric sugar free', 'NAPROXEN enteric coated tablets TAB 250mg-P42 0', 'IBUPROFEN TABLETS 400 MG', 'ORAMORPH unit dose vial 10mg/5ml', 'PREGABALIN caps 50mg', 'Paracetamol 325mg / Isometheptene 65mg capsules', 'DICLOFENAC modified release tablet 75mg', 'CARBAMAZEPINE 200mg tabs', 'Amitriptyline 10mg tablets (Teva UK Ltd)', 'FENTANYL patch 25mcg/hour', 'PARACETAMOL + CODEINE PHOSPHATE tabs 500mg + 15mg', 'Sodium Valproate  E/C Tablets  200 mg', 'AMITRIPTYLINE HYDROCHLORIDE S/R CAP 75mg', 'Ibuprofen M/R Tablets 800 mg', 'Oxycodone Hydrochloride  M/R tablets  10 mg', 'Naproxen TABS 250MG', 'CARBAMAZEPINE TABLETS 100 MG', 'CO-CODAMOL 8/500MG TABLETS', 'DICLOFENAC EC TAB 50MG', 'Ibuprofen gel 5%', 'Co-codamol 30mg/500mg tablets (A A H Pharmaceuticals Ltd)', 'Etoricoxib TABS 90MG', 'Fentanyl 100microgram buccal tablets sugar free', 'CO-CODAMOL dispersible tablet 8mg+500mg', 'TRAMADOL HYDROCHLORIDE MR TABLETS 100MG', 'Co-codamol 30mg/500mg tablets (Almus Pharmaceuticals Ltd)', 'Buprenorphine 2mg sublingual tablets sugar free', 'Diclofenac sodium 50mg gastro-resistant tablets (Sandoz Ltd)', 'Naproxen 250mg gastro-resistant tablets (Teva UK Ltd)', 'CO-CODAMOL sf eff pwdr 30mg + 500mg', 'DICLOFENAC SODIUM SR CAP 75', 'Tramadol 50mg capsules (Genus Pharmaceuticals Ltd)', 'Paracetamol 500mg caplets (Alliance Healthcare (Distribut...', 'Paracetamol And Metoclopramide Tablets 500 mg + 5 mg', 'Paracetamol 500mg soluble tablets (Fannin UK Ltd)', 'Morphine Sulfate  M/R capsules  10 mg', 'Lidocaine 100mg/20ml (0.5%) / Adrenaline (base) 100micrograms/20ml (1 in 200,000) solution for injection vials', 'CO-CODAMOL TABLETS 500 MG + 8 MG', 'Ibuprofen 200mg orodispersible tablets sugar free', 'DICLOFENAC SODIUM (AS DIETHYLAMMONIUM) gel 1%', 'Oxcarbazepine  Tablets  300 mg', 'Fentanyl 100microgram sublingual tablets sugar free', 'BUPRENORPHINE patch 52.5mcg/hour', 'Naproxen 500mg suppositories', 'Tapentadol 100mg modified-release tablets', 'DIPYRIDAMOLE + ASPIRIN mr cap 200mg + 25mg', 'DICLOFENAC SODIUM TAB 25mg', 'CODEINE PHOS/IBUPROFEN SR (20MG/300MG) TAB', 'INDOMETHACIN mr cap 75mg', 'Mirtazapine 30mg tablets (Actavis UK Ltd)', 'IBUPROFEN & CODEINE mr tab 300mg + 20mg', 'LIDOCAINE patch 5%', 'Pregabalin  Capsules  225 mg', 'ibuprofen gel 10%', 'Oxcarbazepine 600mg tablets', 'Codeine  Linctus sugar free  15 mg/5 ml', 'Morphine Sulphate TABS 10MG', 'PARACETAMOL + PSEUDOEPHEDRINE HCl tabs 500mg+60mg', 'IBUPROFEN TAB 400MG', 'Fentanyl 37.5microgram/hour transdermal patches', 'IBUPROFEN TAB 200mg', 'Phenytoin Sodium  Capsules  100 mg', 'Venlafaxine 50mg tablets', 'Co-codamol 30mg/500mg capsules (A A H Pharmaceuticals Ltd)', 'OXYCODONE HYDROCHLORIDE MR TABLETS 10MG', 'Cathejell + lidocaine gel [TELE MED]', 'DIHYDROCODEINE tablets 30mg', 'Mefenamic Acid Tablets 500 mg', 'Oxycodone 80mg modified-release tablets', 'Morphine sulphate 10mg/5ml oral solution', 'CO-CODAMOL (codeine & paracetamol) eff tab 8mg+500mg', 'CODEINE linctus [BERK]', 'Naproxen 250mg tablets (Almus Pharmaceuticals Ltd)', 'PARACETAMOL WITH CODEINE PHOSPHATE capsules 500', 'DICLOFENAC SODIUM SR CAP 75mg', 'Co-codamol 30mg/500mg tablets (Teva UK Ltd)', 'Paracetamol 500mg suppositories', 'PARACETAMOL 500mg tablets', 'Morphine 30mg modified-release granules sachets sugar free', 'Tramadol Hydrochloride Mr TABS 200MG', 'Tramadol Hydrochloride Mr TABS 100MG', 'GABAPENTIN caps & tabs 300mg + 600mg', 'VENLAFAXINE TABS 75MG', 'Pregabalin Capsules 75mg', 'MEFENAMIC ACID CAPS 250 MG', 'Paracetamol 500mg tablets (Aspar Pharmaceuticals Ltd)', 'Venlafaxine Hydrochloride  Tablets  37.5 mg', 'CO-CODAMOL (codeine & paracetamol) caps 8mg+500mg', 'DICLOFENAC injection 75mg/3ml [ANTIGEN]', 'Diclofenac Sodium Tablets 50 mg', 'TRAMADOL HCL CAP 50mg', 'Nortriptyline 10mg tablets (Teva UK Ltd)', 'DICLOFENAC gel 1%', 'Amitriptyline Hydrochloride Tablets 25mg', 'CODEINE linctus [FAMILY H]', 'Dihydrocodeine Tartrate  M/R tablets  120 mg', 'Naproxen 250mg gastro-resistant tablets (Almus Pharmaceuticals Ltd)', 'Ibuprofen 600 mg tablets', 'Pregabalin CAPS 75MG', 'Codeine 15mg tablets (Ranbaxy (UK) Ltd)', 'ASPIRIN + CODEINE tabs 400mg + 8mg', 'IBUPROFEN 400MG', 'morphine sulfate injection 10mg/ml', 'Aspirin with codeine tablets 400mg + 8mg', 'AMITRIPTYLINE 10mg tablets', 'IBUPROFEN tabs s/r 800 mg', 'TRAMADOL 12hr mr tabs 50mg', 'PARACETAMOL & DIHYDROCODEINE tabs 500mg/20mg', 'DICLOFENAC 50MG TAB', 'INDOMETHACIN supp 100mg', 'FENTANYL patch 12micrograms/hour [HILLCROSS]', 'IBUPROFEN TABS 600 MG', 'Paracetamol Sf Oral SUSP 250MG/5ML', 'Morphine sulfate 30mg suppositories', 'Tramadol Hydrochloride CAPSULES 50MG', 'METOCLOPRAMIDE HYD. 5mg/325mg ASPIRIN TAB', 'Amitriptyline 10mg tablets (IVAX Pharmaceuticals UK Ltd)', 'Morphine sulfate 60mg/2ml solution for injection ampoules', 'Morphine sulfate powder', 'OXCARBAZEPINE tabs 600mg', 'diamorphine hydrochloride powder for solution for injection 100mg', 'Paracetamol  Soluble Tablets  500 mg', 'Amitriptyline 50mg/5ml oral solution sugar free', 'Codeine 30mg tablets (Teva UK Ltd)', 'LAMOTRIGINE TAB 100mg', 'Diclofenac Sodium  Tablets  25 mg', 'co-dydramol (dihydrocodeine and paracetamol) tablets 7.46mg + 500mg', 'Lidocaine 40mg/2ml (2%) solution for injection ampoules (Hameln Pharmaceuticals Ltd)', 'Ibutop Ralgex Ibuprofen 5% gel (SSL International Plc)', 'Amitriptyline 50mg tablets (Almus Pharmaceuticals Ltd)', 'IBUPROFEN 600mg tablets', 'Mirtazapine 30mg tablets (Mylan Ltd)', 'Tramadol 50mg capsules (A A H Pharmaceuticals Ltd)', 'Tramadol 37.5mg / Paracetamol 325mg tablets (Teva UK Ltd)', 'Venlafaxine M/R Capsules 75 mg', 'Venlafaxine 75mg tablets (Teva UK Ltd)', 'Nortriptyline Tablets 10 mg', 'Aspirin 75mg gastro-resistant tablets (Teva UK Ltd)', 'Co-codamol 30mg/500mg tablets (Zentiva)', 'Oramorph  Oral solution  10 mg/5 ml', 'CO-CODAMOL 30/500 EFFERVESCENT TAB', 'Amitriptyline Hydrochloride  Oral solution  10 mg/5 ml', 'Naproxen 250mg Tabs', 'Lidocaine 4% / Fluorescein 0.25% eye drops 0.5ml unit dose preservative free', 'Buprenorphine  Transdermal patches  10 micrograms/hour', 'Amitriptyline tablets 25mg', 'Gabapentin Capsules 300 mg', 'Amitriptyline 25mg tablets (Sandoz Ltd)', 'DEPOMEDRONE WITH LIDOCAINE 40MG/1ML', 'Venlafaxine 37.5mg tablets (Teva UK Ltd)', 'Lidocaine 100mg/10ml (1%) solution for injection pre-filled syringes (Martindale Pharmaceuticals Ltd)', 'Naproxen Tablets 250mg', 'Gabapentin 300mg capsules (Actavis UK Ltd)', 'Aspirin Tablets disp. 75 mg', 'DICLOFENAC SODIUM MR TABLETS 100MG', 'Ibuprofen 200mg tablets (Almus Pharmaceuticals Ltd)', 'DICLOFENAC enteric coated tablets 50mg [ALMUS]', 'DIHYDROCODEINE TABLETS 30MG', 'Naproxen 500mg gastro-resistant tablets (Teva UK Ltd)', 'Paracetamol 500mg with codeine phosphate 30 mg tablet', 'Paracetamol 500mg capsules (Actavis UK Ltd)', 'Ibuprofen tablets 600mg', 'ARTHROTEC-DICLOFENAC 50MG/MISOPRO 200MCG TAB', 'Paracetamol 500mg with codeine phosphate 30mg effervescent tablet', 'GABAPENTIN TABLETS 800MG', 'Mefenamic acid 250mg capsules (Arrow Generics Ltd)', 'PREGABALIN CAPSULES 25MG', 'NAPROXEN 500MG/MISOPROSTOL 200MCG TAB', 'DICLOFENAC EC tabs 25 MG', 'Buprenorphine  Transdermal patches  52.5 micrograms/hour', 'Co-Codamol 30/500 Capsules', 'Etoricoxib Tablets 90 mg', 'Aspirin 75 mg dispersible tablets', 'Methylprednisolone 80mg/2ml / Lidocaine 20mg/2ml (1%) sus...', 'Fentanyl  Transdermal patches  75 micrograms/hour', 'Oxycodone Hydrochloride  Solution  Sugar Free 5 mg/5 ml', 'CO-CODAMOL TABS 8/500MG.', 'Gabapentin TABS 600MG', 'LIDOCAINE HCl + CHLORHEXIDINE gel 2% + 0.25%', 'TRAMADOL 50MG', 'ORAMORPH SOL 2mg/ml', 'Methadone 5mg/ml oral solution', 'Diclofenac Retard Tablets 100 mg', 'MORPHINE SULPHATE mr cap 30mg', 'CO-CODAMOL (Generic Manuf) (form not specified)', 'NAPROXEN SODIUM tabs 275mg', 'Morphine sulphate 12 100mg Modified-release capsule', 'Tramadol 37.5mg / Paracetamol 325mg effervescent tablets ...', 'DICLOFENAC TABLETS 75MG SR', 'ASPIRIN 75mg', 'Tramadol And Paracetamol  Tablets  37.5 mg + 325 mg', 'ASPIRIN EC 75MG TAB', 'Ibuprofen  Cream  5 %', 'Duloxetine Gastro Resistant CAPS 20MG', 'Ibuprofen  M/R Tablets  800 mg', 'Paracetamol 500mg caplets (Accord Healthcare Ltd)', 'CO-CODAMOL TABLETS', 'Pregabalin 75mg capsules (Teva UK Ltd)', 'GABAPENTIN capsules 300mg', 'Morphine sulfate 10mg/5ml oral solution (Actavis UK Ltd)', 'ASPIRIN NUSEAL 75MG TABLET', 'Venlafaxine  M/R capsules  37.5 mg', 'TRAMADOL HCL CAP 50MG', 'Aspirin 300mg dispersible tablets (Aspar Pharmaceuticals Ltd)', 'DIHYDROCODEINE + PARACETAMOL tabs 30mg + 500mg', 'IBUPROFEN & CODEINE tabs 200mg + 12.8mg', 'sodium valproate with valproic acid modified release tablet 200mg', 'Paracetamol & Codeine phosphate 500 mg + 30 mg capsules', 'VENLAFAXINE 75mg tablets', 'VENLAFAXINE 37.5mg tablets', 'KAOLIN MORPHINE SF MIXT', 'diclofenac enteric coated tablets 50mg', 'OXYCODONE HCl caps 20mg', 'Diclofenac gel 1.16%', 'Sodium Valproate  M/R capsules  300 mg', 'DIHYDROCODEINE TARTR TAB 120', 'Diclofenac potassium 50mg tablets (Actavis UK Ltd)', 'CO-CODAMOL DISPERSIBLE TABLET', 'naproxen tablets 375mg', 'MEFENAMIC ACID CAPS 250MG', 'DICLOFENAC SODIUM RE CAP 100', 'Paracetamol & Codeine phosphate 500 mg + 30 mg tablets', 'Phenytoin sodium 100mg tablets (A A H Pharmaceuticals Ltd)', 'Ibuprofen 10 % gel', 'perphenazine with amitriptyline tablets 2mg + 25mg', 'MORPHINE SULPHATE tabs 20mg', 'MORPHINE SULPHATE 12hr mr caps 10mg', 'CARBAMAZEPINE tablets TAB 100mg-P42 0', 'DULOXETINE GASTRO RESISTANT CAPSULES 20MG', 'DULOXETINE GASTRO RESISTANT CAPSULES 30MG', 'OXYCODONE HCl + NALOXONE HCl mr tab 20mg + 10mg', 'DICLOFENAC SODIUM AND MISOPROSTOL tablets 50mg+200mcg', 'CODEINE PHOSPHATE + PARACETAMOL caps 30mg + 500mg', 'Paracetamol 500mg caplets (J M McGill Ltd)', 'Carbamazepine controlled release tablet 200mg', 'NORTRIPTYLINE tablets 25mg', 'Phenytoin Sodium CAPS 300MG', 'Mirtazapine  Orodispersible tablets  45 mg', 'Mirtazapine 15mg tablets (Almus Pharmaceuticals Ltd)', 'Duloxetine 60mg gastro-resistant capsules (Actavis UK Ltd)', 'Naproxen Ec TABS 375MG', 'DICLOFENAC SODIUM TABLET 50MG', 'CO-CODAMOL (codeine & paracetamol) eff tab 15mg + 500mg', 'DICLOFENAC SODIUM modified release tablet 75mg', 'Paracetamol 250mg/5ml oral suspension (Rosemont Pharmaceuticals Ltd)', 'Pregabalin 20mg/ml oral solution sugar free', 'Etoricoxib TABLETS 60MG', 'MIRTAZAPINE TAB 30mg', 'NU-SEALS ASPIRIN EC tab 75mg', 'Aspirin 300mg modified-release tablets', 'Lamotrigine 25mg dispersible tablets sugar free', 'CO-CODAMOL 30/500 TABS', 'Ibuprofen  Syrup  100 mg/5 ml', 'MORPHINE SULPHATE TABLETS 20MG', 'Lamotrigine  Dispersible tablets  Sugar Free 25 mg', 'Lidocaine 100mg/5ml (2%) solution for injection Sure-Amp ...', 'Codeine Linctus Sugar Free BP', 'Naproxen Ec TABLETS 500MG', 'CODEINE PHOSPHATE + PARACETAMOL tabs 15mg + 500mg', 'Diclofenac [BERK] tablets 50mg', 'BUPRENORPHINE seven day patch 5micrograms/hour', 'DEXIBUPROFEN tabs 300mg', 'Codeine Phosphate Tablets 30 mg', 'dihydrocodeine (form not specified)', 'TRAMADOL HCL TAB 100mg', 'MIRTAZAPINE tablets 30mg', 'Ibuprofen  M/R capsules  300 mg', 'Diclofenac modified release tablet 100mg', 'Venlafaxine Hydrochloride Tablets 75 mg', 'Mefenamic Acid Capsules 250mg', 'etoricoxib TABS 90mg', 'paracetamol with codeine phosphate tablets 500mg + 8mg', 'Gabapentin 50mg/ml oral solution sugar free (Rosemont Pharmaceuticals Ltd)', 'GABAPENTIN CAP 300mg', 'PHENYTOIN CAP 50mg', 'Gabapentin  Oral Solution  Sugar Free  250 mg/5 ml (50 mg/ml)', 'HYDROCORTISONE + LIDOCAINE rectal spray 0.2% + 1%', 'TRAMADOL tabs 200 mg m/r - SPECIFY SR/XL ! -', 'CO-CODAMOL 12.8MG/500MG TABLETS', 'Diclofenac sodium modified release tablet 75mg', 'pregabalin capsules 75mg', 'PARACETAMOL soluble tablet 500mg', 'NAPROXEN tablets 250mg [M&A PHARM]', 'Amitriptyline 50mg tablets (Actavis UK Ltd)', 'lamotrigine tablets 25mg', 'sodium valproate with valproic acid modified release tablet 300mg', 'Oxycodone Hydrochloride Mr TABS 20MG', 'lamotrigine tablets 50mg', 'Diamorphine Hydrochloride  Injection  5mg/amp', 'Co-codamol Tablets', 'Indomethacin 25mg Caps', 'Co-codamol 8mg/500mg caplets (Doncaster Pharmaceuticals Ltd)', 'Paracetamol soluble tablet 500mg', 'FENTANYL patch 75micrograms/hour [SANDOZ]', 'DEPO-MEDRONE WITH LIDOCAINE (form not specified)', 'DICLOFENAC MR TAB 75mg', 'PARACETAMOL + CODEIN PHOSPHAT CAPS 500MG+30MG', 'CARBAMAZEPINE tablets 200mg [IVAX]', 'Mirtazapine Orodispersible TABS 45MG', 'Co-codamol 30mg/500mg capsules (Actavis UK Ltd)', 'SOLUBLE ASPIRIN', 'PARACETAMOL WITH CODEINE PHOSPHATE effervescent tab 500mg', 'Ibuprofen 400mg tablets (Galpharm International Ltd)', 'Carbamazepine  C/R Tablets  400 mg', 'INDOMETHACIN SR TAB 50mg', 'Naproxen 500mg gastro-resistant tablets (Mylan Ltd)', 'Dihydrocodeine-P42 30mg tablet', 'Co-codamol capsules 30mg + 500mg', 'DICLOFENAC GEL 100', 'phenytoin capsules 50mg', 'Gabapentin TABLETS 600MG', 'IBUPROFEN MODIFIED RELEASE TABLETS 800MG', 'DICLOFENAC MODIFIED RELEASE TABLETS 100MG', 'LIDOCAINE + AMINOACRIDINE sf gel 0.66% + 0.05%', 'Ibuprofen 600mg tablets (Almus Pharmaceuticals Ltd)', 'TRAMADOL modified release capsule 50mg', 'CARBAMAZEPINE TAB 200MG M/R', 'Buprenorphine  Sublingual tablets  Sugar Free 200 micrograms', 'CARBAMAZEPINE TAB C/R 200MG', 'Lidocaine  Injection  1 %  100 mg/10 ml ampoule', 'VENLAFAXINE XL TABS 75MG TAB', 'Codeine Phosphate TABS 60MG', 'Naproxen Sodium TABS 275MG', 'Morphine Sulphate  M/R capsules  30 mg', 'DICLOFENAC 100mg m/r tablets', 'Fentanyl 12 Micrograms/Hour PATCH', 'Ibuprofen SPRAY 5%', 'Codeine phosphate 8mg with paracetamol 500mg tablets', 'Diclofenac Sodium  M/R Tablets  100 mg', 'diclofenac sodium modified release capsule 100mg', 'DICLOFENAC disp tabs 50 MG', 'Lidocaine 2% gel (Teva UK Ltd)', 'Codeine phosphate tablets 30mg', 'Sodium Valproate Mr TABS 300MG', 'TRAMADOL soluble tablet 50mg', 'Gabapentin 600mg tablets (A A H Pharmaceuticals Ltd)', 'Ibuprofen gel 10%', 'ASPIRIN NON TARIFF EC TABLETS 75MG', 'Ibuprofen SYRUP 100MG/5ML', 'NAPROXEN tabs 250 mg', 'Carbamazepine TABS 400MG', 'CARBAMAZEPINE TABLETS 200 MG', 'Paracetamol  Oral suspension  250 mg/5 ml', 'PARACETAMOL+CODEINE PHOSPHATE 500mg/30mg effervescent tabs', 'CARBAMAZEPINE modified release tablet 200mg', 'Oramorph oral solution 10mg/5ml', 'phenytoin suspension sugar-free 90mg/5ml', 'ASPIRIN TABLETS  DISPERSIBLE 300MG-P42 0', 'VENLAFAXINE HYDROCHLORIDE TAB 37.5mg', 'DICLOFENAC SR TABS 75MG', 'ASPIRIN 300mg disp tablets', 'CO-CODAMOL 30mg/500mg effervescent tablets', 'Ibuprofen Tablets 600mg', 'Buprenorphine  Transdermal patches  70 micrograms/hour', 'AMITRIPTYLINE 25MGS', 'IBUPROFEN SYRUP 100MG/5ML', 'Diclofenac sodium 25mg gastro-resistant tablets (Sandoz Ltd)', 'BUPRENORPHINE TAB 200mcg', 'paracetamol with codeine phosphate and buclizine tablets 500mg+8mg+6.25mg', 'phenytoin 100mg capsules', 'Morphine 50mg tablets', 'Mirtazapine 15mg TABS 15mg', 'DICLOFENAC SODIUM eye drp 0.1%', 'NAPROXEN EC TAB 250MG', 'Buprenorphine 8mg / Naloxone 2mg sublingual tablets sugar free', 'Buprenorphine 2mg / Naloxone 500microgram sublingual tablets sugar free', 'IBUPROFEN liq capsules 200mg', 'MORPHINE SULPHATE mr cap 60mg', 'Gabapentin 100mg capsules (Actavis UK Ltd)', 'Tramadol modified release capsule 100mg', 'Phenytoin Sodium (Flynn Pharma)  Capsules  300 mg', 'COCODAMOL DISPERSIBLE 8/500MG TABLETS', 'MORPHINE SULPHATE mr grans for susp 20mg/sachet', 'Paracetamol 500 mg tablets', 'Mefenamic acid 250mg capsules (Essential Generics Ltd)', 'IBUPROFEN 600MG TABS', 'PARACETAMOL 500mg tabs', 'SOL CO CODAMOL', 'SODIUM VALPROATE + VALPROIC ACID mr tab 200mg', 'Diclofenac Sodium Unit Dose Eye DROPS 0.1%', 'lidocaine with adrenaline (base) 1:400,000 injection 1%', 'Mirtazapine 15mg orodispersible tablets (Actavis UK Ltd)', 'lidocaine hydrochloride injection 1%', 'CODEINE & PARACETAMOL TAB 8mg', 'Gabapentin Capsules 300mg', 'INDOMETHACIN TAB 50mg', 'DICLOFENAC NA 75mg m/r tabs', 'DICLOFENAC SODIUM TABLETS 50MG', 'mirtazapine orodispersible tablet 15mg', 'Nuseals Aspirin Gastro Resistant TABS 300MG', 'diamorphine hydrochloride elixir 10mg/5ml', 'DIHYDROCODEINE 10/PARACET 500', 'Co-codamol 30mg/500mg effervescent tablets (Zentiva)', 'Zinc oxide 6.6% / Lidocaine 0.7% cream', 'PHENYTOIN 100mg tabs', 'CO-CODAMOL 30MG/500MG EFFERVESCENT POWDER LEMON SACH', 'Co-codamol 8 mg + 500 mg tablets', 'diclofenac sodium modified release capsule 75mg', 'Pregabalin CAPS 200MG', 'Apomorphine 30mg/3ml solution for injection pre-filled di...', 'Duloxetine CAPS 60mg', 'ASPIRIN TAB 300', 'meprobamate with ethoheptazine citrate and aspirin tablets', 'CO-CODAMOL DISP TAB 8/500', 'morphine elixir 8.4mg/ml', 'Mirtazapine 45mg orodispersible tablets (Teva UK Ltd)', 'Ibuprofen 200mg tablets (Sandoz Ltd)', 'aspirin tablets 75 mg', 'Morphine Sulphate Mr TABS 15MG', 'PARACETAMOL caplets 500mg [WINTHROP]', 'Paracetamol 120mg soluble tablets sugar free', 'aspirin ec tablets 75mg', 'Morphine Sulphate TABS 20MG', 'aspirin modified release tablet 100mg', 'Codeine phosphate 15mg with paracetamol 500mg tablets', 'Mirtazapine 45mg orodispersible tablets (Actavis UK Ltd)', 'CODEINE PHOSPHATE 30MG', 'INDOMETHACIN 75mg m/r tablets', 'PARACETAMOLCAPLETS 500MG', 'Lidocaine 50mg/5ml (1%) solution for injection ampoules (Hameln Pharmaceuticals Ltd)', 'Paracetamol 250mg/5ml oral suspension (A A H Pharmaceuticals Ltd)', 'Cocodamol 500/30', 'INDOMETHACIN 25mg capsules', 'celecoxib capsules 100mg', 'Paracetamol with metoclopramide tablets 500mg+5mg', 'DIHYDROCODEINE', 'Mirtazapine 30mg orodispersible tablets (Actavis UK Ltd)', 'CARE IBUPROFEN tablets 200mg [T & R]', 'CODEINE PHOSPHATE tablets 30mg [CP PHARM]', 'Phenytoin capsules 25mg', 'Oxcarbazepine 600mg tablets (Actavis UK Ltd)', 'nuseal aspirin 300mg tab', 'Dihydrocodeine 30mg tablets (Wockhardt UK Ltd)', 'Paracetamol Oral SUSP 250MG/5ML', 'Paracetamol 500 mg soluble tablets', 'Paracetamol 500mg capsules (Focus Pharmaceuticals Ltd)', 'Diclofenac Sodium  Dispersible Tablets  50 mg', 'NAPROXEN CAP 250', 'Lidocaine Hydrochloride Amp 5ml INJ 2%', '*IBUPROFEN, TAB, 400.00mg', 'Amitriptyline Hydrochloride  Sugar-free oral solution  25 mg/5 ml', 'Mirtazapine 15mg tablets (Actavis UK Ltd)', 'Diclofenac Mr (Trinity)  75Mg Tablets', 'IBUPROFEN TABS 200MG', 'LIDOCAINE medicated plaster 5%', 'PARACETAMOL SOL TAB 500MG', 'INDOMETHACIN capsules 50mg', 'Oxcarbazepine 300mg tablets (Mylan Ltd)', 'VENLAFAXINE MR TABLETS 150MG', 'ASPIRIN TABS 75MG EC', 'SODIUM VALPROATE mr cap 300mg', 'Oxcarbazepine 150mg tablets (Teva UK Ltd)', 'Indomethacin slow-release capsules 75mg', 'Diclofenac Sr 100mg', 'NAPROXEN 500MG TABLETS', 'Morphine Sulphate Mr CAPS 10MG', 'CO-CODAMOL-P42 TABLETS DISP', 'ASPIRIN SOLUBLE TAB 300', 'VENLAFAXINE s/r cap 75mg-P42 0', 'Codeine Phosphate Linctus 15 mg/5 ml', 'INDOMETHACIN SR TAB 25mg', 'celecoxib 200mg cap', 'MEPROBAMATE + ETHOHEPTAZINE CITRATE & ASPIRIN tabs', 'Fentanyl 800microgram lozenges', 'Oxycodone Hydrochloride Mr TABS 10MG', 'OXYCODONE HCl sf oral soln 5mg/5ml', 'Venlafaxine 75mg tablets (Zentiva)', 'MEFENAMIC ACID capsules 250mg', 'Naproxen TABLETS 500MG', 'DULOXETINE CAPSULES 60MG', 'Morphine Sulfate  M/R suspension  20 mg/sachet  granules sugar free', 'CODEINE PHOSPHATE tabs 30 mg -', 'DICLOFENAC SOD CAP 100MG M/R', 'Lamotrigine Sf Dispersible TABS 25MG', 'DIHYDROCODEINE 10/PARACET 500MG', 'Etoricoxib tablets 90mg', 'CELECOXIB 100 MG', 'PARACETAMOL suspension 250mg/5ml [NUCARE]', 'Aspirin 150mg suppositories', 'Diclofenac and misoprostol tablets 50mg + 200micrograms', 'METHADONE MIXTURE', 'NAPROXEN ENTERIC COATED TABLETS 500MG', 'Lidocaine 100mg/5ml (2%) solution for injection ampoules (Hameln Pharmaceuticals Ltd)', 'DICLOFENAC DISPERSIBLE 50mg', 'Oxycodone Hydrochloride  Injection  10 mg/ml  1 ml ampoule', 'Paracetamol & metoclopramide 500 mg + 5 mg tablets', 'Pregabalin CAPS 25mg', 'Lidocaine 400mg/20ml (2%) solution for injection ampoules...', 'MEFENAMIC ACID TAB 500MG', 'methadone injection 10mg/ml', 'Venlafaxine Hydrochloride M/R Capsules 150 mg', 'ASPIRIN 75MG TAB', 'paracetamol oral liquid', 'CARBAMAZEPINE TAB 400mg', 'TRAMADOL', 'paracetamol with codeine phosphate tablets 450mg + 8.1mg', 'Ibuprofen M/R Capsules 300 mg', 'TRAMADOL soluble tab 50mg', 'DICLOFENAC SODIUM P AQUEOUS GEL 1%', 'NAPROXEN 250mg e/c tablets', 'DICLOFENAC MODIFIED RELEASE CAPSULE 100 MG', 'Codeine 60mg tablets (Wockhardt UK Ltd)', 'Paracetamol 120mg/5ml oral suspension paediatric (Rosemon...', 'Phenytoin sodium 100mg tablets (Almus Pharmaceuticals Ltd)', 'CARBAMAZEPINE TABLETS  400 MG', 'DIHYDROCODEINE TABLETS  30 MG', 'PARACETAM/DYHYDROCODEINE(500MG/30MG) MG TAB', 'Aspirin Tablets 75mg', 'DICLOFENAC enteric coated tablets 50mg [GENUS]', 'Diclofenac Modified Relea', 'Methylprednisolone And Lidocaine Injection 40 mg/ml + 10 mg/', 'METOCLOPRAMIDE HYD. 5mg/450mg ASPIRIN TAB', 'Amitriptyline 50mg/5ml oral solution sugar free (Rosemont Pharmaceuticals Ltd)', 'paracetamol with codeine phosphate tablets 500mg + 30mg', 'TRAMADOL TABLETS 100MG M/R-P42 0', 'Buprenorphine 8mg sublingual tablets sugar free (Actavis UK Ltd)', 'Sodium valproate 200mg/5ml oral solution sugar free (IVAX Pharmaceuticals UK Ltd)', 'Etoricoxib tablets 60mg', 'CODEINE PHOSPHATE LINCTUS 15 MG/5ML', 'DICLOFENAC SODIUM 1% gel', 'DICLOFENAC supp 25mg', 'diclofenac sodium and misoprostol tablets 50mg + 200microgra', 'Dihydrocodeine tartrate 30 mg tablets', 'Pregabalin 200mg capsules (Teva UK Ltd)', 'PARACETAMOL capsules [CO-OPERATI]', 'CODEINE PHOS TAB 30MG', 'Co-codamol tablets 8/500', 'IBUPROFEN 600MG TABLETS', 'Lidocaine Injection 2 %, 40 mg/2 ml ampoule', 'IBUPROFEN tabs 600 MG', 'AMITRIPTYLINE caps s/r 25 mg', 'Ibuprofen 300mg / Pseudoephedrine 45mg modified-release capsules', 'Pregabalin Capsules 200 mg', 'Gabapentin 400mg capsules (Sandoz Ltd)', 'Aspirin 900mg / Metoclopramide 10mg oral powder sachets suga', 'Diclofenac sodium 75mg gastro-resistant modified-release cap', 'Dihydrocodeine Tab 30mg', 'Ibuprofen 200mg tablets (A A H Pharmaceuticals Ltd)', 'Methadone 10mg/1ml solution for injection ampoules (Wockhardt UK Ltd)', 'Ibuprofen and codeine tablets 200mg + 12.8mg', 'Nortriptyline TABLETS 25MG', 'Paracetamol 250mg suppositories', 'Paracetamol 125mg suppositories', 'TRAMADOL HCL SR 100MG TABS', 'LIDOCAINE HCl + ADRENALINE 1:80,000 inj 2% w/v', 'CARBAMAZEPINE CONTROLLED RELEASE TABLET 200MG', 'Morphine Sulfate  M/R tablets  15 mg', 'IBUPROFEN MR TAB 800MG', 'Fentanyl  Transdermal patches  37.5 micrograms/hour', 'DICLOFENAC DISPERSIBLE TAB 50', 'DICLOFENAC SODIUM 100MG TAB RETARD', 'diclofenac potassium tablets 50mg', 'Co-codamol 30mg/500mg caplets (Phoenix Healthcare Distrib...', 'Diclofenac Sodium Dispersible Tablets 50 mg', 'Diamorphine 10mg tablets', 'PHENYTOIN 100mg tablets', 'ibuprofen soluble tablet 200mg', 'CO-CODAMOL (codeine & paracetamol) tabs 12.8mg + 500mg', 'Dihydrocodeine 30mg tablets (Zentiva)', 'Lidocaine 50mg/10ml (0.5%) solution for injection ampoules (AMCo)', 'Paracetamol with dihydrocodeine tablets 500mg + 20mg', 'Dihydrocodeine with paracetamol tablets 30mg + 500mg', 'Aspirin 300mg tablets (Aspar Pharmaceuticals Ltd)', 'NORTRIPTYLINE HCL/FL TAB', 'diclofenac suppository 100mg', 'Lidocaine Injection 1%, 20mg/2ml ampoule', 'Paracetamol Orodispersible TABS 250MG', 'KAOLIN & MORPHINE mixture', 'Lidocaine  Injection  2 %, 40 mg/2 ml ampoule', 'PARACETAMOL 500MG/METHIONINE 100MG TAB', 'Ibuprofen Gel 10%', 'Fentanyl 50micrograms/dose nasal spray', 'INDOMETHACIN TAB 75MG M/R', 'Diclofenac suppository 50mg', 'Ibuprofen Cream 5 %', 'Zinc oxide 283.5mg / Lidocaine 13.2mg suppositories', 'Dihydrocodeine 50mg/1ml solution for injection ampoules', 'Lidocaine 50mg/10ml (0.5%) solution for injection ampoules (Martindale Pharmaceuticals Ltd)', 'Gabapentin Titration Pack', 'diclofenac sodium dual release capsules 75mg', 'DICLOFENAC SOD TAB 100MG M/R', 'IBUPROFEN_TAB 400MG', 'CODEINE PHOSPHATE linc diabetic 15mg/5ml', 'Ibuprofen Sf With Syringe SUSP 100MG/5ML', 'DIHYDROCODEINE TARTRATE MR TABLETS 120MG', 'Lidocaine 100mg/5ml (2%) solution for injection Sure-Amp ampoules (AMCo)', 'Diclofenac And Misoprostol Tablets 50mg + 200 micrograms', 'DICLOFENAC MR TAB 100', 'Paracetamol 500mg caplets (Phoenix Healthcare Distributio...', 'PHOLCODEINE LINCTUS', 'naproxen suppository 500mg', 'Co-codamol 8mg/500mg tablets', 'Co-Codamol 30/500  Tablets', 'Ibuprofen 200mg tablets', 'Co-codamol 30mg/500mg tablets', 'ASPIRIN TABLETS 75MG', 'CODEINE PHOSPHATE tabs 30mg', 'Aspirin 75mg tablets', 'Codeine 30mg tablets', 'Tramadol 100mg modified-release capsules', 'DICLOFENAC SODIUM ec tab 50mg', 'Nortriptyline 10mg tablets', 'TRAMADOL caps 50mg', 'Co-Codamol 30/500  Capsules', 'AMITRIPTYLINE HYDROCHLORIDE TABLETS 10MG', 'Ibuprofen 800mg modified-release tablets', 'Mirtazapine 15mg tablets', 'Co-codamol 30mg/500mg effervescent tablets', 'Venlafaxine 75mg modified-release tablets', 'Paracetamol Soluble TABS 500MG', 'Ibuprofen TABS 600MG', 'Celecoxib 100mg capsules', 'Mirtazapine 30mg tablets', 'Co-codamol 15mg/500mg tablets', 'Ibuprofen TABS 400MG', 'Amitriptyline 10mg/5ml oral solution sugar free', 'Naproxen TABS 500MG', 'ibuprofen capsules 200mg', 'OXYCODONE HCl caps 10mg', 'Venlafaxine Mr CAPS 150MG', 'DIHYDROCODEINE tabs 30mg', 'Carbamazepine 200mg tablets', 'Mirtazapine  Tablets  45 mg', 'CO-CODAMOL caps 30mg + 500mg', 'Etoricoxib 90mg tablets', 'Diclofenac Sodium Ec TABS 50MG', 'Etoricoxib 30mg tablets', 'DIHYDROCODEINE tabs 40mg', 'Diclofenac 50mg dispersible tablets sugar free', 'DICLOFENAC SODIUM 75mg m/r capsules', 'Lidocaine 5% medicated plasters', 'TRAMADOL + PARACETAMOL tabs 37.5mg + 325mg', 'CO-CODAMOL 30MG/500MG EFFERVESCENT TABLETS', 'Tramadol 400mg modified-release tablets', 'Phenytoin sodium 100mg capsules (Flynn Pharma Ltd)', 'Morphine 5mg modified-release tablets', 'IBUPROFEN TABLETS 400MG', 'Gabapentin  Capsules  300 mg', 'Diclofenac potassium 50mg tablets', 'Naproxen  E/c tablets  250 mg', 'Oxycodone 20mg/2ml solution for injection ampoules', 'Paracetamol 500mg caplets (Zentiva)', 'Diclofenac 1% transdermal patches', 'Tramadol 50mg orodispersible tablets sugar free', 'Co-Codamol 8/500  Dispersible tablets', 'Co-codamol dispersible tablet 8mg+500mg', 'Co-codamol 30mg/500mg caplets (Actavis UK Ltd)', 'LAMOTRIGINE tabs 50mg', 'MIRTAZAPINE tabs 15mg', 'GABAPENTIN TABLETS 600MG', 'PREGABALIN CAPSULES 100MG', 'IBUPROFEN GEL 5%', 'AMITRIPTYLINE HYDROCHLORIDE TABLETS 25MG', 'Depo-Medrone with Lidocaine suspension for injection 1ml ...', 'CO-CODAMOL caps 8mg+500mg', 'Amitriptyline Hydrochloride TABS 10MG', 'CO-CODAMOL tabs 30mg + 500mg [CP]', 'IBUPROFEN TABLETS 600MG', 'Ibuprofen TABS 200MG', 'Buprenorphine 52.5micrograms/hour transdermal patches', 'Aspirin dispersible tablet 75mg', 'Paracetamol Tablets 500 mg', 'CARBAMAZEPINE RETARD TAB 200', 'Diclofenac 2.32% gel', 'Naproxen enteric coated tablets 500mg', 'DEPO-MEDRONE + LIDOCAINE inj 40mg/ml + 10mg/ml', 'Lidocaine 100mg/10ml (1%) solution for injection ampoules (Hameln Pharmaceuticals Ltd)', 'Codeine Phosphate  Tablets  15 mg', 'Morphine 20mg modified-release granules sachets sugar free', 'Diclofenac Sodium  M/R capsules  75 mg', 'IBUPROFEN tablets 400mg', 'Aspirin 900mg / Metoclopramide 10mg oral powder sachets sugar free', 'AMITRIPTYLINE TAB 75mg', 'Co-Codamol 30mg/500mg Effervescent TABS', 'Aspirin Ec TABS 75MG', 'DICLOFENAC SODIUM GEL 3%', 'Oxycodone 10mg / Naloxone 5mg modified-release tablets', 'Nortriptyline 25mg tablets', 'PARACETAMOL TAB 500', 'LIDOCAINE HCl inj 1%', 'DULOXETINE gastro-res cap 40mg', 'PREGABALIN caps 25mg', 'Venlafaxine TABS 75MG', 'Co-codamol tablets 30mg + 500mg', 'OXYCODONE HCl mr tab 20mg', 'Co-codamol 30mg/500mg tablets (Wockhardt UK Ltd)', 'DICLOFENAC SODIUM enteric coated tablets 50mg', 'VENLAFAXINE tabs 37.5mg', 'Tramadol 50mg capsules (Actavis UK Ltd)', 'Naproxen 500mg / Esomeprazole 20mg modified-release tablets', 'Lidocaine 200mg/20ml (1%) / Adrenaline 100micrograms/20ml...', 'Fentanyl 50micrograms/hour transdermal patches (A A H Pharmaceuticals Ltd)', 'DICLOFENAC mr cap 75mg', 'Paracetamol 500mg tablets (Galpharm International Ltd)', 'Sodium valproate 300mg modified-release capsules', 'Co-Codamol 15/500  Tablets', 'Lidocaine 2% gel', 'DIHYDROCODEINE TARTRATE TAB 30mg', 'IBUPROFEN TABS 400MG', 'Naproxen Ec TABS 250MG', 'Tramadol Hydrochloride  M/R capsules  100 mg', 'Morphine Sulfate  M/R tablets  10 mg', 'CARBAMAZEPINE tabs 100mg', 'DICLOFENAC ec tab 25mg', 'Phenytoin sodium 100mg tablets', 'Diclofenac sodium 50mg tablets', 'DICLOFENAC SOD TAB 100MG', 'Soluble Aspirin Paediatric  Tablets  75 mg', 'DICLOFENAC mr cap 100mg', 'GABAPENTIN CAPSULES 100MG', 'AMITRIPTYLINE HYDROCHLORIDE TABLETS 50MG', 'Lidocaine 100mg/10ml (1%) solution for injection Mini-Plasco ampoules (B.Braun Medical Ltd)', 'Diclofenac Sodium Mr CAPS 75MG', 'Codeine Phosphate TABS 15MG', 'Tramadol Hydrochloride Orodispersible TABS 50MG', 'Paracetamol 500mg caplets (Teva UK Ltd)', 'ASPIRIN tabs 300mg', 'PARACETAMOL + METOCLOPRAMIDE tabs 500mg+5mg', 'Co-codamol 8mg/500mg effervescent tablets (A A H Pharmaceuticals Ltd)', 'IBUPROFEN tabs 800mg', 'Morphine 10mg tablets', 'Tramadol 100mg effervescent powder sachets sugar free', 'NAPROXEN TABLETS 500MG', 'Morphine sulfate 20mg/ml oral solution sugar free', 'Aspirin Dispersible Tablets 300 mg', 'Morphine sulfate 10mg/5ml oral solution (Martindale Pharmaceuticals Ltd)', 'Phenytoin sodium 300mg capsules (Flynn Pharma Ltd)', 'Diclofenac Sodium P Aqueous GEL 1%', 'Oxycodone 10mg/1ml solution for injection ampoules', 'naproxen tablets 500mg', 'CO-CODAMOL 8mg/500mg tablets', 'MORPHINE SULPHATE oral soln 10mg/5ml', 'Paracetamol 500mg/5ml oral suspension sugar free', 'Mirtazapine Orodispersible TABS 15MG', 'Morphine 20mg tablets', 'Aspirin 75mg gastro-resistant tablets (Sandoz Ltd)', 'VENLAFAXINE tabs 75mg', 'ASPIRIN DISPERSIBLE TABLETS 300MG', 'Mirtazapine Orodispersible TABS 30MG', 'NAPROXEN TAB 500mg', 'Venlafaxine 37.5 mg tablets', 'DICLOFENAC EC TABLETS 50 MG', 'LIDOCAINE injection 2% w/v [MARTINDALE]', 'Phenytoin 50mg capsule', 'Gabapentin  Capsules  400 mg', 'NAPROXEN TAB 250', 'NORTRIPTYLINE tabs 25mg', 'Celecoxib CAPS 200MG', 'Ibuprofen tablets 400mg', 'ibuprofen granules 400mg', 'METHYLPREDNISOLONE ACETATE + LIDOCAINE inj 40mg/1ml+10mg/1ml', 'Methylprednisolone 40mg/1ml / Lidocaine 10mg/1ml (1%) sus...', 'DICLOFENAC SODIUM mr cap 100mg', 'IBUPROFEN mr tab 800mg', 'IBUPROFEN JEL 5%', 'Paracetamol 500mg/5ml oral solution sugar free', 'DIHYDROCODEINE 10mg/PARACETAMOL 500mg TAB', 'PREGABALIN caps 150mg', 'CO-CODAMOL 30mg/500mg tabs', 'Ibuprofen lysine 200mg tablets', 'LIDOCAINE crm 2% w/v', 'Naproxen 500mg tablets (Actavis UK Ltd)', 'PARACETAMOL SUPPS 500MG', 'ibuprofen tablets 400mg', 'ASPIRIN DISPERSIBLE TAB 75', 'lubricating chlorhexidine with lidocaine gel', 'Co-Codamol 30mg/500mg Caplet TABS', 'INDOMETHACIN CAP 25mg', 'PARACETAMOL CAPLET TABLETS 500MG', 'AMITRIPTYLINE 25MG TAB', 'Buprenorphine  Transdermal patches  35 micrograms/hour (20 mg per patch)', 'ASPIRIN TAB 75', 'Perphenazine 2mg with Amitriptyline 10mg tablet', 'Carbamazepine TABS 200MG', 'Hydrocortisone And Lidocaine  Ointment  0.275 % + 5 %', 'Tramadol 50mg capsules (Almus Pharmaceuticals Ltd)', 'Lidocaine 10% spray sugar free', 'BUPRENORPHINE patch 20micrograms/hour', 'Pregabalin 300mg capsules (Teva UK Ltd)', 'ASPIRIN tablets 75mg', 'CARBAMAZEPINE TABS 400MG', 'Ibuprofen 600mg tablets (A A H Pharmaceuticals Ltd)', 'Co-Codamol 15mg/500mg Caplet TABS', 'CODEINE PHOSPHATE TAB 30', 'Diclofenac Sodium Mr TABS 75MG', 'DICLOFENAC 75MG/MISOPROSTOL 200MCG TAB', 'CO-CODAMOL 30/500 tabs effervescent 530mg-P42 0', 'PHENYTOIN TABS 100MG', 'VENLAFAXINE mr tab 225mg', 'Naproxen 500mg tablets (A A H Pharmaceuticals Ltd)', 'CODEINE LINCTUS LIQ', 'Tramadol Hydrochloride Mr CAPS 200MG', 'NU-SEALS ASPIRIN EC tablets 75mg [LILLY]', 'Oxycodone Hydrochloride  Capsules  5 mg', 'Diclofenac Sodium  M/R Capsules  75 mg', 'METHYLPREDNISOLONE ACETATE + LIDOCAINE inj 40mg/ml + 10mg/ml', 'MIRTAZAPINE TABLETS 15MG', 'CARBAMAZEPINE tabs 400mg', 'venlafaxine  xl TABS 75mg', 'Diclofenac enteric coated tablets 50mg', 'DICLOFENAC disp tabs 50mg', 'AMITRIPTYLINE tabs 10mg [TEVA]', 'Co-codamol 8mg/500mg tablets (Vantage)', 'Diclofenac Sodium Ec TABLETS 50MG', 'Paracetamol 250mg orodispersible tablets sugar free', 'IBUPROFEN 600mg tabs', 'Naproxen Sodium Tablets 275 mg', 'CARBAMAZEPINE TABLETS 200MG', 'Aspirin 300mg dispersible tablets (Almus Pharmaceuticals Ltd)', 'COCODAMOL TABLETS', 'IBUPROFEN caps 200mg', 'BUPRENORPHINE patch 35mcg/hour', 'CO-CODAMOL EFF_TAB 8MG/500MG', 'Oxcarbazepine TABS 300MG', 'FENTANYL patch 100micrograms/hour', 'Naproxen 500mg modified-release tablets', 'DICLOFENAC EC TAB 25MG', 'DICLOFENAC E/C TAB 50', 'AMITRIPTYLINE HYDROCHLORIDE tablets 25mg', 'Paracetamol 500mg / Domperidone 10mg tablets', 'FENTANYL patch 75micrograms/hour', 'Etoricoxib TABS 60MG', 'Tramadol 37.5mg / Paracetamol 325mg effervescent tablets sugar free', 'DICLOFENAC AND MISOPROSTOL tablets 50mg + 200micrograms', 'protriptyline tablets 10mg', 'DIHYDROCODEINE TARTRATE TAB 30', 'CO-CODAMOL tabs 30mg + 500mg [CP PHARM]', 'codeine phosphate (form not specified)', 'DICLOFENAC TABS 50MG', 'Mexiletine 200mg capsules', 'paracetamol with codeine phosphate capsules 500mg + 30mg', 'morphine sulfate injection 10mg/1ml', 'INDOMETHACIN caps 50 mg', 'Co-codamol 30mg/500mg caplets (Mercury Pharma Group Ltd)', 'Tramadol 200mg modified-release tablets (A A H Pharmaceuticals Ltd)', 'ETORICOXIB TABLETS 90MG', 'desipramine tablets 25mg', 'ASPIRIN TABLETS 300 MG', 'ASPIRIN TABS, E/C 75 MG', 'Aspirin 300mg gastro-resistant tablets (A A H Pharmaceuticals Ltd)', 'Venlafaxine Hydrochloride  Tablets  75 mg', 'IBUPROFEN TABS 800MG SR TAB 800mg', 'Ibuprofen TABLETS 400MG', 'DICLOFENAC tablets 50mg [BERK]', 'DICLOFENAC SR MG 75', 'DIHYDROCODEINE mr tab 90mg', 'Ibuprofen  M/R tablets  800 mg', 'Methadone 1mg/ml oral solution', 'Diclofenac Sodium GEL 3%', 'DICLOFENAC MR 75MG', 'Paracetamol 500mg tablets (A A H Pharmaceuticals Ltd)', 'Morphine Sulphate  Tablets  10 mg', 'Co-Codamol 30/500  Effervescent Tablets', 'PARACETAMOL TABLETS 500 MG', 'PERPHENAZINE + AMITRIPTYLINE tabs 2mg + 25mg', 'SODIUM VALPROATE mr tab 300mg', 'Venlafaxine modified release capsule 75mg', 'Duloxetine Gastro Resistant CAPS 30MG', 'Diclofenac sodium 75mg gastro-resistant / Misoprostol 200...', 'DICLOFENAC EC tabs 25mg', 'Tramadol Hydrochloride Capsules 50 mg', 'CODEINE PHOSPHATE tablets 30mg', 'Ibuprofen Gel 5%', 'DICLOFENAC TABS S/R 100MG', 'Pregabalin CAPS 50MG', 'AMITRIPTYLINE HYDROCHLORIDE TABS 10MG', 'DICLOFENAC MR TAB 75', 'CO-PROXAMO (dextropropoxyphen & paracetamol tab 32.5mg+325mg', 'Diclofenac Sodium M/R Capsules 75 mg', 'IBUPROFEN', 'DICLOFENAC EC tabs 50mg', 'Oxycodone Hydrochloride  M/R tablets  30 mg', 'AMITRIPTYLINE HYDROCHLORIDE TABS 50MG', 'CARBAMAZEPINE tabs 100 MG', 'Tramadol Hydrochloride Mr CAPS 150MG', 'Naproxen 125mg/5ml oral suspension sugar free', 'Aspirin Ec TABS 300MG', 'Lidocaine And Aminoacridine  Oral Gel Sugar Free  0.66 % + 0.05 %', 'INDOMETHACIN SUP 100mg', 'MORPHINE SULPHATE 12hr mr caps 30mg', 'morphine sulfate 12 hour modified release capsules 20mg', 'PARACETAMOL+CODEINE PHOSPHATE 500mg/30mg tablets', 'Phenytoin 30mg/5ml oral suspension', 'MIRTAZAPINE ORODISPERSIBLE TABLETS 30MG', 'DICLOFENAC 100mg m/r capsules', 'CELECOXIB capsules 200mg', 'Co-codamol 8mg/500mg effervescent tablets (Zentiva)', 'DICLOFENAC 50MG/MISOPROSTOL 200MCG TAB', 'Aspirin 400mg with Codeine 8mg tablets', 'Paracetamol 500mg with codeine phosphate 8mg & buclizine 6.25mg', 'IBUPROFEN liq capsules 400mg', 'Venlafaxine tablets 37.5mg', 'Methadone 1 Mg/Ml  Mixture', 'COCODAMOL 8/500G TABLETS', 'DICLOFENAC SODIUM SUPPS 100MG', 'NAPROXEN enteric coated tablets 250mg [GALEN]', 'Depo-Medrone With Lidocaine Injection, 2 Ml Vial', 'Etoricoxib  Tablets  30 mg', 'DIHYDROCODEINE TARTRATE MR TABLETS 60MG', 'Phenytoin sodium 100mg capsules (A A H Pharmaceuticals Ltd)', 'Oxycodone 50mg/1ml solution for injection ampoules', 'VENLAFAXINE HYDROCHLORIDE TAB 37.5', 'MEFENAMIC ACID CAP 250', 'Co-codamol 30mg/500mg effervescent tablets (Almus Pharmaceuticals Ltd)', 'CARBAMAZEPINE 100mg tablets', 'LACOSAMIDE tabs 50mg', 'Morphine sulfate 10mg/1ml solution for injection ampoules (Wockhardt UK Ltd)', 'Depo-medrone with lidocaine 40mg/ml+10mg/ml Injection (Pharmacia Ltd)', 'Paracetamol 500mg soluble tablets (Actavis UK Ltd)', 'Carbamazepine  Tablets  200 mg', 'LAMOTRIGINE TAB 25mg', 'Sodium valproate 200mg/5ml oral solution sugar free', 'PARACETAMOL WITH CODEINE PHOSPHATE tablets 500mg + 30mg', 'DEPO-MEDRONE WITH LIDOCAINE injection 40mg/ml +', 'DICLOFENAC SODIUM SR TABLETS 75MG TAB 28', 'VENLAFAXINE XL 75MG CAPSULES', 'PARACETAMOL supp 500mg', 'Amitriptyline Hydrochloride Tablets 25 mg', 'Gabapentin Capsules 100 mg', 'IBUPROFEN S/R CAP 300', 'LIDOCAINE HYDROCHLORIDE AMP 5ML INJ 1%', 'Sodium Valproate  E/c tablets  200 mg', 'Lidocaine hcl 2% Injection', 'DICLOFENAC ENTERIC COATED TABLETS 50MG', 'Amitriptyline 25mg/5ml oral solution sugar free (A A H Pharmaceuticals Ltd)', 'CO-CODAMOL TABS 8MG+500MG', 'DICLOFENAC-MR', 'CODEINE PHOSPHATE TABS 30MG', 'dihydrocodeine with paracetamol tablets 10mg + 500mg', 'DIHYDROCODEINE TARTR TAB 30', 'Paracetamol 500mg caplets (Kent Pharmaceuticals Ltd)', 'Diclofenac Sodium  Suppositories  50 mg', 'PARACETAMOL sf paed oral soln 120mg/5ml', 'IBUPROFEN mr cap 300mg', 'Aspirin 300mg dispersible tablets (Actavis UK Ltd)', 'IBUPROFEN TABS 400 MG', 'Hydromorphone 4mg modified-release capsules', 'Co-Codamol 8/500 Dispersible Tablets', 'APOMORPHINE HYDROCHLORIDE SLS SUBLINGUAL TABLETS 2MG', 'Carbamazepine  M/R tablets  200 mg', 'Hydromorphone 2mg modified-release capsules', 'CARBAMAZEPINE MR TABLETS 200MG', 'ETORICOXIB 60MG TAB', 'Codeine Phosphate Sf Diabetic LINCT 15MG/5ML', 'Etoricoxib 60 mg tablets', 'OXYCODONE HYDROCHLORIDE CAPSULES 10MG', 'Paracetamol 500mg soluble tablets (Almus Pharmaceuticals Ltd)', 'Mefenamic acid 500mg tablets (Teva UK Ltd)', 'PARACETAMOL 500mg soluble tabs', 'Diclofenac and misoprostol tablets 75mg + 200mcg', 'diclofenac sodium and misoprostol tablets 50mg+200mcg', 'IBUPROFEN 400mg', 'TRAMADOL modified release 200mg', 'AMITRIPTYLINE HYDROCHLORIDE S/R CAP 25mg', 'lidocaine hydrochloride pump spray 10%', 'DICLOFENAC SODIUM ec tabs 25mg', 'TRAMADOL modified release capsule 200mg', 'Tramadol 50mg effervescent powder sachets sugar free', 'ASPIRIN TAB 300mg', 'Aspirin 500mg / Codeine 8mg dispersible tablets sugar free', 'DIHYDROCODEINE TABS 30MG', 'Co-codamol 8mg/500mg tablets (Teva UK Ltd)', 'DICLOFENAC E/C TAB 25mg', 'Codeine phosphate 8mg with Paracetamol 500mg capsules', 'tramadol capsules 50mg', 'ORAMORPH ORAL SOLN 10MG/5ML', 'Phenytoin Tablets 50 mg', 'CELECOXIB TABLETS 200 MG', 'PARACETAMOL WITH CODEINE PHOSPHATE 500m', 'DICLOFENAC 50/MISOPROSTOL 200 TAB.', 'Ibuprofen tablets 200mg', 'Codeine 15mg tablets (Sandoz Ltd)', 'DIHYDROCODEINE TART TAB 30MG', 'DICLOFENAC tabs s/r 100mg', 'Diclofenac Sodium Topical SOL 16MG/ML', 'Diclofenac sodium 75mg gastro-resistant / Misoprostol 200mic', 'Lidocaine 400mg/20ml (2%) solution for injection ampoules (AMCo)', 'Paracetamol 500mg tablets (Kent Pharmaceuticals Ltd)', 'Codeine 15mg tablets (A A H Pharmaceuticals Ltd)', 'NAPROXEN tablets 500mg', 'VENLAFAXINE modified release capsule 150mg', 'Dihydrocodeine with paracetamol 30mg+500mg tablets', 'Naproxen tablets 250mg', 'Morphine Sulfate  M/R tablets  5 mg', 'LAMOTRIGINE TABLETS 50MG', 'NUSEALS ASPIRIN EC TABLETS 300MG', 'Diclofenac sodium 75 mg modified release capsules', 'DICLOFENAC SOD CAP 75', 'Tramadol Hydrochloride Effervescent Powder Lemon SACH 50MG', 'Mirtazapine 30 mg tablets', 'ASPIRIN NUSEAL 75MG', 'BUPRENORPHINE 5 MICROGRAMS / HOUR PATCH', 'DICLOFENAC SODIUM 100 mg modified release capsules', 'NAPROXEN enteric coated tablets 500mg', 'naproxen and misoprostol (combined) tablets 500mg+200mcg', 'ASPIRIN NUSEALS TAB 300', 'ASPIRIN E/C TAB 300', 'CO-CODAMOL tabs 12.8mg + 500mg', 'SODIUM VALPROATE E/C TAB 200mg', 'Venlafaxine Hydrochloride  M/R capsules  150 mg', 'DICLOFENAC 75MG DR CAPS DUAL-RELEASE CAP', 'CARBAMAZEPINE tabs 100 mg', 'Duloxetine  Gastro-Resistant Capsules  60 mg', 'DEXIBUPROFEN tabs 400mg', 'VENLAFAXINE 75mg m/r capsules', 'TRAMADOL MR TAB 100MG', 'Codeine phosphate-P42 30mg tablet', 'Ibuprofen sodium dihydrate 200mg tablets', 'PARACETAMOL SUPPS 125MG', 'amitriptyline hydrochloride tablets 10mg', 'Diamorphine 100mg powder for solution for injection ampoules', 'Hydromorphone 2.6mg capsules', 'TAPENTADOL FC tab 50mg', 'IBUPROFEN 400mg tabs', 'Paracetamol-P42 500mg tab soluble', 'AMITRIPTYLINE 25mg tabs', 'PARACETAMOL + DOMPERIDONE tabs 500mg + 10mg', 'Diclofenac sodium 25mg gastro-resistant tablets (A A H Pharmaceuticals Ltd)', 'Phenytoin Sodium  Capsules  300 mg', 'Aspirin 75mg dispersible tablets (Kent Pharmaceuticals Ltd)', 'PARACETAMOL SOLUBLE TAB', 'ISOMETHEPTENE MUCATE + PARACETAMOL caps 65mg+325mg', 'DICLOFENAC TABS S/R 75 MG', 'Gabapentin 300mg capsules (A A H Pharmaceuticals Ltd)', 'naproxen tablets 250mg', 'DICLOFENAC tabs dispersible 50 mg', 'Tramadol 50mg capsules (Tillomed Laboratories Ltd)', 'PHENYTOIN SODIUM CAPSULES 25MG', 'Buprenorphine 400microgram sublingual tablets sugar free (A A H Pharmaceuticals Ltd)', 'Amitriptyline 10mg tablets (Almus Pharmaceuticals Ltd)', 'Lamotrigine  Dispersible tablets  Sugar Free 100 mg', 'Phenytoin Sodium  Capsules  25 mg', 'AMITRIPTYLINE HYDROCHLORIDE modified release ca', 'INDOMETHACIN CAPSULES 50 MG', 'Paracetamol 500mg caplets (Alliance Healthcare (Distribution) Ltd)', 'AMITRIPTYLINE 50mg tablets', 'Tramadol 50mg capsules (Zentiva)', 'DICLOFENAC GEL', 'APOMORPHINE sublingual tab 3mg', 'Tramadol Hydrochloride  M/R tablets  50 mg', 'Carbamazepine 100mg tablets (Almus Pharmaceuticals Ltd)', 'Lidocaine 100mg/5ml (2%) solution for injection pre-filled syringes (Martindale Pharmaceuticals Ltd)', 'Fentanyl 600microgram lozenges', 'Nu-Seals Aspirin  E/C Tablets  300 mg', 'DICLOFENAC injection 25 mg/ml (3 ml amp)', 'DICLOFENAC TABLETS 50MG-P42 0', 'Aspirin  E/C Tablets  300 mg', 'Aspirin Disp 75mg Tabs', 'DICLOFENAC E/C TAB 25', 'Ibuprofen 200mg tablets (Ranbaxy (UK) Ltd)', 'GABAPENTIN tabs 800mg', 'Paracetamol 500mg with codeine phosphate 8mg capsule', 'OXYCODONE HYDROCHLORIDE CAPSULES 5MG', 'Codeine Linctus Diabetic  Linctus  15 mg/5 ml', 'Sodium Valproate Mr TABS 500MG', 'TRAMADOL SR modified release tablet 150mg [ZENTIVA]', 'Lidocaine 400mg/20ml (2%) / Adrenaline (base) 100microgra...', 'VENLAFAXINE TABLETS 50MG', 'GABAPENTIN CAP 400mg', 'IBUPROFEN CAP 200mg', 'CO-CODAMOL CAPLET 30/500MG CAPLET', 'PARACETAMOL + ISOMETHEPTENE MUCATE caps 325mg + 65mg', 'CODEINE PHOSPHATE diabetic linc 15mg/5ml', 'nortriptyline capsules 10mg', 'sodium valproate ec tablets 200mg', 'Diclofenac  Medicated Plaster  140 mg', 'Pregabalin 50mg capsules (Teva UK Ltd)', 'CODEINE PHOSPHATE-P42 30MG TABLET', 'TRAMADOL sach 50mg', 'Morphine Sulfate  M/R tablets  30 mg', 'Tapentadol 150mg modified-release tablets', 'Fentanyl  Transdermal patches  50 micrograms/hour', 'PARACETAM/DYHYDROCODEINE(500MG/30MG) TAB', 'DICLOFENAC disp tab 50mg', 'Oxycodone Hydrochloride  M/R tablets  80 mg', 'DICLOFENAC mr tab 100 MG', 'MEFENAMIC ACID DISPERSIBLE TAB 250mg', 'Allantoin 0.5% / Lidocaine 0.5% ointment', 'Amitriptyline 50mg tablets (Teva UK Ltd)', 'lidocaine with cetylpyridinium chloride and chlorocresol oral liquid', 'Gabapentin CAPSULES 100MG', 'Phenytoin Sodium CAPS 50MG', 'Codeine 15mg tablets (Wockhardt UK Ltd)', 'Radian B Ibuprofen 5% gel (Thornton & Ross Ltd)', 'Diclofenac tablets 50mg', 'Diclofenac Modified Release Tablet 75mg', 'Diclofenac SR 75mg', 'ASPIRIN EC TAB 75MG', 'Diclofenac sodium 50mg gastro-resistant tablets (Kent Pharmaceuticals Ltd)', 'DICLOFENAC dual release capsule 75mg', 'aspirin ec tablets 300mg', 'Pregabalin CAPS 25MG', 'Mefenamic Acid 500mg', 'ASPIRIN Disp', 'Phenytoin Sodium TABS 50MG', 'Aspirin 300mg effervescent tablets sugar free', 'Diclofenac Sodium E/C Tablets 50mg', 'DEPO-MEDRONE+LIDOCAINE  40mg/1ml inj.', 'NAPROXEN TABS 250 MG', 'Co-Codamol 15/500  Effervescent Tablets Sugar Free', 'Apomorphine 50mg/5ml solution for injection ampoules (Archimedes Pharma UK Ltd)', 'Lidocaine 50mg/5ml (1%) solution for injection ampoules (AMCo)', 'ASPIRIN dispersible tablet 75mg[tab blister pack]', 'PHENYTOIN SODIUM-P42 100mg caps', 'CO-CODAMOL EFF 30MG/ TAB', 'Sodium Valproate Ec TABS 500MG', 'Naproxen 250mg tablets (Actavis UK Ltd)', 'DIHYDROCODEINE TARTRATE C/R TAB 60mg', 'CODEINE PHOSPHATE LINC 15MG/5ML', 'Co-Codamol 15mg/500mg TABS', 'DICLOFENAC SOD 50MG/MISOPROSTOL 200MCG TAB', 'Ibuprofen  Granules  600 mg/sachet', 'Tramadol 50mg capsules (Ranbaxy (UK) Ltd)', 'Buprenorphine 8mg sublingual tablets sugar free', 'Venlafaxine Hydrochloride M/R Capsules 75 mg', 'Paracetamol 500mg capsules (Aspar Pharmaceuticals Ltd)', 'SODIUM VALPROATE 200mg e/c tabs', 'GABAPENTIN CAPS 300MG', 'Carbamazepine tablets 100mg', 'IBUPROFEN spray 5%', 'metoclopramide with aspirin effervescent tablet 5mg + 450mg', 'Diclofenac Sodium  Suppositories  25 mg', 'naproxen enteric coated tablets 500mg', 'ASPIRIN EC 75 MGMS TABLET', 'DICLOFENAC 50MG', 'Diamorphine 10mg powder for solution for injection vials (Teva UK Ltd)', 'CO-CODAMOL tabs 30/500', 'Venlafaxine Mr CAPSULES 150MG', 'NU-SEALS ASPIRIN EC tab 300mg', 'Sodium Valproate  Crushable tablets  100 mg', 'Diclofenac modified release capsule 100mg', 'CODEINE PHOSPHATE 15mg tablets', 'DICLOFENAC SOD CAP 75MG M/R', 'Mirtazapine Tablets 15 mg', 'CARBAMAZEPINE SR tablets 200mg [IVAX]', 'IBUPROFEN SPR 5%', 'CARBAMAZEPINE CHEWABLE TABLETS 200MG', 'Oxycodone Hydrochloride Mr TABS 5MG', 'Celecoxib 100mg cap', 'morphine sulphate tablets 10mg', 'Morphine sulphate 12 60mg Modified-release capsule', 'paracetamol soluble tablet 500mg', 'DICLOFENAC gel 1.16%', 'PRILOCAINE + LIDOCAINE crm 2.5% + 2.5%', 'GABAPENTIN capsules 400mg', 'buprenorphine three-day patch 52.5mcg/hour', 'DEPO-MEDRONE WITH LIDOCAINE (1ML) INJ 40mg', 'Diclofenac Capsules S/R 100 mg', 'Buprenorphine 15micrograms/hour transdermal patches', 'morphine suppository 15mg', 'NAPROXEN EC TAB 250', 'Gabapentin 800mg tablets (Actavis UK Ltd)', 'DICLOFENAC SODIUM TOPICAL SOLUTION 16MG/ML', 'FENTANYL patch 100micrograms/hour [SANDOZ]', 'SODIUM VALPROATE WITH VALPROIC ACID controlledrelease tablet', 'DICLOFENAC modified release capsules 75mg [SANDOZ]', 'Diclofenac sustained release tablets 75mg', 'Paracetamol 1g suppositories', 'FENTANYL patch 50micrograms/hour [SANDOZ]', 'Codeine Phosphate Tablets 30mg', 'ASPIRIN E/C TAB 75mg', 'Paracetamol 500mg capsules (Almus Pharmaceuticals Ltd)', 'LIDOCAINE HCl + ADRENALINE 1:80,000 inj 2%', 'DICLOFENAC GEL 1%', 'Tramadol 50mg capsules (Focus Pharmaceuticals Ltd)', 'CODEINE PHOSPHATE linc paed 3mg/5ml', 'Gabapentin tablets 600mg', 'Ibuprofen 100mg/5ml oral suspension sugar free (Teva UK Ltd)', 'Methadone 50mg/5ml solution for injection ampoules', 'Hydromorphone 24mg modified-release capsules', 'Diamorphine 10mg powder for solution for injection ampoules (A A H Pharmaceuticals Ltd)', 'MEFENAMIC ACID caps 250 mg', 'Tabs Diclofenac 50mg', 'CARBAMAZEPINE TAB 400', 'etoricoxib tablets 90mg', 'NAPROXEN SODIUM TABLETS 275MG', 'CARBAMAZEPINE TABS 200 MG', 'PHOLCODEINE LINCTUS STR', 'Duloxetine CAPS 30mg', 'BUPRENORPHINE SF SUBLINGUAL TABLETS 200MICROGRAMS', 'VENLAFAXINE S/R CAPSULES 150MG-P42 0', 'Oxcarbazepine  Tablets  600 mg', 'Lidocaine  Injection  1 %  20 mg/2 ml ampoule', 'Paracetamol 500mg / Pseudoephedrine hydrochloride 60mg tablets', 'SODIUM VALPROATE CRU TAB 100', 'Pregabalin 150mg capsules (Teva UK Ltd)', 'DICLOFENAC SODIUM dispersible tablet 50mg', 'NAPROXEN 500mg tabs', 'ASPIRIN TABLETS 75 MG', 'PARACETAMOL 500mg', 'Lidocaine 100mg/10ml (1%) solution for injection Sure-Amp...', 'DICLOFENAC modified release tablet 100mg [IVAX]', 'Amitriptyline 10mg / Perphenazine 2mg tablets', 'Oxycodone Hydrochloride Sf Oral LIQ 5MG/5ML', 'TRAMADOL HCL TAB 50', 'Amitriptyline 25mg tablets (A A H Pharmaceuticals Ltd)', 'VENLAFAXINE S/R CAP 75 MG', 'Diclofenac sodium  50 mg enteric coated tablets', 'Mefenamic Acid CAPSULES 250MG', 'PROTRIPTYLINE HCl 10 MG TAB', 'Ibuprofen 400mg caplets (The Boots Company Plc)', 'ASPIRIN 300mg soluble tablets', 'LIDOCAINE HCl inj 0.5%', 'Phenytoin capsules 100mg', 'Lidocaine 0.25% / Aminoacridine 0.025% pastilles', 'Diclofenac Sodium M/R tablets 75mg', 'Venlafaxine tablets 50mg', 'DICLOFENAC SUPP 100MG', 'ORAMORPH (form not specified)', 'Dipyridamole & Aspirin 200 mg + 25 mg modified release capsu', 'Phenytoin sodium capsules 300mg', 'Diclofenac dispersible tablets 50mg', 'Paracetamol with codeine & buclizine tablet', 'Ibuprofen 400mg tablets (IVAX Pharmaceuticals UK Ltd)', 'CARBAMAZEPINE RETARD 200MG TAB', 'Naproxen 500mg gastro-resistant tablets (IVAX Pharmaceuticals UK Ltd)', 'SODIUM VALPROATE SF LIQ 200MG/5ML', 'Paracetamol 500mg / Caffeine 65mg effervescent tablets sugar free', 'VENLAFAXINE S/R CAPSULES 75MG-P42 0', 'Morphine sulfate 15mg suppositories', 'morphine sulfate 24 hour modified release capsules 60mg', 'Lidocaine 200mg/20ml (1%) solution for injection ampoules (Martindale Pharmaceuticals Ltd)', 'PARACETAMOL SOLUBLE TAB 500', 'Venlafaxine Hydrochloride M/R capsules 75mg', 'Mirtazapine Orodispersible tablets 15mg', 'Dihydrocodeine Tartrate  Tablets  40 mg', 'VENLAFAXINE TABS 37.5MG', 'AMITRIPTYLINE CAP 75MG', 'Co-Codamol 30mg/500mg Effervescent Powder Lemon SACH', 'Co-Codamol 30/500 Caplets', 'Paracetamol And Dihydrocodeine Tablets 500 mg + 20 mg', 'IBUPROFEN tabs 200 MG', 'Mefenamic acid 500mg tablets (Almus Pharmaceuticals Ltd)', 'OXYCODONE HYDROCHLORIDE CAPSULES 20MG', 'DEPO-MEDRONE+LIDOCAINE 80mg/20mg injection', 'NAPROXEN 250mg tablets', 'TRAMADOL HCL 50mg capsules', 'Lamotrigine 25mg tablets (A A H Pharmaceuticals Ltd)', 'INDOMETHACIN CAPSULES 25mg', 'Diclofenac Sodium Suppositories 50 mg', 'TRAMADOL HCL TAB 100', 'Pregabalin 200mg capsules (Wockhardt UK Ltd)', 'VENLAFAXINE XL 150MG CAPS', 'Lamotrigine 200mg tablets (Consilient Health Ltd)', 'Diclofenac Sodium  Injection  25 mg/ml  3 ml ampoule', 'Diclofenac sodium & Misoprostol 75 mg + 200 mcg tablets', 'DICLOFENAC SODIUM 75mg m/r cap', 'Codeine phosphate 15 mg / 5 mL linctus', 'Diclofenac Sodium Suppositories 100 mg', 'Diclofenac 10mg dispersible tablets', 'codeine phosphate sugar free linctus 15mg/5ml', 'TRAMADOL orodisp tab 50mg', 'Paracetamol 500mg/5ml oral suspension', 'PARACETAMOL WITH CODEINE PHOSPHATE effervescent', 'IBUPROFEN tablets 400mg [NUCARE]', 'Co-codamol strong tablets 30mg + 500mg', 'OXYCODONE HYDROCHLORIDE MR TABLETS 40MG', 'Diclofenac Sodium SUPPS 25MG', 'IBUPROFEN tabs 200 mg -', 'Fentanyl 50micrograms/hour transdermal patches (Actavis UK Ltd)', 'Tramadol modified release capsule 50mg', 'DICLOFENAC enteric coated tablets 25mg [ALMUS]', 'IBUPROFEN orodisp tab 200mg', 'Ibuprofen 200mg effervescent tablets', 'DICLOFENAC SODIUM inj 25mg/ml', 'Oxcarbazepine 150mg tablets (Mylan Ltd)', 'Gabapentin 300mg capsules (Sandoz Ltd)', 'ASPIRIN DISP 75MG', 'ASPIRIN 75MG', 'Lamotrigine 5mg dispersible tablets sugar free (Actavis UK Ltd)', 'tramadol soluble tablet 50mg', 'ORAMORPH SR tablets 30mg', 'CO-CODAPRIN (codeine & aspirin) sol tab 8mg+500mg', 'INDOMETHACIN slow-release 75mg', 'diclofenac diethylammonium gel 1.16%', 'Depo-Medrone With Lidocaine  Injection  1 ml vial', 'Codeine 60mg tablets (Actavis UK Ltd)', 'Aspirin Dispersible tablets 75mg', 'Carbamazepine 200mg tablets (A A H Pharmaceuticals Ltd)', 'Oxycodone Hydrochloride Mr TABS 80MG', 'Celecoxib 100mg caps', 'MORPHINE SULPHATE mr cap 20mg', 'CARBAMAZEPINE CHEWABLE TABLETS 100MG', 'IBUPROFEN MR TABLETS 800MG', 'Diclofenac Sodium Mr CAPSULES 75MG', 'Diclofenac Sodium Gel 3%', 'Methadone Hydrochloride  Tablets  5 mg', 'diclofenac sodium gel 3%', 'VENLAFAXINE tabs 75 mg', 'Lidocaine Medicated Plaster 5 %', 'Mirtazapine 15mg orodispersible tablets (Mylan Ltd)', 'Aspirin 75mg dispersible tablets (Aspar Pharmaceuticals Ltd)', 'MEFENAMIC ACID 500mg  tabs', 'DICLOFENAC SODIUM+MISOPROSTOL 75mg/200micrograms tablets', 'Diclofenac Sodium Ec TABLETS 25MG', 'Celecoxib Capsules 100mg', 'DICLOFENAC SODIUM 50MG TABS', 'DICLOFENAC EC 50MG', 'Gabapentin 250mg/5ml oral suspension', 'Duloxetine  Gastro-Resistant Capsules  30 mg', 'Tramadol Hydrochloride Soluble Tablets 50 mg', 'CO-CODAMOL CAP 8MG/500MG', 'PARACETAMOL S/F SUS 500mg/5ml', 'Naproxen enteric coated tablets 250mg', 'TRAMADOL modified release tablet 200mg', 'DICLOFENAC TABS DISPERSIBLE 50 MG', 'PARACETAMOL+CODEINE PHOSPHATE', 'PARACETAMOL SOLUBLE TAB 500mg', 'DICLOFENAC SOD TAB E/C 25MG', 'Aspirin 300mg dispersible tablets (A A H Pharmaceuticals Ltd)', 'DICLOFENAC SODIUM TABLETS DICLOFENAC 50MG', 'Diamorphine 10mg powder for solution for injection vials', 'Lidocaine Injection 2 %, 100 mg/5 ml ampoule', 'Venlafaxine modified release capsule 150mg', 'NAPROXEN tabs 250 MG', 'TRAMADOL HYDROCHLORIDE MR TABLETS 50MG', 'PARACETAMOL 500MG/ME', 'IBUPROFEN TABS 800MG SR TAB 800', 'CO-CODAMOL TAB 8MG/500MG', 'PARACETAMOL 500MG/CODEINE 30MG TAB', 'CELECOXIB 100mg', 'Ibuprofen SACH 600MG', 'lidocaine hydrochloride cream 2% w/v', 'CO-CODAMOL disp tabs', 'MEFENAMIC ACID -P42 FORTE 500mg cap', 'Paracetamol 250mg/5ml oral suspension 5ml sachets sugar free', 'DICLOFENAC SODIUM topical soln 1.5%', 'Naproxen Susp 125mg/5ml', 'Mirtazapine 15mg tablets (Teva UK Ltd)', 'Oramorph  Oral Solution  10 mg/5 ml', 'IBUPROFEN CAP 400MG', 'Lidocaine 1% and Chlorhexidine gel', 'Tramadol 50mg capsules (PLIVA Pharma Ltd)', 'SODIUM VALPROATE mr tab 200mg', 'Lidocaine  Injection  2 %, 100 mg/5 ml ampoule', 'Ibuprofen  Capsules  400 mg', 'Mefenamic acid 50mg/5ml oral suspension', 'Nortriptyline And Fluphenazine  Tablets  10 mg + 500 micrograms', 'venlafaxine tablets 37.5mg', 'Carbamazepine TABLETS 100MG', 'OXYCODONE HCl mr tab 15mg', 'Kaolin And Morphine  Mixture  bp', 'IBUPROFEN cream 5%', 'DICLOFENAC 50/MISOPROSTOL 200MCG', 'NAPROXEN SUPP 500MG', 'LIDOCAINE HCl oromucosal spray 2%', 'CO-CODAMOL 8mg/500mg effervescent tablets', 'ASPIRIN EC 75MG', 'AMITRIPTYLINE HYDROCHLORIDE TABLETS 25 MG', 'IBUPROFEN 400MG TABLETS', 'Lidocaine injection 1%', 'CARBAMAZEPINE TABLETS 400MG', 'AMITRIPTYLINE HYDROCHLORIDE tablets 25mg -', 'DIAMORPHINE INJ 5mg', 'Lidocaine 0.05% / Chlorhexidine 0.2% oromucosal spray sug...', 'Cathejell With Lidocaine  Lubricating jelly', 'Pregabalin Capsules 25mg', 'IBUPROFEN TABLETS 800 MG', 'DEEP RELIEF IBUPROFEN gel 5% [MENTHOLA]', 'mefenamic acid capsules 250mg', 'NAPROXEN 250MG TAB', 'Diclofenac suppository 100mg', 'KAOLIN AND MORPHINE MIXTURE', 'DEXTROPROPOXYPHENE HCL /PARACETAMOL TAB 32.5', 'Dihydrocodeine  Elixir  10mg/5 ml', 'TRAMADOL HYDROCHLORIDE 100mg m/r capsules', 'MEXILETINE HYDROCHLORIDE CAPSULES 50MG', 'Aspirin 500mg effervescent tablets sugar free', 'Amitriptyline Hydrochloride  Mixture  Sugar Free 10 mg/5 ml', 'Lidocaine 100mg/10ml (1%) solution for injection Sure-Amp ampoules (AMCo)', 'CODEINE 15mg/5mL linctus', 'codeine phosphate with paracetamol tablets 15mg + 500mg', 'MEFENAMIC ACID CAPSULES 250 MG', 'DICLOFENAC 75 MG INJ', 'Morphine hydrochloride 15mg suppositories', 'AMITRIPTYLINE HCL 10MG/PERPHENAZINE 2MG TAB'"

# Use the SQL query to load the filtered DataFrame
df = spark.sql(f"SELECT eid, issue_date, drug_name FROM {rx_data_name} WHERE drug_name IS NOT NULL AND drug_name IN ({drugs}) OR drug_name LIKE ('Duloxetine 60mg gastro-resistant capsules (Dr Red%')")
df.count()

data_dicts = [
    {"eid": row.eid,
        "issue_date": row.issue_date,
        "drug_name": row.drug_name
    }
    for row in df.rdd.toLocalIterator()
]

# Specify the CSV file path
csv_file_path = "UKB_Rx_analgesics.csv"

# Write data to CSV file
with open(csv_file_path, "w", newline="") as csv_file:
    fieldnames = ["eid", "issue_date", "drug_name"]

    writer = csv.DictWriter(csv_file, fieldnames=fieldnames)

    # Write header
    writer.writeheader()

    # Write rows with progress indication
    for idx, data_dict in enumerate(data_dicts):
        writer.writerow(data_dict)

print("Analgesic/Anti-neuropathic Rx CSV file written successfully.")

## Extract Experience of Pain Questionnaire
This is code to extract the Experience of Pain questionnaire data, which was used to generate the DN4 pain phenotype.

In [ ]:
field_names = ["eid","p120000","p120001","p120002","p120003","p120004","p120005","p120006","p120007",
               "p120008","p120009","p120010","p120011","p120012","p120013","p120014","p120015","p120016","p120017",
               "p120018","p120019","p120020","p120021","p120022","p120023","p120024","p120025","p120026","p120027",
               "p120028","p120029","p120030","p120031","p120032","p120033","p120034","p120035","p120036","p120037",
               "p120038","p120040","p120041","p120042","p120043","p120044","p120045","p120046","p120047",
               "p120048","p120049","p120050","p120051","p120052","p120053","p120054","p120055","p120056","p120057",
               "p120058","p120059","p120060","p120061","p120062","p120063","p120064","p120065","p120066","p120067",
               "p120068","p120069","p120070","p120071","p120072","p120073","p120074","p120075","p120076","p120077",
               "p120078","p120079","p120080","p120081","p120082","p120083","p120084","p120085","p120086","p120087",
               "p120088","p120089","p120090","p120091","p120092","p120093","p120094","p120095","p120096","p120097",
               "p120098","p120099","p120100","p120101","p120102","p120103","p120104","p120105","p120106","p120107",
               "p120108","p120109","p120110","p120111","p120112","p120113","p120114","p120115","p120116","p120117",
               "p120118","p120119","p120120","p120121","p120122","p120123","p120124","p120125","p120126","p120127"]

df = participant.retrieve_fields(names=field_names, coding_values="raw", engine=dxdata.connect())

# Convert DataFrame to a list of dictionaries
data_dicts = [
    {
        "eid": row.eid,
        "p120000": row.p120000,
        "p120001": row.p120001,
        "p120002": row.p120002,
        "p120003": row.p120003,
        "p120004": row.p120004,
        "p120005": row.p120005,
        "p120006": row.p120006,
        "p120007": row.p120007,
        "p120008": row.p120008,
        "p120009": row.p120009,
        "p120010": row.p120010,
        "p120011": row.p120011,
        "p120012": row.p120012,
        "p120013": row.p120013,
        "p120014": row.p120014,
        "p120015": row.p120015,
        "p120016": row.p120016,
        "p120017": row.p120017,
        "p120018": row.p120018,
        "p120019": row.p120019,
        "p120020": row.p120020,
        "p120021": row.p120021,
        "p120022": row.p120022,
        "p120023": row.p120023,
        "p120024": row.p120024,
        "p120025": row.p120025,
        "p120026": row.p120026,
        "p120027": row.p120027,
        "p120028": row.p120028,
        "p120029": row.p120029,
        "p120030": row.p120030,
        "p120031": row.p120031,
        "p120032": row.p120032,
        "p120033": row.p120033,
        "p120034": row.p120034,
        "p120035": row.p120035,
        "p120036": row.p120036,
        "p120037": row.p120037,
        "p120038": row.p120038,
        "p120040": row.p120040,
        "p120041": row.p120041,
        "p120042": row.p120042,
        "p120043": row.p120043,
        "p120044": row.p120044,
        "p120045": row.p120045,
        "p120046": row.p120046,
        "p120047": row.p120047,
        "p120048": row.p120048,
        "p120049": row.p120049,
        "p120050": row.p120050,
        "p120051": row.p120051,
        "p120052": row.p120052,
        "p120053": row.p120053,
        "p120054": row.p120054,
        "p120055": row.p120055,
        "p120056": row.p120056,
        "p120057": row.p120057,
        "p120058": row.p120058,
        "p120059": row.p120059,
        "p120060": row.p120060,
        "p120061": row.p120061,
        "p120062": row.p120062,
        "p120063": row.p120063,
        "p120064": row.p120064,
        "p120065": row.p120065,
        "p120066": row.p120066,
        "p120067": row.p120067,
        "p120068": row.p120068,
        "p120069": row.p120069,
        "p120070": row.p120070,
        "p120071": row.p120071,
        "p120072": row.p120072,
        "p120073": row.p120073,
        "p120074": row.p120074,
        "p120075": row.p120075,
        "p120076": row.p120076,
        "p120077": row.p120077,
        "p120078": row.p120078,
        "p120079": row.p120079,
        "p120080": row.p120080,
        "p120081": row.p120081,
        "p120082": row.p120082,
        "p120083": row.p120083,
        "p120084": row.p120084,
        "p120085": row.p120085,
        "p120086": row.p120086,
        "p120087": row.p120087,
        "p120088": row.p120088,
        "p120089": row.p120089,
        "p120090": row.p120090,
        "p120091": row.p120091,
        "p120092": row.p120092,
        "p120093": row.p120093,
        "p120094": row.p120094,
        "p120095": row.p120095,
        "p120096": row.p120096,
        "p120097": row.p120097,
        "p120098": row.p120098,
        "p120099": row.p120099,
        "p120100": row.p120100,
        "p120101": row.p120101,
        "p120102": row.p120102,
        "p120103": row.p120103,
        "p120104": row.p120104,
        "p120105": row.p120105,
        "p120106": row.p120106,
        "p120107": row.p120107,
        "p120108": row.p120108,
        "p120109": row.p120109,
        "p120110": row.p120110,
        "p120111": row.p120111,
        "p120112": row.p120112,
        "p120113": row.p120113,
        "p120114": row.p120114,
        "p120115": row.p120115,
        "p120116": row.p120116,
        "p120117": row.p120117,
        "p120118": row.p120118,
        "p120119": row.p120119,
        "p120120": row.p120120,
        "p120121": row.p120121,
        "p120122": row.p120122,
        "p120123": row.p120123,
        "p120124": row.p120124,
        "p120125": row.p120125,
        "p120126": row.p120126,
        "p120127": row.p120127
    }
    for row in df.rdd.toLocalIterator()
]

# Specify the CSV file path
csv_file_path = "UKB_EOP.csv"

# Write data to CSV file
with open(csv_file_path, "w", newline="") as csv_file:
    fieldnames = ["eid", "p120000", "p120001", "p120002", "p120003", "p120004", "p120005", "p120006", "p120007",
                  "p120008", "p120009", "p120010", "p120011", "p120012", "p120013", "p120014", "p120015", "p120016", "p120017",
                  "p120018", "p120019", "p120020", "p120021", "p120022", "p120023", "p120024", "p120025", "p120026", "p120027",
                  "p120028", "p120029", "p120030", "p120031", "p120032", "p120033", "p120034", "p120035", "p120036", "p120037",
                  "p120038", "p120040", "p120041", "p120042", "p120043", "p120044", "p120045", "p120046", "p120047",
                  "p120048", "p120049", "p120050", "p120051", "p120052", "p120053", "p120054", "p120055", "p120056", "p120057",
                  "p120058", "p120059", "p120060", "p120061", "p120062", "p120063", "p120064", "p120065", "p120066", "p120067",
                  "p120068", "p120069", "p120070", "p120071", "p120072", "p120073", "p120074", "p120075", "p120076", "p120077",
                  "p120078", "p120079", "p120080", "p120081", "p120082", "p120083", "p120084", "p120085", "p120086", "p120087",
                  "p120088", "p120089", "p120090", "p120091", "p120092", "p120093", "p120094", "p120095", "p120096", "p120097",
                  "p120098", "p120099", "p120100", "p120101", "p120102", "p120103", "p120104", "p120105", "p120106", "p120107",
                  "p120108", "p120109", "p120110", "p120111", "p120112", "p120113", "p120114", "p120115", "p120116", "p120117",
                  "p120118", "p120119", "p120120", "p120121", "p120122", "p120123", "p120124", "p120125", "p120126", "p120127"
                  ]

    writer = csv.DictWriter(csv_file, fieldnames=fieldnames)

    # Write header
    writer.writeheader()

    # Write rows with progress indication
    for idx, data_dict in enumerate(data_dicts):
        writer.writerow(data_dict)

print("Experience of Pain Questionnaire CSV file written successfully.")

## Extract Recruitment Questionnaire Data
This code is to extract recruitment questionnaire data used to generate the chronic pain phenotype

In [ ]:
field_names = ["eid", "p21022", "p31", "p21000_i0", "p6159_i0", "p3799_i0", "p4067_i0", "p3404_i0", "p3571_i0", "p3741_i0", "p3414_i0", "p3773_i0", "p2956_i0", "p6154_i0", "p137_i0"]
df = participant.retrieve_fields(names=field_names, coding_values="raw", engine=dxdata.connect())

# Convert DataFrame to a list of dictionaries
data_dicts = [
    {"eid": row.eid,
        "p21022": row.p21022,
        "p31": row.p31,
        "p21000_i0": row.p21000_i0,
        "p6159_i0": row.p6159_i0,
        "p3799_i0": row.p3799_i0,
        "p4067_i0": row.p4067_i0,
        "p3404_i0": row.p3404_i0,
        "p3571_i0": row.p3571_i0,
        "p3741_i0": row.p3741_i0,
        "p3773_i0": row.p3773_i0,
        "p2956_i0": row.p2956_i0,
        "p6154_i0": row.p6154_i0,
        "p137_i0": row.p137_i0
    }
    for row in df.rdd.toLocalIterator()
]

# Specify the CSV file path
csv_file_path = "UKB_general.csv"

# Write data to CSV file
with open(csv_file_path, "w", newline="") as csv_file:
    fieldnames = ["eid", "p21022", "p31", "p21000_i0", "p6159_i0", "p3799_i0", "p4067_i0", "p3404_i0", "p3571_i0", "p3741_i0", "p3414_i0", "p3773_i0", "p2956_i0", "p6154_i0", "p137_i0"]

    writer = csv.DictWriter(csv_file, fieldnames=fieldnames)

    # Write header
    writer.writeheader()

    # Write rows with progress indication
    for idx, data_dict in enumerate(data_dicts):
        writer.writerow(data_dict)

print("Recruitment Questionnaire CSV file written successfully.")

## Extract Genetic Principal Components
This code extracts UKB's generated genetic principal components

In [ ]:
field_names = ["eid", "p22009_a1", "p22009_a2", "p22009_a3", "p22009_a4", "p22009_a5", "p22009_a6", "p22009_a7", "p22009_a8", "p22009_a9", "p22009_a10", "p22009_a11",
              "p22009_a11", "p22009_a12", "p22009_a13", "p22009_a14", "p22009_a15", "p22009_a16", "p22009_a17", "p22009_a18", "p22009_a19", "p22009_a20"]
df = participant.retrieve_fields(names=field_names, coding_values="raw", engine=dxdata.connect())


# Convert DataFrame to a list of dictionaries
data_dicts = [
    {"eid": row.eid,
     "p22009_a1": row.p22009_a1,
     "p22009_a2": row.p22009_a2,
     "p22009_a3": row.p22009_a3,
     "p22009_a4": row.p22009_a4,
     "p22009_a5": row.p22009_a5,
     "p22009_a6": row.p22009_a6,
     "p22009_a7": row.p22009_a7,
     "p22009_a8": row.p22009_a8,
     "p22009_a9": row.p22009_a9,
     "p22009_a10": row.p22009_a10,
     "p22009_a11": row.p22009_a11,
     "p22009_a12": row.p22009_a12,
     "p22009_a13": row.p22009_a13,
     "p22009_a14": row.p22009_a14,
     "p22009_a15": row.p22009_a15,
     "p22009_a16": row.p22009_a16,
     "p22009_a17": row.p22009_a17,
     "p22009_a18": row.p22009_a18,
     "p22009_a19": row.p22009_a19,
     "p22009_a20": row.p22009_a20        
    }
    for row in df.rdd.toLocalIterator()
]

# Specify the CSV file path
csv_file_path = "UKB_PC.csv"

# Write data to CSV file
with open(csv_file_path, "w", newline="") as csv_file:
    fieldnames = ["eid","p22009_a1", "p22009_a2", "p22009_a3", "p22009_a4", "p22009_a5", "p22009_a6", "p22009_a7", "p22009_a8", "p22009_a9", "p22009_a10", "p22009_a11",
              "p22009_a12", "p22009_a13", "p22009_a14", "p22009_a15", "p22009_a16", "p22009_a17", "p22009_a18", "p22009_a19", "p22009_a20"]

    writer = csv.DictWriter(csv_file, fieldnames=fieldnames)

    # Write header
    writer.writeheader()

    # Write rows with progress indication
    for idx, data_dict in enumerate(data_dicts):
        writer.writerow(data_dict)

print("Principal Components CSV file written successfully.")

### Compress and upload the data to your DNAnexus project

In [ ]:
! gzip *.csv
! dx upload *.csv.gz